# Check fast cholesky updates sufficiently approximate the original

In [1]:
# load packages
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using CSV, DataFrames
using BlockDiagonals

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1342


## Data from gnomAD

In [2]:
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project/group_knockoff_test_data"
mapfile = CSV.read(joinpath(datadir, "Groups_2_127374341_128034347.txt"), DataFrame)
groups = mapfile[!, :group]
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}

3782×3782 Matrix{Float64}:
  1.0           0.00710886    0.364293    …   0.0264997     0.0264099
  0.00710886    1.0           0.0731382       0.0035012     0.002885
  0.364293      0.0731382     1.0             0.0107343     0.0107998
 -0.117632     -0.0432335    -0.0757611      -0.0041054    -0.00329875
  0.122778      0.502735      0.25271        -0.0103248    -0.0114025
  0.124399      0.500263      0.25183     …  -0.0118251    -0.012911
  0.124164      0.500373      0.252304       -0.0123146    -0.0133988
  0.0672066    -0.0298754     0.115745       -0.000961697   0.000611668
 -0.141431     -0.176522     -0.427692       -0.000259765  -0.000154645
 -0.144941     -0.177199     -0.430318       -0.00169021   -0.00145797
 -0.144968     -0.176179     -0.430123    …  -0.00146174   -0.00135189
 -0.155744     -0.0190514    -0.300435       -0.0132113    -0.0131039
 -0.119514     -0.0345581    -0.0729309       0.023471      0.0241919
  ⋮                                       ⋱   ⋮           

## Seems like $\delta$ computed from robust/not-robust matches exactly

In [3]:
p = 1241 # this includes group 263, which is the largest group with 192 members
groups = groups[1:p]
Σ = Σ[1:p, 1:p]

1241×1241 Matrix{Float64}:
  1.0          0.00710886   0.364293    …   0.0316315    0.0860344
  0.00710886   1.0          0.0731382       0.00765639  -0.0201323
  0.364293     0.0731382    1.0             0.0820446   -0.00222317
 -0.117632    -0.0432335   -0.0757611       0.00566909  -0.0379109
  0.122778     0.502735     0.25271         0.0272696   -0.0439615
  0.124399     0.500263     0.25183     …   0.0265975   -0.0436953
  0.124164     0.500373     0.252304        0.0274588   -0.0437401
  0.0672066   -0.0298754    0.115745       -0.0253368   -0.0563336
 -0.141431    -0.176522    -0.427692       -0.0340069   -0.000750859
 -0.144941    -0.177199    -0.430318       -0.0345865   -0.00229511
 -0.144968    -0.176179    -0.430123    …  -0.0339366   -0.00226056
 -0.155744    -0.0190514   -0.300435       -0.0442777    0.00874702
 -0.119514    -0.0345581   -0.0729309      -0.0185629   -0.054856
  ⋮                                     ⋱                ⋮
  0.0329387    0.0170616    0.0909067 

Max entropy group knockoffs

In [12]:
# added try-catch block
@time Snotrubost, _ = solve_s_group(
    Σ, groups, :maxent, 
    m = 1,          # number of knockoffs per variable to generate
    tol = 0.001,    # convergence tolerance
    niter = 1,    # max number of coordinate descent iterations
    robust = false, # whether to use robust cholesky updates
    verbose = true    # whether to print informative intermediate results
);

initial obj = -20744.505546581644
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.000000000153255e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.0000000000152256e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 9.999999998498077e-7)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.0000000003934713e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 9.999999997402645e-7)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.000000000180857e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 9.99999999903683e-7)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.0000000002734129e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 9.999999999460954e-7)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, 1.0000000003655743e-6)
(eigmin(S), eigmin(A)) = (9.999989962672858e-7, -0.005535136086765288)


LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [4]:
# added try-catch block
@time Snotrubost, _ = solve_s_group(
    Σ, groups, :maxent, 
    m = 1,          # number of knockoffs per variable to generate
    tol = 0.001,    # convergence tolerance
    niter = 10,    # max number of coordinate descent iterations
    robust = false, # whether to use robust cholesky updates
    verbose = true    # whether to print informative intermediate results
);

initial obj = -20744.505546581644
Iter 1: obj = -15170.969822386985, δ = 0.8640796161526814, t1 = 9.0, t2 = 17.91, t3 = 0.01
Iter 2: obj = -12775.17854355177, δ = 0.6632487099935324, t1 = 20.25, t2 = 35.92, t3 = 0.02
Iter 3: obj = -10071.785114139919, δ = 0.6393400497466866, t1 = 32.97, t2 = 53.49, t3 = 0.06
Iter 4: obj = -9988.40818666714, δ = 0.20035853809784854, t1 = 45.54, t2 = 71.56, t3 = 0.11
Iter 5: obj = -9973.870487036853, δ = 0.04590715896122788, t1 = 58.29, t2 = 89.3, t3 = 0.15
Iter 6: obj = -9969.044179775456, δ = 0.01325018392062216, t1 = 71.76, t2 = 107.3, t3 = 0.2
Iter 7: obj = -9966.935163918992, δ = 0.0039343423668606325, t1 = 85.52, t2 = 125.05, t3 = 0.24
Iter 8: obj = -9965.489992963834, δ = 0.00233015223749061, t1 = 99.94, t2 = 142.82, t3 = 0.29
Iter 9: obj = -9964.724750015297, δ = 0.0016331750270178079, t1 = 114.25, t2 = 160.74, t3 = 0.34
Iter 10: obj = -9964.3214330175, δ = 0.0013001068642278495, t1 = 128.84, t2 = 178.64, t3 = 0.39
324.972654 seconds (54.16 M all

In [5]:
@time Srobust, _ = solve_s_group(
    Σ, groups, :maxent, 
    m = 1,          # number of knockoffs per variable to generate
    tol = 0.001,    # convergence tolerance
    niter = 10,    # max number of coordinate descent iterations
    robust = true,  # whether to use robust cholesky updates
    verbose=true    # whether to print informative intermediate results
);

initial obj = -20744.505546581644
Iter 1: obj = -15170.969822386392, δ = 0.8640796161526805, t1 = 37.97, t2 = 17.82, t3 = 0.01
Iter 2: obj = -12775.178543551618, δ = 0.6632487099935477, t1 = 138.54, t2 = 35.59, t3 = 0.02
Iter 3: obj = -10071.785114138442, δ = 0.6393400497467019, t1 = 327.56, t2 = 53.89, t3 = 0.1
Iter 4: obj = -9988.408186667075, δ = 0.20035853809785986, t1 = 497.92, t2 = 71.5, t3 = 0.16
Iter 5: obj = -9973.870487036795, δ = 0.045907158961163486, t1 = 675.06, t2 = 89.79, t3 = 0.23
Iter 6: obj = -9969.04417977541, δ = 0.013250183920600067, t1 = 856.51, t2 = 107.85, t3 = 0.29
Iter 7: obj = -9966.93516391874, δ = 0.003934342366859855, t1 = 1043.51, t2 = 125.66, t3 = 0.36
Iter 8: obj = -9965.489992950279, δ = 0.0023301522370145346, t1 = 1237.79, t2 = 143.27, t3 = 0.42
Iter 9: obj = -9964.724750426703, δ = 0.001633175315710238, t1 = 1440.61, t2 = 160.99, t3 = 0.48
Iter 10: obj = -9964.321432403536, δ = 0.0013001056829139248, t1 = 1651.7, t2 = 178.61, t3 = 0.53
1831.516681 se

In [7]:
[vec(Snotrubost) vec(Srobust)]

1540081×2 Matrix{Float64}:
 0.335975    0.335975
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 ⋮           
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.00393945  0.00393945
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.00990279  0.00990279
 0.0         0.0
 0.0         0.0
 0.0263194   0.0263194

MVR group knockoffs

In [13]:
@time Snotrubost, _ = solve_s_group(
    Σ, groups, :mvr, 
    m = 1,          # number of knockoffs per variable to generate
    tol = 0.001,    # convergence tolerance
    niter = 5,    # max number of coordinate descent iterations
    robust = false, # whether to use robust cholesky updates
    verbose = true    # whether to print informative intermediate results
);

Iter 1: δ = 0.2552444081300506, t1 = 9.04, t2 = 49.88, t3 = 0.04
Iter 2: δ = 0.13386076712843895, t1 = 18.06, t2 = 99.68, t3 = 0.08
Iter 3: δ = 0.018951678106012275, t1 = 27.13, t2 = 149.57, t3 = 0.12
Iter 4: δ = 0.008985466294668539, t1 = 36.14, t2 = 199.32, t3 = 0.16
Iter 5: δ = 0.011029077311844033, t1 = 45.14, t2 = 249.19, t3 = 0.19
295.515900 seconds (4.17 M allocations: 238.324 MiB, 0.00% gc time, 0.00% compilation time)


In [14]:
@time Srobust, _ = solve_s_group(
    Σ, groups, :mvr, 
    m = 1,          # number of knockoffs per variable to generate
    tol = 0.001,    # convergence tolerance
    niter = 5,    # max number of coordinate descent iterations
    robust = true,  # whether to use robust cholesky updates
    verbose=true    # whether to print informative intermediate results
);

Iter 1: δ = 0.2552444081300506, t1 = 187.1, t2 = 51.76, t3 = 0.04
Iter 2: δ = 0.13386076712843895, t1 = 373.31, t2 = 103.69, t3 = 0.12
Iter 3: δ = 0.018951678106012275, t1 = 560.04, t2 = 155.59, t3 = 0.18
Iter 4: δ = 0.008985466294668539, t1 = 747.29, t2 = 207.42, t3 = 0.24
Iter 5: δ = 0.011029077311844033, t1 = 935.12, t2 = 259.56, t3 = 0.3
1196.119976 seconds (4.17 M allocations: 238.321 MiB, 0.00% gc time)


In [15]:
[vec(Snotrubost) vec(Srobust)]

1540081×2 Matrix{Float64}:
 0.0451467  0.0451467
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 ⋮          
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.0266443  0.0266443

## Check explicitly that fast vs slow updates give same answer

In [139]:
function lowrankupdate_turbo!(C::Cholesky{T}, v::AbstractVector) where T <: AbstractFloat
    A = C.factors
    n = length(v)
    if size(C, 1) != n
        throw(DimensionMismatch("updating vector must fit size of factorization"))
    end
    # if C.uplo == 'U'
    #     conj!(v)
    # end

    early_term = 0
    idx_start = something(findfirst(!iszero, v))
    for i = idx_start:n

        # Compute Givens rotation
        c, s, r = LinearAlgebra.givensAlgorithm(A[i,i], v[i])
#         @show c, s, r

        # check for early termination
        if abs(s) < 1e-15
            early_term += 1
            if early_term > 10
                println("early break at $i")
                break
            end
        else
            early_term = 0
        end

        # Store new diagonal element
        A[i,i] = r

        # Update remaining elements in row/column
        if C.uplo == 'U'
            for j = i + 1:n
                Aij = A[i,j]
                vj  = v[j]
                A[i,j]  =   c*Aij + s*vj
                v[j]    = -s*Aij + c*vj
            end
        else
            for j = i + 1:n
                Aji = A[j,i]
                vj  = v[j]
                A[j,i]  =   c*Aji + s*vj
                v[j]    = -s*Aji + c*vj
            end
        end
    end
    return C
end

lowrankupdate_turbo! (generic function with 1 method)

First update to cholesky in max entropy

In [141]:
m = 1
s = Knockoffs.solve_equi(Σ, m=m)
L = cholesky(Symmetric((m+1)/m*Σ - Diagonal(s)) + 0.00001I)
δ = 0.45151442767723093
x = zeros(size(Σ, 1))
x[1] = sqrt(abs(δ))

Lcopy = deepcopy(L)
xcopy = deepcopy(x)

@time lowrankupdate!(L, x)
@time lowrankupdate_turbo!(Lcopy, xcopy);

  0.001335 seconds
  0.001010 seconds


In [142]:
all(L.factors .≈ Lcopy.factors)

true

In [143]:
all(x .≈ xcopy)

true

10th Cholesky update

In [146]:
δ = 0.00950789808447346
A = [2.0000099999999956 0.01421771566474002 0.7285853934885521 -0.23526404660190403 0.24555585777525 0.248797917996144 0.24832893421854402 0.134413108119967 -0.28286248895553606 -0.289882743731338 -0.289935847623404 -0.31148734825651203 -0.23902756844356 0.32904661072422203 -0.29056056552241605 0.1059669392656466 -0.292548916732556 -0.290701714914374 -0.44388560313138004 -0.292139748189504 -0.289548328867968 -0.28862965599951607 -0.28908172671505805 -0.28486351039827007 -0.289647915830624 -0.307442237663064 -0.443753050149738 -0.443486324358716 0.32525606701714005 -0.01136228416632392 -0.436768971396264 -0.011354322415162542 -0.3021042486247141 -0.2857983104809261 -0.312640424228454 -0.30514725520871605 -0.309313026436166 -0.08194878159049862 -0.4210759770824721 -0.319471447999278 -0.278611159086956 -0.446249690411152 -0.447696087235946 -0.05910707139588801 -0.32309236784920603 -0.36455086480808 0.07729767926150481 -0.020029341995835403 -0.11515533241960982 -0.3671523962858281 -0.01897624351873668 -0.363908903025608 -0.0610662497820636 -0.450391097637428 -0.44954167968048 0.11125965616186761 -0.3269069133171101 -0.44902710498381 0.342901678427432 0.029803994881861005 0.036537282753455205 0.1515327797090082 -0.07089980365066421 0.07665957102475 -0.32735170321486606 -0.317765535590652 -0.31703040909064406 -0.31956892602277204 -0.32145008228598604 -0.4509584560837361 -0.0157266699876828 -0.06512987854116341 -0.44689068979544405 -0.37987573590117607 -0.01496287037262112 0.208808038224008 0.1834869710471098 -0.015360113628743002 -0.33410730093819807 0.0035374763978456 0.07826907631321121 0.07826907631321121 -0.45105753160426 0.1593946897204906 -0.0646608612800804 -0.0135256684462829 -0.1096437696703938 -0.448171405818764 0.1383746585817472 -0.06658276095457862 -0.06919369778353861 -0.016173461376342905 -0.4466430160012221 -0.32939823714894406 -0.064360895530088 -0.01685424922853526 -0.124262727410146 -0.33204660783667006 0.0032244107056488202 -0.38722448208980204; 0.01421771566474002 2.0000099999999956 0.1462763855838052 -0.0864669361764128 1.005470973633004 1.0005260334307897 1.000746017843454 -0.0597508243992432 -0.35304460071898 -0.354398421934954 -0.352358384554076 -0.038102765605123 -0.0691162416262572 0.632320217710234 -0.355903305524048 0.0475445006280202 -0.355195018495448 -0.354378575491928 -0.1668511762504894 -0.35418378300357 -0.358483322858354 -0.358345238285012 -0.353364880920596 -0.357944419709962 -0.35042935029441 -0.0318977221570812 -0.1648344481210226 -0.1621090537833188 0.646473628652416 0.7599510160064359 -0.1631465659349158 0.7601559444304719 -0.0267325188214662 -0.34744980305127 -0.027727494357774198 -0.0300499496408008 -0.0322497579223544 0.0573593662242916 -0.1697650315128646 -0.01739789300177162 -0.33349133726487 -0.1817683589117658 -0.1832312696478672 0.0608993352752146 0.331657337254574 0.36828217697404 -0.0548767866031736 0.7480789247976299 -0.0406906591284272 0.362455603880094 0.744392773590644 0.368663603381578 0.0693265622061516 -0.18742951454268503 -0.186108471745065 -0.01749531547013788 0.334128151093278 -0.1864007919160652 -0.10974745459061919 0.023670518203683596 -0.0363175731738038 -0.0255623133428966 0.0614915554767196 -0.1479991960307148 0.33365469869051 0.330106419626658 0.325632838562562 0.33147351052797 0.331787574947022 -0.1867223724559976 0.7473303994337461 0.0635306419944254 -0.1910011564899292 -0.19186548525174 0.74414277283952 0.588163018837498 -0.0512014534263878 0.74873962886435 0.339352992650176 -0.01046257864775174 0.0635570708347174 0.0635570708347174 -0.1788160018148424 0.0874953001275356 0.0665320380814448 0.751201427081112 -0.0271573350932232 -0.1817812968986856 -0.0608751360044052 0.073088602990442 0.0719655345204688 0.746523496229862 -0.1866631885113028 0.336561530697284 0.49020926995112 0.75033326657725 -0.01683495726510172 0.337383941757628 0.01741327091353462 -0.1660792662931726; 0.7285853934885521 0.1462763855838052 2.0000099999999943 -0.1515222641699546 0.50542015029383 0.503659254310984 0.504607484257704 0.23148934310957198 -0.855383264924982 -0.860635636174712 -0.860245551532984 -0.600869718389114 -0.145861728023472 0.686958528513734 -0.857294514492362 0.1135798900541096 -0.85621159466852 -0.855069005966556 -0.351830587960694 -0.857361915351376 -0.85582034362388 -0.854235159207492 -0.854107855368162 -0.8558065696797601 -0.85480394514042 -0.5967007880345699 -0.35408158689031 -0.35422361156816 0.672039872958448 -0.0439577160041532 -0.35319550437112 -0.04377135782101119 -0.58613070039247 -0.8529667954061 -0.58832132438355 -0.590728710164618 -0.592341640975694 -0.39417041978307 -0.36297191007250595 -0.563511146658762 -0.8689721782130939 -0.291289588742478 -0.29025783826155 -0.42606056377684 -0.628003392214566 -0.25742905501839 0.1969982318485788 -0.04524912588801759 -0.1078038194120386 -0.26183088787134 -0.0462371486056818 -0.254612591778004 -0.44070155345754397 -0.29150652967704 -0.292275410260638 0.1848549524007316 -0.632327051580724 -0.292211537652602 0.501913338610464 0.0909616508575788 0.057014125749948796 0.252001515283496 -0.435182941980378 -0.262187731486076 -0.63181208616348 -0.61746965585742 -0.615445987215406 -0.61838377478192 -0.621197916824386 -0.28965008662715 -0.04705701220155581 -0.430093953714458 -0.292930422958788 -0.258944989331928 -0.0449532907483468 -0.1521225585574358 -0.06665321321953882 -0.046998362428002795 -0.643929820455668 0.0290801054461942 0.0952076939060244 0.0952076939060244 -0.293924306218962 0.1992696521103332 -0.436986939978444 -0.048178177905395206 -0.320595116464426 -0.294568656708296 -0.0868277235701932 -0.42892315280782595 -0.44029000748224 -0.0487941473169642 -0.291214930177112 -0.640334558063286 -0.30164682005889 -0.04815402792640701 -0.0811733386327222 -0.642895907398978 0.0010025752387468458 -0.259461694289858; -0.23526404660190403 -0.0864669361764128 -0.1515222641699546 2.000009999999996 -0.097521994311026 -0.096872450955024 -0.0973296733375142 -0.0347084879402874 -0.0747177426269536 -0.06886890143048459 -0.07173387151267101 0.132514374703069 0.334925455354046 -0.0774109367349318 -0.07113901319920299 0.026844318017629404 -0.0654672523160364 -0.06691566984508698 0.4839754428944039 -0.0672391364662744 -0.07079398777805619 -0.071119576863897 -0.06692564791354479 -0.0677058646904476 -0.06993905806838718 0.1316345602641472 0.4718783245417081 0.47537003679438394 -0.07597457430601101 -0.051324811105649 0.474774414783656 -0.0512341512553208 0.134841735773137 -0.0688441457803472 0.1338708931006492 0.1315001178464184 0.1347170820482714 -0.1128881871921458 0.4640675739055379 0.13702972820119458 -0.0545417024445404 0.43647728934461005 0.43718382508565 -0.096949879748831 0.11234634392879961 0.29243399223516 -0.0330889463095628 -0.0570854680524212 -0.016697124583651555 0.298608434567632 -0.0562242422631456 0.29695918046447595 -0.09892089318242982 0.431064669725476 0.42329893505519 -0.003971530534057983 0.10906509052134859 0.433329196431518 -0.0789947356735186 -0.01712455101441538 -0.0320993208382518 -0.0380465583103734 -0.10092905385572519 -0.0377453841917496 0.11115327132720301 0.10952514383309561 0.1092993681153854 0.1103218614002486 0.10519132951356099 0.43225358156813 -0.057010694791726 -0.09893638315118242 0.43130920968429187 0.417532007726124 -0.056959033626946 -0.0434739350317378 -0.1497492994290674 -0.057065609627642996 0.11291260562091399 1.5178830414797062e-18 0.025931021438649594 0.025931021438649594 0.42885224998999993 0.026519938879124398 -0.09734176080594861 -0.0565935214556978 -0.0693476318335304 0.416585268652806 -0.0281720937682352 -0.1002801230294158 -0.0967403331521768 -0.056307809007697995 0.425183371144186 0.1097243996781948 -0.032331302492839 -0.0561392302130172 -7.300065243528303e-5 0.1103117180723438 -0.0374117153258862 0.389404461035444; 0.24555585777525 1.005470973633004 0.50542015029383 -0.097521994311026 2.000009999999988 1.98490961869331 1.9819128872617937 -0.10753378581610061 -0.727185289216454 -0.7199584973665 -0.722943683068332 0.09139333767874241 -0.11863434614631942 1.47994827009095 -0.72325697154883 0.0588072310406062 -0.723411673978638 -0.7243482616261601 -0.242332875789744 -0.7227100126285542 -0.722274221124492 -0.723847540421398 -0.720322077292406 -0.72275693791161 -0.71935729984299 0.09162489079821282 -0.241296892368356 -0.23881210325446403 1.45971144895913 1.0631543026643022 -0.23857026581518798 1.0634522614872581 0.10301480645695182 -0.719333020925696 0.09598188843432182 0.09772783756874401 0.09342449720273202 0.21806332662126598 -0.239452366779474 0.1063094586071652 -0.6980076973115401 -0.255863035142746 -0.255996455012822 0.22544630676201 0.61324229125987 0.5158860013880741 -0.010684483526619541 1.061968674815038 0.307673878076684 0.513494499305726 1.0552168498510082 0.516734326781856 0.20877076661038402 -0.259088066148552 -0.25947365021957003 -0.033855195875345405 0.613770041032582 -0.257266082718288 -0.206335522650284 0.006452945567395018 -0.05448413735998281 -0.12470084005590881 0.19929529838261997 -0.1874560585939726 0.6149713035308161 0.609094282263652 0.607904148509878 0.608219482697056 0.608159081063878 -0.261671828140146 1.060161872580994 0.22575441492882797 -0.262246401987818 -0.253440501116064 1.057162086099642 0.572445899672658 0.0775583736703264 1.061873296771698 0.588232745081964 -0.01701923375174028 0.058744213320829 0.058744213320829 -0.2592598180874 0.1238895134355204 0.207550851054416 1.061308064455734 -0.23771974550859004 -0.261972567381606 -0.061255874080214 0.20613085090873398 0.209940926902878 1.058999386783456 -0.260352236871198 0.5864139687516101 0.805633630741486 1.05555549172241 -0.0632336227436086 0.584991201147052 0.0168377906904345 -0.219905256884248; 0.248797917996144 1.0005260334307897 0.503659254310984 -0.096872450955024 1.98490961869331 2.000009999999996 1.996483839113676 -0.1082943245959134 -0.7264829796367419 -0.719025311136804 -0.722177050289686 0.0917172106540826 -0.1178637877251296 1.478434193244402 -0.72250605430187 0.0585292974040912 -0.7226633956447801 -0.723073740893584 -0.24130431876957004 -0.721962969911538 -0.720859807663962 -0.722748410431686 -0.719582499689604 -0.72178142873288 -0.7195407225147121 0.0918970939296164 -0.240255926645644 -0.23775680196484 1.459587697907676 1.0569541603135681 -0.237519503983682 1.057252288455922 0.1029406286053528 -0.718690857872638 0.09623097973067202 0.09805698764765261 0.09371005500418839 0.21775982071044203 -0.238346701838268 0.10646789375417759 -0.6967069039606919 -0.254683373682142 -0.254821544594242 0.225005450559646 0.611283134043266 0.5126164613625301 -0.011679634289869519 1.0558610229558103 0.302057129805978 0.51020081705564 1.0489490134178479 0.513461631013042 0.20940321194432002 -0.257917233790456 -0.258172688233436 -0.03328844270121141 0.610793962568172 -0.255955170653256 -0.206379594285396 -0.004478590773016482 -0.05458462195529 -0.12101474931083 0.19985817699509117 -0.1864364959291206 0.611444677176006 0.606819283735428 0.605648834176398 0.605978593756508 0.605916923242928 -0.260512880588436 1.054033029975662 0.225135597207618 -0.261087523136404 -0.252296705898104 1.050910991530852 0.577528224667252 0.076939184260247 1.0557790504796922 0.586646246420846 -0.01710575016007932 0.0584883027230254 0.0584883027230254 -0.258071335626892 0.1233496370640692 0.20819088721302803 1.05515828468904 -0.238629060598566 -0.260791757226894 -0.0606160924503232 0.20694732976684999 0.210583221327394 1.052813386456216 -0.259184057465806 0.584862181320432 0.7986172294709502 1.04929149028255 -0.0660514412792494 0.582908079287208 0.01613802078865866 -0.219496730966528; 0.24832893421854402 1.000746017843454 0.504607484257704 -0.0973296733375142 1.9819128872617937 1.996483839113676 2.000009999999996 -0.10867419765488759 -0.7247287132751319 -0.717284950975282 -0.720449694406834 0.09089807698687599 -0.1174812532345862 1.475354938136672 -0.7210743431278821 0.05875418463201381 -0.720923862618878 -0.721332691833698 -0.24238529159243194 -0.72022697284671 -0.7194213513138159 -0.721004812547926 -0.717824246984508 -0.7202809023414899 -0.7178267272827878 0.091087617955935 -0.24134073527276 -0.23884368794393 1.457337053533252 1.056254686052346 -0.238604989643422 1.056553495932786 0.1019220771976186 -0.717235762579834 0.09544762160034917 0.0972462458577638 0.092911992929863 0.217553173958836 -0.239452530917954 0.1057491913529856 -0.694841411081272 -0.25584493085417803 -0.255981390833272 0.22462173562297197 0.6099339237342741 0.511220808666218 -0.01199290571961476 1.055150263031872 0.301585326367216 0.5088068486104599 1.04825387094736 0.5120638889707061 0.209726810839316 -0.25907857102664 -0.259333261316358 -0.0335996477453926 0.609607555859912 -0.257117233167734 -0.204892521492684 -0.00451528112450476 -0.0545909227593894 -0.121081992269888 0.1999485499729938 -0.1859646257819202 0.610258016140776 0.605848775845572 0.603799108965848 0.604989117817662 0.604933006163428 -0.261673538392932 1.05332434443566 0.224976744171208 -0.262249082993938 -0.25327132548501996 1.050206099721468 0.578377099347008 0.0770853284712352 1.055126344207052 0.586148421973928 -0.01709623848189614 0.0587032419682022 0.0587032419682022 -0.259240136532618 0.1238030123523062 0.20850871861881 1.054453544964748 -0.234900407280784 -0.261961677728648 -0.0570105583201274 0.207294509810786 0.210904027398708 1.052111173415842 -0.260345815139686 0.584003973852642 0.797665402786024 1.04865492730522 -0.0663286960281528 0.582397629844414 0.01590218371426646 -0.220125444507318; 0.134413108119967 -0.0597508243992432 0.23148934310957198 -0.0347084879402874 -0.10753378581610061 -0.1082943245959134 -0.10867419765488759 2.000009999999996 -0.1264063183901438 -0.12464327325234639 -0.1275002631781986 -0.205435774882312 -0.0326921940270026 -0.080160080261325 -0.12695634761583657 0.0005818736630584832 -0.1267828354505432 -0.126435604496429 -0.052767493002809605 -0.12255668773641121 -0.1274773933752134 -0.12459213311412558 -0.125651975595835 -0.12192219848662961 -0.1270942747346228 -0.204619514265646 -0.05376395026729619 -0.053845814304047404 -0.0783825996343408 -0.06975410721274641 -0.05342406315508961 -0.06967484736960299 -0.2045690898784 -0.1259624697431376 -0.202881467525808 -0.20437649418971 -0.20335684781706798 -0.1713364168616992 -0.05685136577995299 -0.19812798251491037 -0.12039212090067541 -0.059323096081023395 -0.059121221372574 -0.16845407599912443 -0.24314405884295004 -0.0954818363281962 -0.0288528955738182 -0.0704655502104354 -0.0277722132355774 -0.09544473180695921 -0.0699038697047052 -0.09562169269607 -0.1679474375552618 -0.0599990240274266 -0.05996455300722381 -0.029750627137549793 -0.24575248875851796 -0.058965954391791 0.179382101660847 0.00734618789318976 -0.0149839969297924 -0.016230305903146044 -0.1681775346798106 -0.058161037963753 -0.24259369326353597 -0.241117809520786 -0.239732465134794 -0.23843922117023003 -0.236913522280266 -0.05923235621834301 -0.0704185897003638 -0.1702060031014684 -0.0593570452785034 -0.0369152568542118 -0.06915958542380822 -0.011297129585522397 0.1890356650125568 -0.06913323041204539 -0.24175813471905 -0.005004012410249099 -0.024948456575417 -0.024948456575417 -0.0585576610402596 0.0252890019780794 -0.1682870475438822 -0.07008924414831721 -0.0427362268329288 -0.060379462873831204 -0.00466386764147392 -0.1654087788765672 -0.16780742046147978 -0.0698825467395196 -0.0594918259287324 -0.238732454231068 -0.1488291023275346 -0.069824084170376 -0.0267016352728282 -0.23413078891196198 0.0269672535774878 -0.05577946427732379; -0.28286248895553606 -0.35304460071898 -0.855383264924982 -0.0747177426269536 -0.727185289216454 -0.7264829796367419 -0.7247287132751319 -0.1264063183901438 2.000009999999996 1.9791771004206098 1.9807190329566118 1.3696057594144577 0.350609915862604 -0.5545163970501059 1.9802528861720177 0.039423941400903606 1.973105065186294 1.9747322688834958 -0.44704775116185597 1.9741507107477156 1.970852117839894 1.970279952685368 1.9680550915779738 1.9730889255050217 1.9648322439662622 1.36401561670662 -0.4471496399048199 -0.447782939145448 -0.547545194750862 -0.41183122521200805 -0.44488124914528393 -0.411738335704006 1.345942975310518 1.963769423098288 1.349509070682434 1.360809171789828 1.363116450637308 1.56089694851138 -0.4064815442553801 1.308202596472538 1.9030039766782259 -0.31681072367447394 -0.31607489547591605 1.50206174164777 1.177023353782174 -0.521559623208504 0.14972337622354298 -0.4076531153904239 -0.1116136671776792 -0.52330083641751 -0.40831491751908 -0.519533137913354 1.516262926875128 -0.31337127175877594 -0.310849878573794 -0.13619192715455997 1.1847236034075557 -0.31464105774674794 -0.38400311625056993 -0.07788211463879119 -0.04542717715423679 -0.1527055084696822 1.5114240455479118 0.5082398199415519 1.184659055534008 1.158391705186126 1.163973762120228 1.1634584977486158 1.164889028949652 -0.3112125123443479 -0.407046431771424 1.507601816811292 -0.30948503721423803 -0.278836456005206 -0.40459540992754195 -0.227860331635208 1.20607223822249 -0.408223926389768 1.191533183571474 -0.0331469271132328 0.026550674812263204 0.026550674812263204 -0.312234658379028 0.12799820384342558 1.51590296816022 -0.406755861875524 0.595496064083608 -0.303326692980216 0.1972451235934018 1.4903536087235763 1.5150653157754357 -0.405682180727742 -0.31161763852396196 1.191516932349192 0.5862623300867079 -0.404900117588566 0.18687715764013618 1.1894014066549057 0.01721751328352808 -0.28217547053008596; -0.289882743731338 -0.354398421934954 -0.860635636174712 -0.06886890143048459 -0.7199584973665 -0.719025311136804 -0.717284950975282 -0.12464327325234639 1.9791771004206098 2.000009999999996 1.987288558065944 1.370889469391356 0.353619841647842 -0.556798592899362 1.984313369480994 0.0399967711251708 1.9777890658398583 1.979418912942312 -0.45021353794187396 1.97885971778983 1.9748999711752238 1.974883924053598 1.972878667884482 1.977168410117902 1.9697233258594438 1.365850783840928 -0.45028769899669807 -0.450922701167988 -0.5497378095782182 -0.409507511172544 -0.44801767975320006 -0.409412884841996 1.346595850708154 1.967006604496712 1.350320246852424 1.36203686925415 1.364907254788154 1.563794014527774 -0.409634549957848 1.30869965924334 1.9052476457522503 -0.32226107918364794 -0.32152368716353197 1.508893845912564 1.182184024266976 -0.523675293732576 0.1491154497488192 -0.40451080574453996 -0.112332169529549 -0.524862265806274 -0.40595171713560796 -0.522660411497212 1.521725327343652 -0.318800423980796 -0.31627545586833 -0.1369702835663832 1.188274273653616 -0.320041985278646 -0.385386879944946 -0.08195557342279498 -0.0454948546327726 -0.1527599079287876 1.516185038801612 0.506945715538876 1.18801680298954 1.161779804624866 1.16735674388242 1.1668050252904678 1.167713212384028 -0.316644792851718 -0.40389857597147 1.512334779917522 -0.31491838867449 -0.284294915537444 -0.403053113118334 -0.224537767080512 1.209117031648736 -0.405086595903802 1.1952919731184737 -0.0331642110245982 0.027095401570100403 0.027095401570100403 -0.31767032625126596 0.1291847214148318 1.5200967318501841 -0.404453651935646 0.59344143449037 -0.308762122926318 0.19962408505252 1.493377250587152 1.520218345170022 -0.402504533943598 -0.316965567438228 1.195457942677812 0.590609754593264 -0.40256596658829 0.1883113005860716 1.193318904772864 0.0197610053166223 -0.285656203970554; -0.289935847623404 -0.352358384554076 -0.860245551532984 -0.07173387151267101 -0.722943683068332 -0.722177050289686 -0.720449694406834 -0.1275002631781986 1.9807190329566118 1.987288558065944 2.000009999999996 1.373713067593874 0.351912953337646 -0.5599299839562022 1.987181018955186 0.0405363926236588 1.9803742918343274 1.98200061744815 -0.451363254530908 1.9814001561364338 1.9772702621350757 1.9772607501913537 1.9737798122220942 1.980340220950888 1.971689584465302 1.367122624726314 -0.45147968853955395 -0.45211527277953395 -0.55290419217512 -0.4098472138049639 -0.449208772823586 -0.40975068640334394 1.345742518240382 1.9692498903217783 1.350212611124154 1.362163727366326 1.365048592578264 1.564191184577488 -0.41098117901873 1.30916178965244 1.90675157108853 -0.321530610851974 -0.320787845580374 1.508136758306766 1.181513073922212 -0.523990604021204 0.14342083127081823 -0.40574136942229805 -0.1128379813996048 -0.525694043576042 -0.4063016107243841 -0.521972474128392 1.52307119707167 -0.318110602693056 -0.315590767467548 -0.1374761892769756 1.189321330764572 -0.31933614760408 -0.3857188594095 -0.0820284356616558 -0.045601105254602196 -0.15301128547459342 1.51557712240322 0.50793919653215 1.188742998017144 1.162299694535892 1.167855888428198 1.167265765474092 1.168178774280006 -0.315837497157374 -0.405129364532288 1.51165926942248 -0.31423925201511 -0.28256278891634 -0.402666233684388 -0.225412135051478 1.208802795647704 -0.406314577890976 1.196612151219816 -0.0330925658669494 0.0276386470539806 0.0276386470539806 -0.316928890882602 0.1267779227264464 1.5203459496447522 -0.4048158324524 0.595473952687648 -0.30813203653595 0.1982277924745446 1.49447918555575 1.520773375632126 -0.403724554028012 -0.316291566971188 1.196228074013144 0.589978010355654 -0.402951924961782 0.1937048435259516 1.193510774492518 0.0216194375699942 -0.28536830894634; -0.31148734825651203 -0.038102765605123 -0.600869718389114 0.132514374703069 0.09139333767874241 0.0917172106540826 0.09089807698687599 -0.205435774882312 1.3696057594144577 1.370889469391356 1.373713067593874 2.000009999999996 0.242805057228712 0.60670755669249 1.371290271782084 0.08452261933518398 1.36542732280257 1.364318404590128 0.4709656506216579 1.365233093937576 1.3651360923897802 1.3639341439742683 1.36246422258226 1.366894643307562 1.359054567003074 1.9757305090241841 0.470800040360654 0.4705039759471401 0.5968791830559801 -0.593959658107888 0.4678711951247441 -0.593704727883182 1.9636149690910498 1.3567092577818503 1.9603285737680363 1.97437805017835 1.9728223164457959 1.697082479061854 0.48556816831690197 1.92609296521871 1.316672435838044 0.506710192578968 0.5068504004062879 1.652000454999252 1.7163592207488363 0.0036010787810237532 0.0988318659600848 -0.581864467898318 -0.16635957492764963 0.005542760826919874 -0.58304610805676 0.00401002990354804 1.648860496255838 0.512419706189088 0.513517654126874 -0.18449100747797242 1.72297740704557 0.5090987530703441 -0.641508137826268 -0.0896639118592702 -0.0951108535323732 -0.266311778919864 1.639768932041908 0.353111508646924 1.722360274648468 1.6892851833953342 1.6913382188043702 1.692641817741188 1.693174079808942 0.510914340548718 -0.5817445776249981 1.6529406050395217 0.5112199494371541 0.4590990119172861 -0.5781230420396462 -0.331051450864294 1.227601172860718 -0.583341607748366 1.708173251836652 -0.0431530143336506 0.0808151829946312 0.0808151829946312 0.5074776922479922 0.217891164823698 1.647785171008176 -0.5824661127662 0.397461191499696 0.5082072535826863 0.1422527976517972 1.624219460499614 1.6500240973017757 -0.579511171480682 0.5118355702906461 1.70902571950142 0.8615903627838781 -0.579011428933174 0.1274217421708584 1.70651883667335 -0.021290877527545014 0.45098803299310797; -0.23902756844356 -0.0691162416262572 -0.145861728023472 0.334925455354046 -0.1186343461463194 -0.11786378772512959 -0.11748125323458622 -0.0326921940270026 0.350609915862604 0.353619841647842 0.351912953337646 0.242805057228712 2.00000999999999 -0.0889415132158322 0.34987176024503397 0.030942462030522602 0.353622211157992 0.353113789477138 -0.08436078986625378 0.35041947985914007 0.353477749403538 0.351761364102252 0.34444598558779 0.345913196269102 0.347862704967532 0.23886231983294604 -0.08471319922021639 -0.08479862857834121 -0.0871683867025626 -0.06306632786653478 -0.0837765051625792 -0.0624940880599786 0.239947092915096 0.34686621579802807 0.24075519878859994 0.24282584322667 0.24751188812303 0.28926691151833006 -0.0847756846001724 0.25147608029697993 0.360939064747266 -0.07595567014125457 -0.07568388835672499 0.288794348277248 0.219070864598646 -0.10947429338126181 -0.03843470904300061 -0.0683812209015642 -0.0365209670621566 -0.11019496157075659 -0.067417854607932 -0.1104309399585196 0.292666590611448 -0.0799333406593606 -0.0804514565458078 -0.039630570004769994 0.219264558860492 -0.08204922268112082 -0.081474639480053 -5.7499631792235634e-5 -0.01590595098552282 -0.0408447332422288 0.288824034702926 0.02071017598497 0.21818932517904 0.212105922504278 0.211270018689816 0.22131901773298995 0.218831263813698 -0.0787740922187014 -0.0719827941765472 0.286823753226606 -0.08069572664188421 -0.0695685042233066 -0.067760771795004 -0.039295954476892 0.237172734701482 -0.0683551786130858 0.223840149964688 -0.00377962508749056 0.029744010482333 0.029744010482333 -0.0811184930178682 0.0380296429012654 0.291536959020652 -0.0678233775598626 -0.0856499496161806 -0.0814515911889314 -0.03522597314541381 0.291444951480594 0.292027926821088 -0.067501007662499 -0.08332556726977058 0.22004882313752 0.08001625332639282 -0.0668334951093194 -0.021090034233634793 0.217062976756358 0.0020058202244751604 -0.0708779019326876; 0.32904661072422203 0.632320217710234 0.686958528513734 -0.0774109367349318 1.47994827009095 1.478434193244402 1.475354938136672 -0.080160080261325 -0.5545163970501059 -0.556798592899362 -0.5599299839562022 0.60670755669249 -0.0889415132158322 2.0000099999999903 -0.5592837533697979 0.04533187260715819 -0.5588579988249621 -0.5614042196308201 -0.18330390166821264 -0.560327393351028 -0.5569810800273979 -0.557456959883166 -0.5565990692877142 -0.558525612815206 -0.555649902314328 0.607982103059712 -0.18315497208295023 -0.18321358937820537 1.96466317858656 -0.18844508075570843 -0.18217458562973188 -0.18817048074551612 0.620686524052848 -0.5555090261030078 0.6084571015469918 0.61485911416559 0.6082581216019041 0.697584735392398 -0.1811730908192598 0.615258232735796 -0.53952294413035 -0.1938703925675268 -0.19290774672291056 0.695387567378712 0.527670968043846 -0.2697584229021238 -0.03593140853487279 -0.17708119034156794 -0.03993749883736184 -0.26768430482922184 -0.17805920620100318 -0.266971400388754 0.67183385139202 -0.19306441262096674 -0.19319515474121524 -0.053456133722767404 0.53094376971942 -0.19221240520951577 -0.17140539673783 0.0235514767255218 -0.04348787791849519 -0.1204585937055592 0.66035613151298 -0.1358372152694424 0.530922465531174 0.5217627708772501 0.5232195531246221 0.523031714735358 0.523951041913848 -0.19538824246382042 -0.17624683572703181 0.694424569236658 -0.19661745894492078 -0.193251593287642 -0.17570042814818837 -0.1056542356365848 0.567626573101854 -0.17701549145181045 0.49690234778935793 -0.01188416033627098 0.040500472622407 0.040500472622407 -0.19271911708375483 0.09545846290740198 0.674259135632548 -0.17716023501258177 -0.1791046082143352 -0.19595209721529183 -0.036919083809066 0.667568255092486 0.673678160222254 -0.17651723542974357 -0.19399255889168376 0.499856709287212 0.5513900546005861 -0.17768494245422434 -0.0499656224258618 0.49890449858229 0.01868054853043164 -0.17573070320781242; -0.29056056552241605 -0.355903305524048 -0.857294514492362 -0.07113901319920299 -0.72325697154883 -0.72250605430187 -0.7210743431278821 -0.12695634761583657 1.9802528861720177 1.984313369480994 1.987181018955186 1.371290271782084 0.349871760245034 -0.5592837533697979 2.000009999999992 0.040234642067578597 1.982956016138132 1.984544505292966 -0.45189259890149197 1.9847829845631317 1.9815033708273115 1.9814924979343884 1.9771664798166997 1.9829107345957997 1.9756635726150762 1.368982757125216 -0.452006071251392 -0.4526398616959079 -0.5522893015374503 -0.41132369880482594 -0.4497469384774939 -0.41122806308191395 1.350298571128412 1.972850272119284 1.35425087016856 1.365179056204488 1.368050224515294 1.567872776064418 -0.41145757874942207 1.3127686213397423 1.910593354990506 -0.321924334139112 -0.32118556547742805 1.5100565977232923 1.182535873399396 -0.5260386730136961 0.14829487529205923 -0.407195904986052 -0.11244616811684659 -0.527909833413508 -0.407783858804142 -0.5240243994023921 1.5241434320609877 -0.318486138507776 -0.31596746502386 -0.1370484606599576 1.189574097243268 -0.31971917525662 -0.383369790023848 -0.0817872517236306 -0.045414096817148204 -0.1522150335850972 1.52074538655041 0.501296511002728 1.189525377831622 1.163481396310716 1.16901867096029 1.16846858424325 1.16937299691506 -0.316335934322078 -0.407710443749344 1.515851990551322 -0.31461603960149387 -0.282878347265994 -0.405261562627326 -0.226776681418948 1.212424026793272 -0.408887964795616 1.1973226323891821 -0.03295668150294901 0.027354045436972 0.027354045436972 -0.317373569554154 0.1428465741712368 1.522504680650362 -0.406280949123858 0.597530702524484 -0.309697017371476 0.197843409189512 1.496830949509252 1.522925701018232 -0.40632460748536 -0.31666111630402 1.197072099595568 0.58768702403468 -0.404405298504174 0.1973190383158844 1.194303012527314 0.01618769324443572 -0.284894538351428; 0.1059669392656466 0.0475445006280202 0.1135798900541096 0.026844318017629404 0.0588072310406062 0.0585292974040912 0.05875418463201381 0.0005818736630584832 0.039423941400903606 0.0399967711251708 0.0405363926236588 0.08452261933518398 0.0309424620305226 0.04533187260715819 0.040234642067578597 2.000010000000056 0.053919502987649984 0.05372792693094381 0.029161908625946406 0.05387938682095899 0.050410865246845786 0.05381692967742039 0.0531923117653374 0.0534972145812656 0.05067423677846081 0.0931162034655666 0.028086743631676594 0.028134682936374583 0.04427555305681177 0.0286858619848384 0.029113686467974187 0.02864006471129839 0.0929405312037584 0.04870675881600501 0.09650820527144459 0.096959829337929 0.09694447878571341 0.079608407980375 0.031177195844025812 0.08781051100378 0.046997396789937404 0.0354072345677488 0.035289806639021 0.0761796801858742 0.10923470242472919 0.0417430165821088 0.0168776141529762 0.029175477270882 0.016245464791048124 0.041702421130835596 0.028737819431945005 0.0418061437059964 0.0760134538508796 0.0357961568592504 0.0357757304263448 0.0174027457435593 0.110474221910595 0.034324237982611 0.001347164627409165 0.01354864426646 0.0110885233669381 0.0374487824558946 0.0761594985889974 -0.0008258516885011145 0.1105663240213984 0.105569939859791 0.1046315453770322 0.1067859474479324 0.1005059136126884 0.035801414024599 0.0291381593144674 0.0771775470821854 0.035874591204189205 0.030387944741091615 0.0291137051973266 0.030671270747427197 0.1024309067311692 0.0291650002162096 0.108831442391778 0.19251420962638097 1.8001405844262839 1.8001405844262839 0.028120812073082 0.8301106761471821 0.0762389907652934 0.0289286227162354 0.01573554075655426 0.0282805088594576 -0.005679960334690259 0.07563355806104957 0.0759681917408366 0.0287854893129932 0.03595056528119139 0.1087511170941968 0.06105827930297221 0.028702293245968003 -0.024457371719871602 0.1095895288551842 0.020925224259080805 0.03346623902027839; -0.292548916732556 -0.355195018495448 -0.85621159466852 -0.0654672523160364 -0.723411673978638 -0.7226633956447801 -0.720923862618878 -0.1267828354505432 1.973105065186294 1.9777890658398583 1.9803742918343274 1.36542732280257 0.353622211157992 -0.5588579988249621 1.982956016138132 0.053919502987649984 2.0000099999999956 1.9900833716208037 -0.44947498543885794 1.986842774379502 1.9812267402769301 1.98066223995012 1.9771490370760683 1.982089299779424 1.9754635522142 1.370013495676044 -0.449584907635884 -0.45022001892251395 -0.553384800422192 -0.41308500691964595 -0.44731220367055624 -0.41298994203104 1.3508055482619379 1.972820415925222 1.3545385427527958 1.36674257753025 1.36906978602366 1.568111033254488 -0.408988181200104 1.314142064375778 1.910041459563558 -0.319511168071684 -0.31877085130757804 1.511526130064112 1.183424019230948 -0.52456591053304 0.14427486982261087 -0.40893144013150595 -0.11237041168375742 -0.5264390851669459 -0.40954073389994394 -0.522544320621852 1.523966991439826 -0.31608128504193 -0.31355901573649 -0.13699928675832604 1.190695704691792 -0.317310071383474 -0.380536070429888 -0.0816420801712328 -0.045253802783767 -0.1521648736965052 1.518434200014072 0.505501206574426 1.189986677436286 1.164493062420836 1.1695122770330661 1.169505640769454 1.169877718558668 -0.313924324736136 -0.40944884942056 1.516221107372078 -0.31235837236554603 -0.281043754772232 -0.407001172719588 -0.228668498192638 1.212517529448816 -0.410625289659844 1.197507550114146 -0.0316108067608586 0.0475164587115662 0.0475164587115662 -0.314965951148054 0.1475959039846078 1.522180193282318 -0.409149812718966 0.594988521758998 -0.306066677253456 0.198539383507327 1.4958574815117218 1.522599185898856 -0.408233530110672 -0.314136303651148 1.197105495661432 0.59108965781002 -0.407290002699554 0.200996009235316 1.194471220820864 0.0228598306337352 -0.284429639590232; -0.290701714914374 -0.354378575491928 -0.855069005966556 -0.06691566984508698 -0.7243482616261601 -0.723073740893584 -0.721332691833698 -0.126435604496429 1.9747322688834958 1.979418912942312 1.98200061744815 1.364318404590128 0.353113789477138 -0.5614042196308201 1.984544505292966 0.05372792693094381 1.9900833716208037 2.0000099999999956 -0.45239066643355796 1.9882282889111405 1.9823351820355357 1.981739575630046 1.9782281460411222 1.98315685888771 1.9762372328878717 1.368053426517016 -0.452500054375786 -0.45313281555184004 -0.5553252489600021 -0.40954939488868 -0.4502485392948559 -0.40945500395817797 1.3487402209962178 1.973909181021378 1.352458477176832 1.3646725455119701 1.3669930609989798 1.568220851378554 -0.413096828243522 1.312740953993644 1.9116749833632838 -0.323297313740664 -0.322562986493032 1.51131430800027 1.1834028866039439 -0.52509589469435 0.14891188292111798 -0.40541226294366794 -0.11210544116828079 -0.5269696366534179 -0.406001166840034 -0.523077464668382 1.5243680623453402 -0.319826204929504 -0.317304843121738 -0.1366999221582218 1.189803893926414 -0.321147278629838 -0.380069368056266 -0.0815417985034014 -0.04519185977997521 -0.1495052327650618 1.518862994419782 0.508968440506602 1.189616321650208 1.164126069794912 1.1691330767301982 1.168614340634656 1.169513793520374 -0.317675863516948 -0.404802483094786 1.51500594185277 -0.31630473800934206 -0.28279600431095403 -0.40234256741699 -0.226147757521846 1.212762888823296 -0.405985279758228 1.197538486725512 -0.0315812982612354 0.0473460807734466 0.0473460807734466 -0.318690602329012 0.1471879024061932 1.5220478377386981 -0.404500219117898 0.593508205496476 -0.309795894635648 0.1985221399112734 1.495617098387878 1.522463534674872 -0.403416974860448 -0.317989266234812 1.197267581844066 0.591661107795694 -0.402651951164144 0.201045650669792 1.19451209867197 0.0205651561207762 -0.286771066839516; -0.44388560313138004 -0.1668511762504894 -0.351830587960694 0.4839754428944039 -0.242332875789744 -0.24130431876957004 -0.24238529159243194 -0.052767493002809605 -0.44704775116185597 -0.45021353794187396 -0.451363254530908 0.4709656506216579 -0.0843607898662538 -0.18330390166821264 -0.45189259890149197 0.029161908625946406 -0.44947498543885794 -0.45239066643355796 2.000009999999996 -0.4524304588575681 -0.45074681472201406 -0.451347805034348 -0.4490392992806879 -0.44940441522521 -0.4521462140052901 0.474090885745666 1.9913170014855177 1.992026089716718 -0.18324935036244658 -0.1325726288302848 1.9784959351107476 -0.13235658576082104 0.4764423558384299 -0.450090949741396 0.477592491479728 0.47490727958503803 0.4743098577865341 -0.547729969409288 1.938973588330172 0.4793535396778759 -0.430351111487918 1.825944419971994 1.8236270865039321 -0.525496229052798 0.42638878378675793 1.33579776199753 -0.044414433995435 -0.13277319829840176 -0.0557989761824642 1.3405997819978779 -0.13070065121124194 1.32937972418263 -0.528491071210614 1.8294632811744382 1.82695611351429 -0.027407894861026207 0.422458274914998 1.822663873667364 -0.303884318708952 -0.0442634218501708 -0.042376857036071205 -0.075125576245344 -0.521742632626482 -0.116829739000513 0.422257186335326 0.419407231357818 0.41210746989039404 0.415100285550634 0.413233545299446 1.826663538213326 -0.1325930057140114 -0.52847641031646 1.8241594939328878 1.6610109754026459 -0.1311773767388252 -0.0770377754916616 -0.61621538274729 -0.1324804489968868 0.42086122291093 -0.004821957055213119 0.0524486145835512 0.0524486145835512 1.817503012588226 0.0627945417147464 -0.530371597544732 -0.133481204533166 -0.138558767337928 1.804944302035034 -0.0501138688057602 -0.5203559855888481 -0.526973184767034 -0.1306644107051204 1.826094449725946 0.419846304382694 -0.12528899290495998 -0.1299684563332714 -0.0751937727995896 0.419059597051312 -0.0980700949526656 1.6270004419867403; -0.292139748189504 -0.35418378300357 -0.857361915351376 -0.0672391364662744 -0.7227100126285542 -0.721962969911538 -0.72022697284671 -0.12255668773641121 1.9741507107477156 1.97885971778983 1.9814001561364338 1.365233093937576 0.35041947985914 -0.560327393351028 1.9847829845631317 0.05387938682095899 1.986842774379502 1.9882282889111405 -0.4524304588575681 2.0000099999999965 1.9842008392739583 1.9836386304314737 1.9801458090960502 1.9850506350092085 1.9781491312747757 1.370483619736914 -0.45253689810311 -0.453169932498114 -0.5541793886561358 -0.40919865068441597 -0.450283689664038 -0.409103713834202 1.350733446545092 1.973884524331852 1.355056567041818 1.3672173762083157 1.3695410594897757 1.570130651477692 -0.411930863994388 1.3140497309107337 1.9129128874252597 -0.322286363624936 -0.32154970726610005 1.5117201432948422 1.1851095728289998 -0.5240935201521442 0.1485714538092218 -0.40507388128131994 -0.1122547487007956 -0.525963814012018 -0.405652492323792 -0.5220756726982039 1.525676572261582 -0.318871907125614 -0.31631518076931797 -0.136848839566666 1.192591001304072 -0.320267773925472 -0.380895806092596 -0.0819199193355676 -0.0455156256149488 -0.15251329220104182 1.52088768980816 0.502107861964872 1.192409863516032 1.167175813624938 1.1726894468879439 1.172184564462816 1.172916296915826 -0.316684482402068 -0.405221468337012 1.517003236998288 -0.315120497528018 -0.28278415832392 -0.402003237670828 -0.224849250765408 1.213019785035986 -0.405646903193782 1.199238521999076 -0.03156033134375719 0.0474679861674532 0.0474679861674532 -0.317711676398122 0.1496863614010826 1.5244492927341038 -0.405017402344258 0.598508610110518 -0.308825655924054 0.198830791909921 1.4974206320368662 1.523796131659676 -0.403933619176 -0.317004735986794 1.19885632503825 0.592464664345306 -0.403138474658852 0.200609561665802 1.197309241548844 0.01641282179148042 -0.285332057727932; -0.289548328867968 -0.358483322858354 -0.85582034362388 -0.07079398777805619 -0.722274221124492 -0.720859807663962 -0.7194213513138159 -0.1274773933752134 1.970852117839894 1.9748999711752238 1.9772702621350757 1.3651360923897802 0.353477749403538 -0.5569810800273979 1.9815033708273115 0.050410865246845786 1.9812267402769301 1.9823351820355357 -0.45074681472201406 1.9842008392739583 2.000009999999984 1.98935026996817 1.978531955860162 1.9829942544180361 1.9748402627258983 1.3700500864150358 -0.45085682185341197 -0.45149199587007605 -0.551050443832678 -0.4108749105123479 -0.44858730475200387 -0.41087997827682193 1.351298746389742 1.972566573942926 1.355035221815652 1.36666451130137 1.369580983264698 1.5697507895615819 -0.41020218976805 1.3161257003170599 1.910075248736238 -0.321610881894094 -0.32087199939252203 1.5107222683502401 1.183605120874668 -0.5247585701277501 0.14442087257949396 -0.40683155733495 -0.112427508356207 -0.5264697041326057 -0.407422192974442 -0.52273507926286 1.525277894758434 -0.318159993022148 -0.31563506273495606 -0.1370735535238956 1.191324001017246 -0.319592637423464 -0.38251902193617 -0.0780745904228366 -0.045585045252288 -0.14992266045180058 1.521846983171746 0.50357774811483 1.19145827952372 1.165946720041458 1.1704393645074977 1.169902920922656 1.171340647409506 -0.316003335715004 -0.40622083063817 1.516043195001128 -0.31443550772884793 -0.28540581329496206 -0.403758299476094 -0.225206951055504 1.21395640457082 -0.407404928256878 1.1977997235410203 -0.032025145144044004 0.0379174506919254 0.0379174506919254 -0.317232116046418 0.1459619234785302 1.5231077195424203 -0.406677912202316 0.593652017583916 -0.308128232062922 0.1994287447064488 1.497030068428902 1.522453795756958 -0.404831725590828 -0.316328302056372 1.198551529930858 0.5905319730322601 -0.40419786268809 0.1976762712582686 1.197459291529198 0.01584974872641562 -0.28714198230456994; -0.28862965599951607 -0.358345238285012 -0.854235159207492 -0.071119576863897 -0.723847540421398 -0.722748410431686 -0.721004812547926 -0.12459213311412558 1.970279952685368 1.974883924053598 1.9772607501913537 1.3639341439742683 0.351761364102252 -0.557456959883166 1.9814924979343884 0.05381692967742039 1.98066223995012 1.981739575630046 -0.451347805034348 1.9836386304314737 1.98935026996817 2.0000099999999943 1.9788726540812178 1.983140484005878 1.9752201223176917 1.369316548640492 -0.451456667586468 -0.45209123066878 -0.5517717156695738 -0.4141614816684979 -0.4493746194651279 -0.414166725298894 1.3512565887665582 1.9733914389394958 1.3550662915974379 1.365500340990706 1.368367429805144 1.568209841222824 -0.410761533393336 1.3149114268411377 1.9099865669044078 -0.322098444347 -0.321361184655928 1.510319650703058 1.1808055518669802 -0.528304405920884 0.14920537518428142 -0.41009207343931003 -0.11230154001151178 -0.5300213935648137 -0.41071575729006804 -0.526285133080764 1.52504416256527 -0.319850866736518 -0.3173250024471262 -0.1369411084477162 1.188872637106574 -0.321292790895088 -0.381610444219572 -0.0818724629801674 -0.04542211817145699 -0.14975839516349437 1.520904518854826 0.505803107361028 1.18878018560793 1.16309846990721 1.1681425513529942 1.1675996561811257 1.168506166486302 -0.317695915287022 -0.409515674076842 1.516349262459552 -0.3163223087449801 -0.28499167440393797 -0.407065873198158 -0.22713824981876 1.213804987096866 -0.410661735268112 1.1949700830263918 -0.031640469759388196 0.0414766446627796 0.0414766446627796 -0.318911141593202 0.1491364717080702 1.52343483519006 -0.409187289857266 0.594792252606254 -0.309803460269006 0.1996428445638304 1.496210254943366 1.523443210640456 -0.408109419341696 -0.318011916673898 1.196189837664424 0.589637869490564 -0.40752145881418 0.1979031294666386 1.194083498493086 0.01612449823957136 -0.287099986157324; -0.28908172671505805 -0.353364880920596 -0.854107855368162 -0.06692564791354479 -0.720322077292406 -0.719582499689604 -0.717824246984508 -0.125651975595835 1.9680550915779738 1.972878667884482 1.9737798122220942 1.36246422258226 0.34444598558779 -0.5565990692877142 1.9771664798166997 0.0531923117653374 1.9771490370760683 1.9782281460411222 -0.4490392992806879 1.9801458090960502 1.978531955860162 1.9788726540812178 2.0000099999999965 1.9861930217462582 1.9722502173528123 1.368152218789558 -0.4490270417075539 -0.4496607343306741 -0.5507363418989282 -0.41059660199191195 -0.44702172980541194 -0.4105042983852438 1.348906892848598 1.9693066801664003 1.3537757917698503 1.36441484533008 1.3666734032622323 1.564832780327474 -0.408302747460318 1.3113617651227238 1.9076777875792443 -0.321741785122924 -0.321010253287358 1.510247676376742 1.1818599742603302 -0.525287151515396 0.14576383697830617 -0.4064106359039059 -0.1116477801757364 -0.52717087821749 -0.40703403214412387 -0.523258187435488 1.523663610201782 -0.31922726455229 -0.31669734206771394 -0.1362730582021808 1.188970660627322 -0.319496875486816 -0.3810567340858 -0.0775695051700192 -0.04511826927853601 -0.15055898086445338 1.518076708245562 0.502589783914118 1.188770451958248 1.162428611218676 1.168015160266446 1.167529772195726 1.168427422021728 -0.317068567512426 -0.405701131905402 1.514305897269 -0.31549219711088394 -0.282123596473932 -0.403501272534264 -0.225339025794572 1.21110072216712 -0.406985240054844 1.1955069793458237 -0.03170185400875979 0.0408563222636136 0.0408563222636136 -0.318058478848986 0.1479476495720966 1.5220722485937557 -0.40551087874996 0.601719747377386 -0.309159578859924 0.201153979939914 1.4949569995062657 1.521383348957504 -0.404433223660288 -0.317138913671374 1.194639802106298 0.590542085013952 -0.403642856708664 0.1961361770596734 1.193646425602126 0.0219971398117882 -0.283913886066368; -0.28486351039827007 -0.357944419709962 -0.8558065696797601 -0.0677058646904476 -0.72275693791161 -0.72178142873288 -0.7202809023414899 -0.12192219848662961 1.9730889255050217 1.977168410117902 1.980340220950888 1.366894643307562 0.345913196269102 -0.558525612815206 1.9829107345957997 0.0534972145812656 1.982089299779424 1.98315685888771 -0.44940441522521 1.9850506350092085 1.9829942544180361 1.983140484005878 1.9861930217462582 2.000009999999994 1.97742681308368 1.372672579842174 -0.44951050050836605 -0.4501431244326 -0.553306809288526 -0.411103581176862 -0.44743384283077 -0.41100983831343396 1.351978700627232 1.974078115569786 1.3567561958606382 1.3672391126136658 1.3700889958989342 1.5685351338039157 -0.40888967731562 1.3151742507058921 1.911727350815006 -0.320372723130392 -0.3196383694230379 1.5140239311713437 1.1868181775459683 -0.5248116960055121 0.14451591886257498 -0.4069472165266621 -0.11181331943889682 -0.526686167830644 -0.4075606204054841 -0.52279368977783 1.527422741958836 -0.31812967761252 -0.3156098784457899 -0.13638134207557778 1.19391831261957 -0.319373882001068 -0.380168791607936 -0.0869842404267454 -0.0453802909377058 -0.15285059117316677 1.521008561168936 0.510593694007492 1.193736393900168 1.167695744722794 1.173203388263856 1.1728704484633723 1.173757315710634 -0.315978895332732 -0.406339968419804 1.51716698120348 -0.314412187115254 -0.2831047393952 -0.40388918204482 -0.224023449910326 1.216641696573872 -0.407517807603846 1.2005161348937199 -0.0315705576354392 0.041191973802334 0.041191973802334 -0.3169901140363 0.1483998008688426 1.5265096217251017 -0.4060440680579 0.597522369921072 -0.308134317818992 0.1995112613280192 1.4984574976727918 1.525822967641014 -0.404967358842252 -0.316293904023194 1.200560938863816 0.592480908386968 -0.404212673602654 0.20116230639445 1.198611619176738 0.01786229921844434 -0.285509089009056; -0.289647915830624 -0.35042935029441 -0.85480394514042 -0.06993905806838718 -0.71935729984299 -0.7195407225147121 -0.7178267272827878 -0.1270942747346228 1.9648322439662622 1.9697233258594438 1.971689584465302 1.359054567003074 0.347862704967532 -0.555649902314328 1.9756635726150762 0.05067423677846081 1.9754635522142 1.9762372328878717 -0.4521462140052901 1.9781491312747757 1.9748402627258983 1.9752201223176917 1.9722502173528123 1.97742681308368 2.000009999999996 1.37069996471443 -0.45230599660107407 -0.452946655268246 -0.5488013479853221 -0.41110020880538406 -0.45008727069604393 -0.41123248109013993 1.35012389783494 1.969393897704182 1.35487943137257 1.366394959153012 1.3639029894904577 1.564347816955558 -0.411804542880162 1.3131397183035858 1.9068939114346697 -0.323683010266856 -0.322809883680748 1.5083571641099902 1.1828671970265725 -0.5214467921236239 0.1473437374480482 -0.40487121812901194 -0.1083959427672668 -0.5233072400792 -0.40651264653162406 -0.519254846071514 1.5221157625698822 -0.32137125408746997 -0.3190021464070281 -0.13325548871048803 1.19119327478525 -0.321732222582522 -0.380599150044554 -0.079456233568143 -0.04673976313269459 -0.1538217277536922 1.51841801756947 0.504254428831042 1.190478779012006 1.16480991113884 1.1703953895688521 1.170145090308206 1.17107744558779 -0.319336158776898 -0.40540988259845 1.5142435386433 -0.31777571459971 -0.28766227773932396 -0.402930886964736 -0.227130572121176 1.209672450203668 -0.406573570899004 1.1971595397684698 -0.03202863845434759 0.0428737023341064 0.0428737023341064 -0.313578902032208 0.150413862473184 1.5204494106271038 -0.40392107455832 0.594414560866334 -0.304604850760754 0.1972714830026484 1.4943526078968659 1.520356930156844 -0.405073029838674 -0.31983511613833 1.196672849493984 0.5876530619075481 -0.402048544152328 0.1953054924036906 1.196496800148334 0.0210325982875448 -0.28765099904197816; -0.307442237663064 -0.0318977221570812 -0.5967007880345699 0.1316345602641472 0.09162489079821282 0.0918970939296164 0.09108761795593498 -0.204619514265646 1.36401561670662 1.365850783840928 1.367122624726314 1.9757305090241841 0.238862319832946 0.607982103059712 1.368982757125216 0.09311620346556661 1.370013495676044 1.368053426517016 0.4740908857456658 1.370483619736914 1.370050086415036 1.369316548640492 1.3681522187895583 1.3726725798421737 1.37069996471443 2.0000099999999943 0.47601728560349793 0.47484247982225614 0.5989491553343061 -0.5944966210408021 0.4710018822303218 -0.5941143897555561 1.9760665846249301 1.36521337880391 1.9736948359227997 1.987731897806968 1.9814786310952 1.703461593949548 0.488739824021204 1.9380847352129376 1.3221831328373643 0.510016238494028 0.51015346112594 1.6598702084980537 1.7290364083765082 0.011234697185163027 0.09907482910527783 -0.5833475322403618 -0.1657993969192232 0.013183720231982974 -0.5834495923017561 0.01165425540358626 1.6561961694973315 0.5157283936113679 0.516825433466582 -0.17988688282756943 1.736179603811928 0.511540690800888 -0.642331171578038 -0.0890725962944572 -0.09463715310539841 -0.2574059797621559 1.64663487267145 0.354833033282776 1.735444333681858 1.702320599680676 1.7048767541034162 1.7061473539302199 1.706648091884344 0.514224284584128 -0.582122486562906 1.660348195053234 0.5143769549682758 0.46068600277272015 -0.578504841602148 -0.330001841083352 1.236322065603742 -0.583716790359416 1.7198226599647577 -0.0411652999738892 0.094524600408868 0.094524600408868 0.517530451159762 0.238799646651136 1.6556054208153022 -0.582854348707188 0.402028267219384 0.517559570105704 0.1456931252704646 1.6289261796410301 1.655817918192532 -0.579905943771108 0.515145282077026 1.721050537390214 0.8696727551285479 -0.579269085742326 0.128265543308566 1.719001383155128 -0.0209623704751088 0.4537807845609759; -0.443753050149738 -0.1648344481210226 -0.35408158689031 0.4718783245417081 -0.241296892368356 -0.240255926645644 -0.24134073527276 -0.05376395026729619 -0.4471496399048199 -0.45028769899669807 -0.45147968853955395 0.470800040360654 -0.08471319922021639 -0.18315497208295023 -0.452006071251392 0.028086743631676594 -0.449584907635884 -0.452500054375786 1.9913170014855177 -0.45253689810311 -0.45085682185341197 -0.451456667586468 -0.4490270417075539 -0.44951050050836605 -0.45230599660107407 0.476017285603498 2.0000099999999965 1.9974078676462397 -0.18303072129585277 -0.1311093733960589 1.9784113087458381 -0.13089283693878445 0.4762894919584081 -0.450407394238488 0.47757941870978804 0.4748761520553321 0.4735306769966359 -0.5476956336642578 1.9388641083030118 0.4793806348405461 -0.43052284157681 1.826025812600004 1.8237089534359658 -0.5255526791314561 0.4270875624815441 1.3368420859795078 -0.04461649778647078 -0.1313265212516038 -0.0559976160497044 1.34159038230185 -0.12923321221570858 1.330422743184334 -0.528551902982016 1.8295424149203858 1.8270347171694703 -0.027610405108156797 0.423137757342192 1.822742101704044 -0.30239710645584 -0.0445762713602278 -0.042634853881147794 -0.07450705091795062 -0.52194646344004 -0.1170069890985934 0.422937952562682 0.420125390796104 0.4128223217080921 0.4158046039592039 0.4139393208727 1.826741095715938 -0.1311443727097308 -0.528548271700834 1.8242359872561598 1.6567584036878418 -0.1297211995922838 -0.0740554817424872 -0.615510845008294 -0.1310356023905316 0.42156148965033 -0.004823488614131424 0.0526107113405802 0.0526107113405802 1.817572933790634 0.0625262881850598 -0.53043663807865 -0.1320279773332634 -0.138904813837836 1.805051114256342 -0.050363016860799 -0.5206947098521442 -0.527177746917576 -0.1292053477717174 1.826171090762884 0.420393462801888 -0.1245824632667152 -0.1285358906586236 -0.0754840685624906 0.41948608983116 -0.0993557980271658 1.6305097589460942; -0.443486324358716 -0.1621090537833188 -0.35422361156816 0.47537003679438394 -0.23881210325446403 -0.23775680196484 -0.23884368794393 -0.053845814304047404 -0.447782939145448 -0.450922701167988 -0.45211527277953395 0.4705039759471401 -0.0847986285783412 -0.18321358937820537 -0.4526398616959079 0.028134682936374583 -0.45022001892251395 -0.45313281555184004 1.992026089716718 -0.453169932498114 -0.45149199587007605 -0.45209123066878 -0.4496607343306741 -0.4501431244326 -0.452946655268246 0.4748424798222561 1.9974078676462397 2.0000099999999916 -0.18308396955466857 -0.12699471288541825 1.979133088189596 -0.12681332914832574 0.4759960308639301 -0.45104223174546804 0.4772900372419619 0.47458138538398975 0.472968521041748 -0.5483462771949962 1.93954841720688 0.47896164092154386 -0.4311364717738261 1.8266908020480601 1.824376636034578 -0.5262010757869439 0.42879807999984393 1.3401157237971981 -0.04467402145066303 -0.12727419525311995 -0.056048382013119624 1.3448677786121517 -0.12514486776152695 1.3337035697439519 -0.5291954268123878 1.8301910824745762 1.827683813962848 -0.027676205726190405 0.424838395664208 1.82339949650847 -0.303507482288628 -0.0447513702621218 -0.0427759890135258 -0.07872897936035342 -0.522594891685904 -0.1171670384931954 0.424641620351828 0.421862677037542 0.4145508736361901 0.41754026706078606 0.415673537271116 1.82739206267933 -0.1270885696486308 -0.52919786142762 1.8248873410297923 1.6575035113128063 -0.1256504048473944 -0.0741624955888604 -0.617235457432154 -0.1269879818818022 0.42327399364740004 -0.004821787503484239 0.0526488433731698 0.0526488433731698 1.818218224980204 0.063218615791918 -0.531079899512276 -0.1279651593457644 -0.1394162419279198 1.805699456735256 -0.0504361600313696 -0.5213465091032539 -0.527820762681474 -0.1251352776958136 1.826816112144736 0.422099811054802 -0.12202617876396697 -0.1244488823307284 -0.075561285276644 0.421192198565256 -0.0993407599502302 1.6281160475285805; 0.32525606701714005 0.646473628652416 0.672039872958448 -0.07597457430601101 1.45971144895913 1.459587697907676 1.457337053533252 -0.0783825996343408 -0.547545194750862 -0.5497378095782182 -0.55290419217512 0.5968791830559801 -0.0871683867025626 1.96466317858656 -0.5522893015374503 0.04427555305681177 -0.553384800422192 -0.5553252489600021 -0.18324935036244658 -0.5541793886561358 -0.551050443832678 -0.5517717156695738 -0.5507363418989282 -0.553306809288526 -0.5488013479853221 0.598949155334306 -0.18303072129585277 -0.18308396955466857 2.000009999999996 -0.18791732697862998 -0.18176831745074476 -0.18764495805364131 0.6127023032042019 -0.5498506053800077 0.599808661818678 0.6065851323582941 0.5998654844104558 0.6899185978386638 -0.1820932104994258 0.6065940516351981 -0.533087306385828 -0.1959197752189516 -0.19496391835753868 0.6904173525068819 0.5211995129523679 -0.2707575182911581 -0.035676794285744634 -0.17637276455528111 -0.044302611813033806 -0.268669755858524 -0.17741590750988323 -0.267937480214578 0.6920934347250981 -0.19502458872154008 -0.19535223058749904 -0.052054237705792004 0.522121225375532 -0.194390630523743 -0.1710418870529044 0.0246845067591134 -0.0432282703044846 -0.1150410306858748 0.680519905717298 -0.134095639751663 0.52157144561165 0.511831178794354 0.5144586049949139 0.5132272782658361 0.515506249259842 -0.1976530152865714 -0.1755343411931768 0.686339889706998 -0.1973169338769773 -0.1957065966429201 -0.1749433160528126 -0.1033051935193258 0.56419731356307 -0.1763067529421102 0.5176426819311439 -0.012010587454743876 0.039442312776734 0.039442312776734 -0.1946021168526248 0.0933544036048056 0.6945648621841339 -0.176482181418827 -0.178807315553719 -0.1978498848917368 -0.038724431133398 0.6877940393658138 0.693945064216124 -0.1740761010625834 -0.1959877063985404 0.52081693202413 0.5654107407499022 -0.1751002819603332 -0.046163876588734 0.52061857438521 0.0206475256537512 -0.17817015041735054; -0.01136228416632392 0.7599510160064359 -0.0439577160041532 -0.051324811105649 1.0631543026643022 1.0569541603135681 1.056254686052346 -0.06975410721274641 -0.41183122521200805 -0.409507511172544 -0.4098472138049639 -0.593959658107888 -0.06306632786653481 -0.18844508075570843 -0.41132369880482594 0.0286858619848384 -0.41308500691964595 -0.40954939488868 -0.1325726288302848 -0.40919865068441597 -0.4108749105123479 -0.4141614816684979 -0.41059660199191195 -0.411103581176862 -0.41110020880538406 -0.594496621040802 -0.1311093733960589 -0.12699471288541825 -0.18791732697862998 2.0000099999999956 -0.12811723339233666 1.9998442729077264 -0.594878657513348 -0.4115191005418081 -0.5896716068697001 -0.5938672573998299 -0.591163611669032 -0.5175185936502741 -0.1300475539537658 -0.5852247099408979 -0.39886839864720214 -0.1377960494538975 -0.1393314555302844 -0.5061032391602 0.37108095627169213 1.22799788055828 -0.04033801590870939 1.9804776590068962 -0.03806601959245881 1.221878011901988 1.973380226411904 1.225779288938212 -0.5035096215899139 -0.14388412648839058 -0.1441251436827507 0.008650847322835767 0.366906911044398 -0.1422602832616534 -0.1274429311382528 -0.029112380405497 -0.04143467512442639 -0.03075661022467805 -0.505550339089268 -0.11311022216078681 0.36483900404273795 0.369030956230098 0.36738013526759006 0.3661369037621319 0.36368259063973396 -0.145269420126618 1.9785336149836419 -0.505470617904108 -0.14453334492972345 -0.13560512886677187 1.973025798202698 1.079657215795062 -0.56930766898883 1.982509629394428 0.3674952123050581 -0.011540622306626598 0.035197220265997 0.035197220265997 -0.1444086796847304 0.064970561178168 -0.5085206968737599 1.9818137826147537 -0.1489096957946818 -0.1444125593557128 -0.0475769581935204 -0.5013602473603598 -0.504127567243308 1.97001859053617 -0.144550801094908 0.363382382427476 0.628580229437 1.9729184380644837 -0.03761053861906081 0.360629134239416 0.00629744602598834 -0.10866643198655045; -0.436768971396264 -0.1631465659349158 -0.35319550437112 0.474774414783656 -0.23857026581518798 -0.237519503983682 -0.238604989643422 -0.05342406315508961 -0.44488124914528393 -0.44801767975320006 -0.449208772823586 0.4678711951247441 -0.0837765051625792 -0.18217458562973188 -0.4497469384774939 0.029113686467974187 -0.44731220367055624 -0.4502485392948559 1.9784959351107476 -0.450283689664038 -0.44858730475200387 -0.4493746194651279 -0.44702172980541194 -0.44743384283077 -0.45008727069604393 0.471001882230322 1.9784113087458381 1.979133088189596 -0.18176831745074476 -0.12811723339233666 2.000009999999996 -0.12786597464965047 0.4783029019455518 -0.44850631973331206 0.473677553077754 0.47104919551251795 0.46966872094125 -0.5448822134359301 1.92779093142526 0.47622613358898014 -0.428179619193028 1.8148325246974977 1.812482489625036 -0.522540060599542 0.4255200597874601 1.3351390593799282 -0.04439560002830022 -0.12834035221240503 -0.05584625910634082 1.3352209640841601 -0.12622405244854848 1.324003751434734 -0.5255911445933541 1.8188271310005582 1.8226848584979563 -0.027296291544512803 0.421485291944608 1.811911835836732 -0.301542378715826 -0.0441559160789686 -0.0423197057189402 -0.0736584578735408 -0.519075079730708 -0.1163736814303172 0.420588635977902 0.417631026532832 0.410315589876576 0.4133196734701679 0.4114453864443 1.814647305226104 -0.1281559625395698 -0.525688779837652 1.8134816804139218 1.65005720455791 -0.1265009965604764 -0.0730696924976616 -0.61200558480494 -0.1280515503859076 0.41915232959441806 -0.004846130057935867 0.0524758276563558 0.0524758276563558 1.806889208460476 0.0621205546081258 -0.52762632454297 -0.1290450495288518 -0.1384901104839494 1.794279981111848 -0.0499742521820926 -0.5176439569039979 -0.524354828087266 -0.1262075445983674 1.815544253098556 0.418691867677488 -0.12498564903832744 -0.125525435869751 -0.0706695717782838 0.417793300000092 -0.0987979347563288 1.610724179632448; -0.011354322415162542 0.7601559444304719 -0.04377135782101119 -0.0512341512553208 1.0634522614872581 1.057252288455922 1.056553495932786 -0.06967484736960299 -0.411738335704006 -0.409412884841996 -0.40975068640334394 -0.593704727883182 -0.0624940880599786 -0.18817048074551612 -0.41122806308191395 0.02864006471129839 -0.41298994203104 -0.40945500395817797 -0.13235658576082104 -0.409103713834202 -0.41087997827682193 -0.414166725298894 -0.4105042983852438 -0.41100983831343396 -0.41123248109013993 -0.594114389755556 -0.13089283693878445 -0.12681332914832574 -0.18764495805364131 1.9998442729077264 -0.12786597464965047 2.000009999999996 -0.5946104160561639 -0.41158937561875597 -0.5892931556059882 -0.5934855074884922 -0.5907839140209402 -0.5173829576981419 -0.12982762032647463 -0.5850022392093639 -0.39879717640975 -0.13756456114915375 -0.13910017769519809 -0.5059782620140999 0.3714868249948639 1.2282460808444662 -0.04028090965340959 1.9805732416140163 -0.03801108780497991 1.222126591530468 1.9734755150730436 1.2259770364247662 -0.503262023214442 -0.14365208120673498 -0.14389319457574767 0.008707383106200096 0.3673482614476 -0.142028377001675 -0.127436977298756 -0.0291110203423004 -0.04143273939207081 -0.03075514662879138 -0.505431315873104 -0.1129531887508784 0.365280896629966 0.369571214606338 0.367916094362118 0.36668107660247806 0.364225148547536 -0.1450371782092214 1.97862924207017 -0.505219168972386 -0.14430092643946005 -0.1353639639233486 1.973121946824992 1.079707129834616 -0.569024766452334 1.982604978584374 0.3679313386683699 -0.01154008315539794 0.0351528406935502 0.0351528406935502 -0.1441753061658884 0.0654063593425504 -0.5082720980496938 1.981908700764864 -0.1490036259124158 -0.1441787310739906 -0.0474947931737718 -0.5012378393163119 -0.503880048864598 1.970113762381014 -0.144280983103633 0.363818171378112 0.628947299664364 1.973013579150226 -0.0375168769008068 0.36080289008648 0.00634994975325094 -0.10848662984212973; -0.3021042486247141 -0.0267325188214662 -0.58613070039247 0.134841735773137 0.10301480645695182 0.1029406286053528 0.1019220771976186 -0.2045690898784 1.345942975310518 1.346595850708154 1.345742518240382 1.9636149690910498 0.23994709291509597 0.620686524052848 1.350298571128412 0.0929405312037584 1.3508055482619379 1.3487402209962178 0.4764423558384299 1.350733446545092 1.351298746389742 1.3512565887665582 1.348906892848598 1.351978700627232 1.35012389783494 1.9760665846249301 0.4762894919584081 0.4759960308639301 0.6127023032042019 -0.594878657513348 0.4783029019455518 -0.5946104160561639 2.000009999999996 1.36545056893977 1.965196027557614 1.977130111922952 1.970831425317538 1.69249644417999 0.490429274240156 1.92881146076403 1.304669090716182 0.507812290738924 0.50793797010183 1.648958716974666 1.716004251675986 0.00991440220890218 0.0972344734077414 -0.583652164924314 -0.1658889471435314 0.01071765688087508 -0.58376451858336 0.009185099021554479 1.645901837413898 0.513637832194898 0.516171862254034 -0.1764243743154172 1.723319140386736 0.509236583937404 -0.636590569080592 -0.0895213267934 -0.09512396725332478 -0.26479604432761594 1.638659772708806 0.345755489552976 1.722542866148962 1.689652782675302 1.69235626764926 1.693158917339756 1.693688310337582 0.512118571485372 -0.582421103564162 1.65116121176557 0.5129930147152361 0.46001192844016 -0.578779496978468 -0.325685914792224 1.228337899566208 -0.584025589413218 1.7063009407948142 -0.04139171534780062 0.094383315134568 0.094383315134568 0.514514184449854 0.241468433957488 1.64429175227479 -0.583165597608524 0.400848905604026 0.514830706283714 0.1475130747499844 1.6197482793403717 1.64546998197322 -0.580319472601804 0.512941506847184 1.708750525344098 0.861490218473646 -0.579823608089042 0.1249698360870716 1.70783169666272 -0.0228189464283204 0.4511873871118; -0.2857983104809261 -0.34744980305127 -0.8529667954061 -0.0688441457803472 -0.719333020925696 -0.718690857872638 -0.717235762579834 -0.1259624697431376 1.963769423098288 1.967006604496712 1.9692498903217783 1.3567092577818503 0.346866215798028 -0.5555090261030078 1.972850272119284 0.04870675881600501 1.972820415925222 1.973909181021378 -0.450090949741396 1.973884524331852 1.972566573942926 1.9733914389394958 1.9693066801664003 1.974078115569786 1.969393897704182 1.3652133788039102 -0.450407394238488 -0.45104223174546804 -0.5498506053800077 -0.4115191005418081 -0.44850631973331206 -0.41158937561875597 1.36545056893977 2.000009999999996 1.351893527788008 1.36138205941344 1.360681490138676 1.560085713792064 -0.410772927254862 1.309995403265292 1.90315733612244 -0.325522669904258 -0.324792317113416 1.506327680981112 1.178857217076138 -0.524592209107324 0.1459102725006932 -0.40958789840607 -0.1119021325212382 -0.52802024765439 -0.407951570750988 -0.5241019444826917 1.520985707972704 -0.323214080297984 -0.320716864629792 -0.1329382021450108 1.185982743671394 -0.324526047477568 -0.37990630090782 -0.0822352560334222 -0.0456433167743684 -0.14865744437958917 1.51638471361599 0.504566471251076 1.185775368008716 1.159893739121686 1.1655197251066762 1.1649892726323798 1.165898095976324 -0.321055541819204 -0.406717893639896 1.510928227264624 -0.31927770679931206 -0.286735764094704 -0.404250327898502 -0.22584201004215 1.20882825373387 -0.407903434845882 1.192693092098168 -0.03175146782509179 0.040984278939812 0.040984278939812 -0.316580414937066 0.1483142167018904 1.5176295351636682 -0.406426070722958 0.591424523971966 -0.30743359345952 0.201306382652476 1.4927426712990124 1.518014750423182 -0.405509491535552 -0.321546282118654 1.192895280404066 0.588441190153452 -0.404718134531666 0.1962712797725604 1.190873723885458 0.01766044221268382 -0.289132046526306; -0.312640424228454 -0.027727494357774198 -0.58832132438355 0.1338708931006492 0.09598188843432182 0.09623097973067202 0.09544762160034917 -0.202881467525808 1.349509070682434 1.350320246852424 1.350212611124154 1.9603285737680363 0.24075519878859997 0.6084571015469918 1.35425087016856 0.09650820527144459 1.3545385427527958 1.352458477176832 0.477592491479728 1.355056567041818 1.355035221815652 1.3550662915974379 1.3537757917698503 1.3567561958606382 1.35487943137257 1.9736948359227997 0.47757941870978804 0.4772900372419619 0.599808661818678 -0.5896716068697001 0.473677553077754 -0.5892931556059882 1.965196027557614 1.351893527788008 2.000009999999996 1.982271914068174 1.971357827959224 1.691164650432742 0.4918993756394198 1.929189724857834 1.308431067295622 0.512355691894374 0.512473617555392 1.6457173716085602 1.717895926307988 0.01668912803083078 0.1027274880454236 -0.578419921772794 -0.1606482141110972 0.01864488934481086 -0.579661775805942 0.01711940526852701 1.6416573181269962 0.518316351657796 0.5194194413279479 -0.1827795485857558 1.725129005415358 0.514080609197706 -0.640043564435122 -0.087956473682792 -0.09385713849455178 -0.2623874936123799 1.633475633901288 0.347706040410824 1.72459823599414 1.69142445892559 1.69418680136781 1.6955262778273377 1.696150193884608 0.517361759957974 -0.577188283644524 1.648008205026794 0.5168061109758879 0.46399746762775 -0.573764382024822 -0.328143695565134 1.22438757678285 -0.578798137624734 1.7108343055649398 -0.040972335338218195 0.0934474020163306 0.0934474020163306 0.520912304581714 0.239528616389224 1.6401387711737798 -0.57794499157626 0.398889417735992 0.521258751171062 0.1492454953938666 1.615322459264464 1.64185402625609 -0.57610709093152 0.51776448507678 1.709378778992802 0.864002364573614 -0.574484934562096 0.1196319173534062 1.707375039792772 -0.0178569913185381 0.45471402500238; -0.30514725520871605 -0.0300499496408008 -0.590728710164618 0.1315001178464184 0.09772783756874401 0.09805698764765261 0.0972462458577638 -0.20437649418971 1.360809171789828 1.36203686925415 1.362163727366326 1.97437805017835 0.24282584322667 0.61485911416559 1.365179056204488 0.096959829337929 1.36674257753025 1.3646725455119701 0.47490727958503803 1.3672173762083157 1.36666451130137 1.365500340990706 1.36441484533008 1.3672391126136658 1.366394959153012 1.987731897806968 0.4748761520553321 0.47458138538398975 0.6065851323582941 -0.5938672573998299 0.47104919551251795 -0.5934855074884922 1.977130111922952 1.36138205941344 1.982271914068174 2.0000099999999943 1.985930169363566 1.70775728959176 0.48879602049450593 1.943081254203104 1.318799131348006 0.510218141258872 0.510354566862052 1.6603248726134978 1.730311256494144 0.01191488430155066 0.09916376924512162 -0.582719778530672 -0.16562140698819758 0.01386342514614744 -0.583939756751762 0.012335257715437281 1.6571243966624143 0.516050239404732 0.517146650601408 -0.18370099350081742 1.73800464385588 0.511844468704008 -0.643295452090924 -0.0892643041225768 -0.09478636183129421 -0.26418988694890194 1.649211388012714 0.352539261054428 1.737272871984338 1.704597980303164 1.7071334485218905 1.7084830533004023 1.708977692882884 0.515256595061214 -0.581495271000148 1.663850047800842 0.514700047348126 0.461303990859066 -0.578105468710722 -0.329659793626984 1.236360740543588 -0.583089215919174 1.721700497661878 -0.04115073861804 0.0944209869443368 0.0944209869443368 0.517854592699664 0.238779401272568 1.656051638397606 -0.582228123238914 0.401103031275986 0.518021607627294 0.1457541290042128 1.6310419964984202 1.657272030686004 -0.580393125980884 0.515467774968694 1.72215462845273 0.8714018851289118 -0.578789742909284 0.1259524632740562 1.720098718842452 -0.02045235248145 0.451885603855138; -0.309313026436166 -0.0322497579223544 -0.592341640975694 0.1347170820482714 0.09342449720273202 0.09371005500418839 0.092911992929863 -0.20335684781706798 1.363116450637308 1.364907254788154 1.365048592578264 1.9728223164457959 0.24751188812303002 0.6082581216019041 1.368050224515294 0.09694447878571341 1.36906978602366 1.3669930609989798 0.4743098577865341 1.3695410594897757 1.369580983264698 1.368367429805144 1.3666734032622323 1.3700889958989342 1.3639029894904577 1.9814786310951997 0.4735306769966359 0.472968521041748 0.5998654844104558 -0.591163611669032 0.46966872094125 -0.5907839140209402 1.970831425317538 1.360681490138676 1.971357827959224 1.985930169363566 2.000009999999996 1.7147354507961678 0.48854084963058797 1.941883263804152 1.326299796272074 0.5100398814397241 0.510151694077284 1.666403176286158 1.72883708843042 0.001457707341681774 0.096516791988812 -0.585544768533392 -0.1685126431888184 0.003387266491904361 -0.58681674444208 0.001864253340060712 1.6626227582108937 0.515754439818948 0.5168498100068281 -0.1829160409891948 1.736166417737592 0.51138576348656 -0.643979481215574 -0.0884090350932878 -0.09407144368755921 -0.2631664560477482 1.65577280892078 0.354048991976746 1.734749818475938 1.702481635422396 1.7051390987610817 1.706222336640839 1.707765128404932 0.51496193015986 -0.584326056145046 1.669413641377986 0.514407112045998 0.4616527587883162 -0.580842382862152 -0.328272306316742 1.241595402587384 -0.58590947806264 1.7198024531426683 -0.015511191675209531 0.0938618057464836 0.0938618057464836 0.5070140742115 0.232678712167196 1.6615575411537915 -0.585201492223706 0.402792840554106 0.50703651030705 0.1464464615406284 1.6366452362333597 1.662762587685308 -0.58325207657055 0.515181942721112 1.720250938061888 0.8695194185873423 -0.581670560668656 0.1267644181653288 1.718653910761466 -0.01933387233622986 0.45244602959192426; -0.08194878159049862 0.0573593662242916 -0.39417041978307 -0.1128881871921458 0.21806332662126598 0.21775982071044203 0.217553173958836 -0.1713364168616992 1.56089694851138 1.563794014527774 1.564191184577488 1.697082479061854 0.28926691151833006 0.697584735392398 1.567872776064418 0.079608407980375 1.568111033254488 1.568220851378554 -0.547729969409288 1.570130651477692 1.5697507895615819 1.568209841222824 1.564832780327474 1.5685351338039157 1.564347816955558 1.703461593949548 -0.5476956336642578 -0.5483462771949962 0.6899185978386638 -0.5175185936502741 -0.5448822134359301 -0.5173829576981419 1.69249644417999 1.560085713792064 1.691164650432742 1.70775728959176 1.7147354507961678 2.00000999999999 -0.5622262998001019 1.662581559088242 1.5467554137663257 -0.48094591412429 -0.47968276579162605 1.93297065278034 1.47857481900006 -0.7236016953244802 0.123268168221737 -0.5119033744109179 -0.1363371776715764 -0.7229065143867122 -0.514306782808674 -0.71994511471148 1.9318107232861081 -0.47673918516773595 -0.476102882878794 -0.16508136569266096 1.487754791418134 -0.47953907047183 -0.477093689285048 -0.062827518889648 -0.06974608646500022 -0.22319796269857398 1.92014702853976 0.412354558700968 1.48677149683327 1.456454819263222 1.4624785381544898 1.461697267105876 1.463434180864766 -0.476153285853602 -0.51079701379524 1.937151769559144 -0.477134497138474 -0.43718051656037793 -0.508092091120306 -0.281709866054206 1.562879299177372 -0.512415018818984 1.472500225443096 -0.016197815979716226 0.0645166714097012 0.0645166714097012 -0.479952949820008 0.208869949733124 1.9312528993922518 -0.511233913024244 0.47240330222454 -0.474062854487296 0.1698206791029694 1.901019558024684 1.929695286473376 -0.510776472584546 -0.477163667491836 1.473776349662092 0.923724923291038 -0.509575724337924 0.166391400641491 1.472070204286012 0.0341089006047072 -0.43269842589839; -0.4210759770824721 -0.1697650315128646 -0.36297191007250595 0.4640675739055379 -0.239452366779474 -0.238346701838268 -0.239452530917954 -0.056851365779953006 -0.4064815442553801 -0.409634549957848 -0.41098117901873 0.485568168316902 -0.0847756846001724 -0.18117309081925975 -0.41145757874942207 0.0311771958440258 -0.408988181200104 -0.41309682824352195 1.938973588330172 -0.411930863994388 -0.41020218976805 -0.41076153339333593 -0.408302747460318 -0.40888967731562 -0.411804542880162 0.4887398240212041 1.938864108303012 1.9395484172068802 -0.1820932104994258 -0.13004755395376588 1.9277909314252597 -0.12982762032647457 0.490429274240156 -0.410772927254862 0.49189937563941993 0.48879602049450604 0.48854084963058797 -0.562226299800102 2.000009999999986 0.4958557865583799 -0.445919893824986 1.8798750397692743 1.8777285930740601 -0.540810316947424 0.44318706225886195 1.381723098054662 -0.053623836157194 -0.13037132523105593 -0.05773556995128879 1.3840355734055056 -0.12821749951175182 1.3755492776337555 -0.544729894014918 1.88259405115457 1.8834415252984698 -0.0302136506501122 0.439402941583148 1.879721951117832 -0.307593286657962 -0.0469918053043848 -0.044470875297265997 -0.07613981765415959 -0.537282161903814 -0.1207291627982366 0.43781009460972 0.433896787550708 0.4274491736214921 0.4302875155559221 0.429758346332002 1.878656356569458 -0.1301836526874662 -0.543940324569334 1.8808115389807096 1.70330042702005 -0.128566792234397 -0.0813106752772054 -0.636269926002828 -0.1300889138285274 0.43600920397414195 -0.004713020753265619 0.053786750419897 0.053786750419897 1.87227932904728 0.054473679176068 -0.546456913858388 -0.1310033358077056 -0.1376322756838568 1.861390357513022 -0.0470725809343382 -0.535065723037482 -0.541374252222438 -0.1282125159134256 1.882302970725808 0.435911083350308 -0.12335573877387417 -0.1275292042047442 -0.073205257348666 0.4366162545338 -0.1021405033906666 1.6793945515116782; -0.319471447999278 -0.01739789300177162 -0.563511146658762 0.13702972820119458 0.1063094586071652 0.10646789375417759 0.1057491913529856 -0.19812798251491037 1.308202596472538 1.30869965924334 1.30916178965244 1.92609296521871 0.25147608029698 0.615258232735796 1.3127686213397423 0.08781051100378 1.314142064375778 1.312740953993644 0.4793535396778759 1.3140497309107337 1.3161257003170599 1.3149114268411377 1.3113617651227238 1.3151742507058921 1.3131397183035858 1.938084735212938 0.4793806348405461 0.47896164092154386 0.6065940516351981 -0.5852247099408979 0.47622613358898014 -0.5850022392093639 1.92881146076403 1.309995403265292 1.929189724857834 1.943081254203104 1.941883263804152 1.662581559088242 0.4958557865583798 2.0000099999999943 1.2960978618535397 0.517977176472658 0.518065311088466 1.6203069122727203 1.6949920469624158 0.018856255846406475 0.1016115026557052 -0.5816461499916199 -0.15899336099510278 0.020775724011378005 -0.58211339447973 0.01912113204138302 1.6162832973769459 0.5238699026259239 0.524959121795784 -0.187660318092618 1.70169285300895 0.519233355191498 -0.631172538297252 -0.090360560690039 -0.08421149479723938 -0.259731454972974 1.60866159019174 0.322115228791586 1.699621423618284 1.668474935574968 1.6694458163259502 1.670265761009844 1.670691829467452 0.52236538349532 -0.580445701099608 1.62185107051218 0.5225467647907439 0.46910641475092624 -0.577140743396884 -0.321671243424356 1.208280106449208 -0.58200717055356 1.6859561230452882 -0.040095513688351986 0.0894161952136468 0.0894161952136468 0.526916340947486 0.241612061461784 1.6159237267910276 -0.58139931117074 0.411000601192354 0.526814731817954 0.153351319328634 1.589866329569358 1.61604802500608 -0.578643422241362 0.52239856843839 1.68522771091316 0.8477095628469401 -0.57822721601025 0.1157286760764876 1.68371525766402 -0.01882764934318234 0.4604185258028704; -0.278611159086956 -0.33349133726487 -0.8689721782130939 -0.0545417024445404 -0.6980076973115401 -0.6967069039606919 -0.694841411081272 -0.12039212090067541 1.9030039766782259 1.9052476457522503 1.90675157108853 1.316672435838044 0.360939064747266 -0.53952294413035 1.910593354990506 0.046997396789937404 1.910041459563558 1.9116749833632838 -0.430351111487918 1.9129128874252597 1.910075248736238 1.9099865669044078 1.9076777875792443 1.911727350815006 1.9068939114346697 1.3221831328373639 -0.43052284157681 -0.4311364717738261 -0.533087306385828 -0.39886839864720214 -0.428179619193028 -0.39879717640975 1.304669090716182 1.90315733612244 1.308431067295622 1.318799131348006 1.326299796272074 1.5467554137663257 -0.44591989382498604 1.2960978618535397 2.000009999999992 -0.45661440985821194 -0.45607757038294 1.553399855484612 1.1513598810830583 -0.630487487305532 0.1620420211572802 -0.39896055665873004 -0.1041949861740766 -0.628447238155632 -0.40009588222156006 -0.6286498239939858 1.56860864874678 -0.45352044194849594 -0.45275050410454004 -0.13192916364187518 1.159325004498398 -0.453781163000646 -0.358032806973864 -0.0778270282189734 -0.0377630647496336 -0.15016143576968993 1.563790805349864 0.49339485720462 1.15891036931341 1.133497872935574 1.1377419519287797 1.1387945888406252 1.1380928560522 -0.451517696248196 -0.399634791929444 1.558354916732516 -0.45101010556533405 -0.39971319324095395 -0.397338786904044 -0.215553279012756 1.258221814392052 -0.400661330268192 1.1661076743490515 -0.013620619761941611 0.0348352034860234 0.0348352034860234 -0.455636247044058 0.1492568493109948 1.5665291570897426 -0.398187421574784 0.611222336856568 -0.44818499769445 0.207687847368804 1.536999428021134 1.565710198839516 -0.39958433463678 -0.452966177489756 1.165872822430296 0.583367245484936 -0.396457415919902 0.20433253458195 1.16413924733856 0.0297037472687006 -0.405885625714192; -0.446249690411152 -0.1817683589117658 -0.291289588742478 0.43647728934461005 -0.255863035142746 -0.254683373682142 -0.25584493085417803 -0.059323096081023395 -0.31681072367447394 -0.32226107918364794 -0.321530610851974 0.506710192578968 -0.0759556701412546 -0.1938703925675268 -0.321924334139112 0.0354072345677488 -0.319511168071684 -0.323297313740664 1.825944419971994 -0.322286363624936 -0.321610881894094 -0.322098444347 -0.321741785122924 -0.320372723130392 -0.323683010266856 0.510016238494028 1.826025812600004 1.8266908020480601 -0.1959197752189516 -0.1377960494538975 1.8148325246974977 -0.13756456114915375 0.507812290738924 -0.325522669904258 0.512355691894374 0.510218141258872 0.5100398814397241 -0.48094591412429 1.8798750397692743 0.517977176472658 -0.45661440985821194 2.000009999999996 1.9980433230855197 -0.558256260809142 0.4657823239024639 1.4411084999577481 -0.05861575493653261 -0.1403195432449056 -0.06237560402869279 1.441370677521996 -0.1378433098029938 1.4394413522228682 -0.564511646484132 1.9638879492543502 1.964665716026468 -0.0360366481938854 0.460438385208196 1.954003142060334 -0.325493471308552 -0.0447806333450496 -0.050196195906291805 -0.08302138746245741 -0.558392770501556 -0.1250571269114794 0.458964527957344 0.455166098191172 0.44954536488736785 0.4520293294248039 0.451796475592138 1.960492679168622 -0.1416762788858876 -0.56155280505825 1.960758805900814 1.764366912723302 -0.1388365357752518 -0.086455639709724 -0.659033984636308 -0.1400381204375534 0.458423117136658 -0.004586025450803341 0.0573188060312406 0.0573188060312406 1.951602291345702 0.0555181913739004 -0.564703669986622 -0.1411308565255088 -0.1396274703584742 1.940917631452368 -0.0537062426980142 -0.552532987024134 -0.562293025219372 -0.1380796575793176 1.963127186092696 0.457838193215034 -0.12697336702468004 -0.1373925245580642 -0.0801029934765542 0.457834810145048 -0.1090313066246154 1.7393725734387624; -0.447696087235946 -0.1832312696478672 -0.29025783826155 0.43718382508565 -0.255996455012822 -0.254821544594242 -0.255981390833272 -0.059121221372574 -0.31607489547591605 -0.32152368716353197 -0.320787845580374 0.5068504004062879 -0.07568388835672499 -0.19290774672291056 -0.32118556547742805 0.035289806639021 -0.31877085130757804 -0.322562986493032 1.8236270865039321 -0.32154970726610005 -0.32087199939252203 -0.321361184655928 -0.321010253287358 -0.3196383694230379 -0.322809883680748 0.51015346112594 1.8237089534359658 1.824376636034578 -0.19496391835753868 -0.1393314555302844 1.812482489625036 -0.13910017769519809 0.50793797010183 -0.324792317113416 0.512473617555392 0.510354566862052 0.510151694077284 -0.47968276579162605 1.87772859307406 0.518065311088466 -0.45607757038294 1.9980433230855197 2.000009999999988 -0.557913150769656 0.4651644979581281 1.43977608839499 -0.0584835651231924 -0.14184142990859044 -0.0622539285261066 1.44003420330721 -0.1393821187229276 1.4381042708985623 -0.564176988682374 1.9635089019224896 1.9642881484015877 -0.035877549403499805 0.459835055842888 1.9536029769257 -0.325518121538022 -0.0443640471182932 -0.04986790679178999 -0.0826610983794358 -0.558052500342216 -0.1246593399715664 0.458357486967198 0.454512100931156 0.44889037870193 0.4513842950622942 0.45115005559934 1.960107751093464 -0.143201247837454 -0.561202126701636 1.9603756236858516 1.7638854639522257 -0.1403648656622718 -0.0899096176985996 -0.658386923931712 -0.1415579324001534 0.45779555794751786 -0.004590309497503562 0.0572276761004468 0.0572276761004468 1.9512271463898 0.0552648961626604 -0.5643663946512201 -0.1426594155667312 -0.1409434349592922 1.940535540702896 -0.0535244061009766 -0.5521904119178421 -0.561956506092408 -0.1396107389305284 1.962756639363024 0.45721605225773 -0.12722590476450318 -0.1389308586504762 -0.0799111333536114 0.457218816687704 -0.1089089478694214 1.7384613293263858; -0.05910707139588801 0.0608993352752146 -0.42606056377684 -0.096949879748831 0.22544630676201 0.225005450559646 0.22462173562297197 -0.16845407599912443 1.50206174164777 1.508893845912564 1.508136758306766 1.652000454999252 0.2887943482772479 0.695387567378712 1.5100565977232923 0.0761796801858742 1.511526130064112 1.51131430800027 -0.525496229052798 1.5117201432948422 1.5107222683502401 1.510319650703058 1.510247676376742 1.5140239311713437 1.5083571641099902 1.659870208498054 -0.5255526791314561 -0.5262010757869439 0.6904173525068819 -0.5061032391602 -0.522540060599542 -0.5059782620140999 1.648958716974666 1.506327680981112 1.6457173716085602 1.6603248726134978 1.666403176286158 1.93297065278034 -0.5408103169474239 1.6203069122727203 1.553399855484612 -0.558256260809142 -0.557913150769656 2.00000999999999 1.483935522263536 -0.7896774697601862 0.12933024183294858 -0.505857646856994 -0.12919614413993902 -0.7863623765110739 -0.507674033232926 -0.7858644832896597 1.964048678767304 -0.564568945788066 -0.5638978056181221 -0.15094865377733022 1.472954990990228 -0.563556966389964 -0.46256559886912 -0.0634180348801382 -0.06618732295942939 -0.21611500821737603 1.950727271075902 0.41820005056616 1.47081613410132 1.442317736720956 1.4470011276244523 1.4466340614378537 1.44791903235554 -0.564072604822714 -0.505986133223078 1.966156971062204 -0.5643623703391659 -0.513301133997904 -0.503133342689172 -0.277067484366398 1.59549861253429 -0.507478958869626 1.4585040660658457 -0.038049309449281206 0.0611655475965878 0.0611655475965878 -0.567458918109478 0.20892703134707796 1.962580589846696 -0.505252382846262 0.473675044573168 -0.562392990733096 0.1774744568135972 1.9320390224676183 1.961965166819164 -0.505892846131596 -0.564679578909644 1.459877989656672 0.9348872925605141 -0.503590170612106 0.169083285829049 1.457584183139066 0.0390698458601614 -0.507235088715774; -0.32309236784920603 0.331657337254574 -0.628003392214566 0.11234634392879961 0.61324229125987 0.611283134043266 0.6099339237342741 -0.24314405884295004 1.177023353782174 1.182184024266976 1.181513073922212 1.7163592207488363 0.219070864598646 0.527670968043846 1.182535873399396 0.10923470242472919 1.183424019230948 1.1834028866039439 0.42638878378675793 1.1851095728289998 1.183605120874668 1.1808055518669802 1.1818599742603302 1.1868181775459683 1.1828671970265725 1.729036408376508 0.4270875624815441 0.42879807999984393 0.5211995129523679 0.37108095627169213 0.4255200597874601 0.3714868249948639 1.716004251675986 1.178857217076138 1.717895926307988 1.730311256494144 1.72883708843042 1.47857481900006 0.44318706225886195 1.6949920469624158 1.1513598810830583 0.4657823239024639 0.4651644979581281 1.483935522263536 2.0000099999999943 0.6257806148484563 0.08041096870052901 0.375375625481692 -0.192340375345545 0.6247766689675159 0.36932246823898 0.6251056985413818 1.461522791659524 0.4716229546830639 0.472630119006924 -0.18414170450666287 1.979302039723672 0.468431583691458 -0.72565294232141 -0.1067834509277634 -0.11848106153820502 -0.28577088145578794 1.450470252523822 0.30448720622723 1.9758003919118 1.946440235363056 1.948178354015212 1.947481906951508 1.94746140897876 0.46939928745611 0.374533060342216 1.463965683005764 0.4699220051963237 0.4167603186251718 0.375532216508984 0.1942904957781912 0.993601989463448 0.374858442680326 1.9639896550717157 -0.04811659585771721 0.1139384700508848 0.1139384700508848 0.47421368394739 0.281718427972998 1.4574202287488018 0.374101428720424 0.335942266865226 0.473435550076498 0.1279839272234792 1.4354287350216817 1.461131253057908 0.372585652454192 0.470677916604308 1.962483073772432 1.2095438373855119 0.373353679836216 0.110824089857002 1.958095387881362 -0.0218770717678356 0.42271772898568; -0.36455086480808 0.36828217697404 -0.25742905501839 0.29243399223516 0.5158860013880741 0.5126164613625301 0.511220808666218 -0.0954818363281962 -0.521559623208504 -0.523675293732576 -0.523990604021204 0.0036010787810237532 -0.1094742933812618 -0.2697584229021238 -0.5260386730136961 0.0417430165821088 -0.52456591053304 -0.52509589469435 1.33579776199753 -0.5240935201521442 -0.5247585701277501 -0.528304405920884 -0.525287151515396 -0.5248116960055121 -0.5214467921236239 0.01123469718516304 1.3368420859795078 1.3401157237971981 -0.2707575182911581 1.22799788055828 1.3351390593799282 1.2282460808444662 0.00991440220890218 -0.524592209107324 0.01668912803083078 0.01191488430155066 0.001457707341681774 -0.7236016953244802 1.3817230980546618 0.018856255846406475 -0.630487487305532 1.4411084999577481 1.43977608839499 -0.7896774697601862 0.6257806148484563 2.000009999999996 -0.07439380949221258 1.2307418652848678 -0.07571077743659761 1.9794714322313374 1.224900345055688 1.9813502184578797 -0.7923226807016222 1.4568486673543382 1.461606818256556 -0.028753982041302403 0.618064268181864 1.449497392879226 -0.340507166830422 -0.056656971471084 -0.06881876278457599 -0.08558009747944142 -0.788837065470154 -0.1761480805498964 0.616720808986114 0.616519281244046 0.6113973642012698 0.612447826296498 0.61060539333368 1.4530912346409197 1.231332265087056 -0.792605068179738 1.4542216144754634 1.3113617121382022 1.228639657934744 0.661419542673064 -0.910998730518758 1.233907932721372 0.617682760073006 -0.011383676551019174 0.0695465072506064 0.0695465072506064 1.4645069034861 0.0900471415586588 -0.79651731381761 1.229687998494256 -0.208243155665892 1.456175451805384 -0.0758730178398276 -0.7812653160959236 -0.790175138009394 1.226995828549094 1.455534446929108 0.61437784803479 0.3257377333616759 1.22748292610101 -0.089817224979964 0.612776557381158 -0.08392503073335 1.3046243528607375; 0.07729767926150481 -0.0548767866031736 0.1969982318485788 -0.0330889463095628 -0.010684483526619541 -0.011679634289869519 -0.01199290571961476 -0.0288528955738182 0.14972337622354298 0.1491154497488192 0.14342083127081823 0.0988318659600848 -0.03843470904300061 -0.03593140853487279 0.14829487529205923 0.0168776141529762 0.14427486982261087 0.14891188292111798 -0.044414433995435 0.1485714538092218 0.14442087257949396 0.14920537518428142 0.14576383697830617 0.14451591886257498 0.1473437374480482 0.09907482910527782 -0.04461649778647078 -0.04467402145066303 -0.035676794285744634 -0.04033801590870939 -0.04439560002830022 -0.04028090965340959 0.0972344734077414 0.1459102725006932 0.1027274880454236 0.09916376924512162 0.096516791988812 0.123268168221737 -0.053623836157194 0.1016115026557052 0.1620420211572802 -0.05861575493653261 -0.0584835651231924 0.12933024183294858 0.08041096870052901 -0.07439380949221258 2.000009999999996 -0.04091765437527979 -0.020179057241674403 -0.07437356755130478 -0.04042273765974241 -0.07450207166464917 0.132012903921583 -0.0589537519766062 -0.0589340215798704 0.00289916536093034 0.0754916634924354 -0.0588366555765448 -0.011348769627791942 -0.00956729141793946 -0.007830092248325482 -0.031312032822376615 0.1294563829248108 -0.0195164260493617 0.0814110665404096 0.077979143784848 0.0836101542188232 0.0854361331610256 0.0827029026949694 -0.058986739752528 -0.0399830404587712 0.1271115753258574 -0.05907382049916678 -0.05001562631607256 -0.0399687923106368 -0.009691826519402438 0.0930046020107494 -0.0408993789371912 0.0807545762455372 -0.0034554234327494 0.01622391688766406 0.01622391688766406 -0.0595576915748754 0.0337930546601588 0.13147159939939743 -0.0406214108600942 -0.04504309891681241 -0.0597436291849122 -0.025692310350150196 0.12407082874085362 0.1280756852537348 -0.040452252112978 -0.0591149578707866 0.0764340117563976 0.010573541986831952 -0.0403679424129154 -0.0272905273291158 0.0785681280045098 -0.00433393971496532 -0.055265050974238; -0.020029341995835403 0.7480789247976299 -0.04524912588801759 -0.0570854680524212 1.061968674815038 1.0558610229558103 1.055150263031872 -0.0704655502104354 -0.4076531153904239 -0.40451080574453996 -0.40574136942229805 -0.581864467898318 -0.06838122090156422 -0.17708119034156794 -0.407195904986052 0.029175477270882 -0.40893144013150595 -0.40541226294366794 -0.13277319829840176 -0.40507388128131994 -0.40683155733495 -0.41009207343931003 -0.4064106359039059 -0.4069472165266621 -0.40487121812901194 -0.5833475322403621 -0.1313265212516038 -0.12727419525311995 -0.17637276455528111 1.9804776590068962 -0.12834035221240503 1.9805732416140163 -0.583652164924314 -0.40958789840607 -0.578419921772794 -0.582719778530672 -0.585544768533392 -0.5119033744109179 -0.13037132523105588 -0.5816461499916199 -0.39896055665873004 -0.1403195432449056 -0.14184142990859044 -0.505857646856994 0.375375625481692 1.2307418652848678 -0.04091765437527979 2.0000099999999907 -0.038628561801105805 1.229392466841212 1.9800403331964398 1.234348335234046 -0.5064494675828959 -0.14637478206956897 -0.14661273478697723 0.007748594288816758 0.36916728828165 -0.1444373696603136 -0.1295533564859706 -0.0295545959033158 -0.041689851893588016 -0.0317513585721482 -0.506274263056132 -0.1148090057927122 0.367984844638028 0.370948276204128 0.37017056154204403 0.3686437186856101 0.366208261273804 -0.1477536468711468 1.989330641678778 -0.505199504271952 -0.14670213072475574 -0.13777781538177272 1.9839788563008 1.071758234962088 -0.571715484476788 1.99323424574528 0.37071345618442797 -0.01251345038986564 0.035621664793258 0.035621664793258 -0.1469156132150064 0.0664478160298096 -0.5092449226078761 1.990329290286602 -0.1412453601732004 -0.1469262538623256 -0.048311367695861 -0.5022223501970681 -0.504865163614788 1.980994390318452 -0.1467174756876938 0.366596892583384 0.6334174157904919 1.979309002290024 -0.0383328417736334 0.363670967756974 0.00527106915321058 -0.11127797448819722; -0.11515533241960982 -0.0406906591284272 -0.1078038194120386 -0.016697124583651555 0.307673878076684 0.302057129805978 0.301585326367216 -0.0277722132355774 -0.1116136671776792 -0.112332169529549 -0.1128379813996048 -0.16635957492764963 -0.036520967062156594 -0.03993749883736184 -0.11244616811684659 0.016245464791048124 -0.11237041168375742 -0.11210544116828079 -0.0557989761824642 -0.1122547487007956 -0.112427508356207 -0.11230154001151178 -0.1116477801757364 -0.11181331943889682 -0.1083959427672668 -0.1657993969192232 -0.0559976160497044 -0.056048382013119624 -0.044302611813033806 -0.03806601959245881 -0.05584625910634082 -0.03801108780497991 -0.1658889471435314 -0.1119021325212382 -0.1606482141110972 -0.16562140698819758 -0.1685126431888184 -0.1363371776715764 -0.05773556995128877 -0.15899336099510278 -0.1041949861740766 -0.06237560402869279 -0.0622539285261066 -0.12919614413993902 -0.192340375345545 -0.07571077743659761 -0.020179057241674403 -0.038628561801105805 2.000009999999996 -0.07569678763249621 -0.03814579529082661 -0.07582041796785881 -0.13580709872902244 -0.06265707739120943 -0.0626252035941798 0.003665464406959899 -0.1881159509143814 -0.0615017969488408 0.0708259413092672 0.0255361533402724 0.036932428646491794 -0.004718622947192937 -0.1325304703615948 -0.0359806445368112 -0.1799436814143892 -0.1860826400149448 -0.1891684795375411 -0.18851627845263086 -0.1878670757370144 -0.061628466165902 -0.0385870072406008 -0.1270184940291096 -0.061709744792359175 -0.05769722183911235 -0.0385716882747636 -0.0228825128840668 -0.1421492669188834 -0.0386118218013416 -0.19004330881679735 7.48099499014998e-18 0.0156162516918874 0.0156162516918874 -0.063205989231674 0.0329389155557384 -0.13257876340553565 -0.0383417348204846 -0.01793464844525252 -0.0633819959841098 -0.003809285337894961 -0.1361562927387514 -0.1356814404169924 -0.0381686522919848 -0.062784619483811 -0.1952519862920242 -0.12668860620181466 -0.0372159875066744 -0.026945210974867004 -0.1917152658696336 -0.019787918762358442 -0.05825823063450618; -0.3671523962858281 0.362455603880094 -0.26183088787134 0.298608434567632 0.513494499305726 0.51020081705564 0.5088068486104599 -0.09544473180695921 -0.52330083641751 -0.524862265806274 -0.525694043576042 0.005542760826919874 -0.11019496157075659 -0.26768430482922184 -0.527909833413508 0.041702421130835596 -0.5264390851669459 -0.5269696366534179 1.3405997819978779 -0.525963814012018 -0.5264697041326057 -0.5300213935648137 -0.52717087821749 -0.526686167830644 -0.5233072400792 0.013183720231982998 1.34159038230185 1.3448677786121517 -0.268669755858524 1.221878011901988 1.3352209640841601 1.222126591530468 0.01071765688087508 -0.52802024765439 0.01864488934481086 0.01386342514614744 0.003387266491904361 -0.7229065143867122 1.3840355734055059 0.020775724011378005 -0.628447238155632 1.441370677521996 1.44003420330721 -0.7863623765110739 0.6247766689675159 1.9794714322313374 -0.07437356755130478 1.229392466841212 -0.07569678763249621 2.0000099999999943 1.2366485241606902 1.977391247455232 -0.7911071400540557 1.4553325888410804 1.455763293160926 -0.028668904441374002 0.617490203593926 1.447955825245752 -0.340194446431234 -0.0566702166150944 -0.0688596601870478 -0.08595221447407278 -0.786101094840394 -0.176054018184926 0.61571701225045 0.615744875127594 0.609922170450464 0.6109699235460803 0.609126466225196 1.45156621780604 1.226834553884626 -0.789349095436784 1.4526996096499518 1.307738823900992 1.224114581464174 0.652608894557314 -0.910060723066462 1.229426191758474 0.61666701135035 -0.011398918046007837 0.0695468746366416 0.0695468746366416 1.463018481036022 0.0891077134496244 -0.7945277292122038 1.22517769820595 -0.206050367088222 1.454678849715994 -0.075825869016405 -0.7794333860806602 -0.788180090424784 1.22246919285668 1.454408060346454 0.613364989755986 0.3291226440275499 1.220253893530348 -0.089667982261575 0.611337213911556 -0.0834122628372142 1.3040828623775158; -0.01897624351873668 0.744392773590644 -0.0462371486056818 -0.0562242422631456 1.0552168498510082 1.0489490134178479 1.04825387094736 -0.0699038697047052 -0.40831491751908 -0.40595171713560796 -0.4063016107243841 -0.58304610805676 -0.067417854607932 -0.17805920620100318 -0.407783858804142 0.028737819431945005 -0.40954073389994394 -0.406001166840034 -0.13070065121124194 -0.405652492323792 -0.407422192974442 -0.41071575729006804 -0.40703403214412387 -0.4075606204054841 -0.40651264653162406 -0.583449592301756 -0.12923321221570858 -0.12514486776152695 -0.17741590750988323 1.973380226411904 -0.12622405244854848 1.9734755150730436 -0.58376451858336 -0.407951570750988 -0.579661775805942 -0.583939756751762 -0.58681674444208 -0.514306782808674 -0.1282174995117518 -0.58211339447973 -0.40009588222156006 -0.1378433098029938 -0.1393821187229276 -0.507674033232926 0.36932246823898 1.224900345055688 -0.04042273765974241 1.9800403331964398 -0.03814579529082661 1.2366485241606902 2.0000099999999943 1.225346020165492 -0.5048157073608701 -0.1439476829420124 -0.1441893672300659 0.00868051513501298 0.365279121460426 -0.141993958286785 -0.1306470903738604 -0.0298041059427246 -0.04204181192792398 -0.030897115683359407 -0.507969814829798 -0.113128858228699 0.36320266875666 0.367375972215488 0.36584055750245603 0.363687165794818 0.361231620937806 -0.1453359330947246 1.9780875981719 -0.506821703309282 -0.1442723908555253 -0.13445802614067462 1.972548116086196 1.066316911210348 -0.569374585859762 1.982082728929832 0.3658632181427679 -0.011567502129040397 0.0352720557242618 0.0352720557242618 -0.1444741016797524 0.0656370489222158 -0.5098381290834039 1.98136818872701 -0.1443177380430706 -0.144478031988149 -0.0476763154041928 -0.5039106661678622 -0.506540425161886 1.971865630083698 -0.1442909857206804 0.361748397937188 0.6277523190558977 1.970083069629284 -0.037684968233321 0.358821052893732 0.00604250836605052 -0.10981174390878928; -0.363908903025608 0.368663603381578 -0.254612591778004 0.29695918046447595 0.516734326781856 0.513461631013042 0.512063888970706 -0.09562169269607 -0.519533137913354 -0.522660411497212 -0.521972474128392 0.0040100299035480125 -0.1104309399585196 -0.266971400388754 -0.524024399402392 0.04180614370599638 -0.5225443206218521 -0.5230774646683821 1.3293797241826302 -0.522075672698204 -0.52273507926286 -0.526285133080764 -0.5232581874354879 -0.5227936897778299 -0.5192548460715142 0.01165425540358625 1.330422743184334 1.3337035697439519 -0.26793748021457797 1.2257792889382118 1.3240037514347343 1.225977036424766 0.00918509902155446 -0.524101944482692 0.01711940526852704 0.01233525771543734 0.001864253340060686 -0.71994511471148 1.3755492776337555 0.019121132041383097 -0.6286498239939861 1.439441352222868 1.438104270898562 -0.78586448328966 0.625105698541382 1.9813502184578802 -0.0745020716646492 1.234348335234046 -0.07582041796785881 1.9773912474552318 1.225346020165492 2.000009999999996 -0.7920528305226737 1.4510579078041865 1.451490255424176 -0.028802115858902 0.617382140756104 1.443666798693916 -0.33894856809028 -0.0565056523536252 -0.0687246450769114 -0.0860306763946494 -0.78704598444395 -0.175859545410766 0.616034714003902 0.615818367489948 0.6106998175460236 0.6117392730812438 0.609462983485036 1.447287334155972 1.231794997126266 -0.789585939868104 1.4484211940126037 1.3025275131773983 1.234213155632868 0.655244758358654 -0.909315068573342 1.234376293376876 0.617266822012632 -0.01139874973642072 0.0696461448852752 0.0696461448852752 1.458754347709716 0.0893169862614444 -0.7942377172207601 1.230147062380324 -0.206474825793748 1.450415645062112 -0.075985590407111 -0.7791258211082801 -0.789125298937328 1.227449288479364 1.450143640158798 0.613693499425468 0.33129736260302994 1.225204810244262 -0.0898856484162782 0.611660960960744 -0.0832286070769954 1.29947572686534; -0.0610662497820636 0.0693265622061516 -0.44070155345754397 -0.09892089318242982 0.20877076661038402 0.20940321194432002 0.209726810839316 -0.1679474375552618 1.516262926875128 1.521725327343652 1.52307119707167 1.648860496255838 0.29266659061144806 0.67183385139202 1.5241434320609877 0.0760134538508796 1.523966991439826 1.5243680623453402 -0.528491071210614 1.525676572261582 1.525277894758434 1.52504416256527 1.523663610201782 1.527422741958836 1.5221157625698822 1.6561961694973317 -0.528551902982016 -0.5291954268123878 0.6920934347250981 -0.5035096215899139 -0.5255911445933541 -0.503262023214442 1.645901837413898 1.520985707972704 1.6416573181269962 1.6571243966624143 1.6626227582108937 1.9318107232861081 -0.5447298940149181 1.6162832973769459 1.56860864874678 -0.564511646484132 -0.564176988682374 1.964048678767304 1.461522791659524 -0.7923226807016222 0.132012903921583 -0.5064494675828959 -0.13580709872902244 -0.7911071400540557 -0.5048157073608701 -0.7920528305226738 2.0000099999999965 -0.5700880039480519 -0.5694173468253478 -0.15718641665310787 1.47388115231371 -0.56899706890911 -0.466415989606066 -0.0629683097860004 -0.0657353201367724 -0.216920540913318 1.973239943465366 0.4202502376325 1.472478749083816 1.443462665249488 1.4493783125344073 1.4477492444226097 1.449992914076618 -0.570567594867972 -0.504386158668068 1.970762877956694 -0.570044907298648 -0.517319400320458 -0.502486910735916 -0.274079793930092 1.600618592537498 -0.505997711615624 1.47780312360785 -0.03782572778978761 0.0610766462582742 0.0610766462582742 -0.572948294765548 0.208361387254362 1.984862121399278 -0.504580724645288 0.47621729336608 -0.568696225680212 0.1759977906947854 1.9522191209230262 1.982689847487534 -0.502124763132144 -0.57033200724324 1.480361698562358 0.9463754184952281 -0.50077239804683 0.170366333463153 1.4785572564331 0.041324323803675 -0.5127152362350541; -0.450391097637428 -0.18742951454268503 -0.29150652967704 0.431064669725476 -0.259088066148552 -0.257917233790456 -0.25907857102664 -0.0599990240274266 -0.31337127175877594 -0.318800423980796 -0.318110602693056 0.512419706189088 -0.0799333406593606 -0.19306441262096674 -0.318486138507776 0.0357961568592504 -0.31608128504193 -0.319826204929504 1.8294632811744382 -0.318871907125614 -0.318159993022148 -0.319850866736518 -0.31922726455229 -0.31812967761252 -0.32137125408746997 0.5157283936113679 1.8295424149203858 1.8301910824745762 -0.19502458872154008 -0.14388412648839058 1.8188271310005582 -0.14365208120673498 0.513637832194898 -0.323214080297984 0.518316351657796 0.516050239404732 0.515754439818948 -0.47673918516773595 1.8825940511545702 0.5238699026259239 -0.45352044194849594 1.9638879492543502 1.9635089019224896 -0.564568945788066 0.4716229546830639 1.4568486673543382 -0.0589537519766062 -0.14637478206956897 -0.06265707739120943 1.4553325888410804 -0.1439476829420124 1.451057907804186 -0.5700880039480519 2.000009999999996 1.9932506970727344 -0.042030615754254 0.466222396851054 1.981427469417186 -0.324362206727396 -0.0457807381376694 -0.050890658608872597 -0.0811821571874368 -0.563873601160842 -0.1263504029806642 0.465532871356394 0.462050069157134 0.4566199981465801 0.4588870179180883 0.458652555660514 1.988902820731438 -0.1459497929414074 -0.56877590893936 1.9894311988859 1.7941340150184617 -0.1446451841932408 -0.0921159570195362 -0.665582395230438 -0.1460888525883392 0.46434085748982584 -0.004544031519223697 0.0574794642936464 0.0574794642936464 1.980118128618888 0.0564557686644794 -0.5720078897167516 -0.1468959182714628 -0.1394911029681674 1.969498774158258 -0.0543000384597742 -0.5598588082726419 -0.567730113827008 -0.1441585516357384 1.991595970719202 0.464987123015272 -0.12800271025893145 -0.143496976821784 -0.080534871902349 0.463692123584816 -0.110210500924414 1.7699269715024162; -0.44954167968048 -0.186108471745065 -0.292275410260638 0.42329893505519 -0.25947365021957003 -0.258172688233436 -0.259333261316358 -0.05996455300722381 -0.310849878573794 -0.31627545586833 -0.315590767467548 0.513517654126874 -0.08045145654580782 -0.19319515474121524 -0.31596746502386 0.0357757304263448 -0.31355901573649 -0.317304843121738 1.82695611351429 -0.31631518076931797 -0.31563506273495606 -0.3173250024471262 -0.31669734206771394 -0.3156098784457899 -0.3190021464070281 0.516825433466582 1.8270347171694703 1.827683813962848 -0.19535223058749904 -0.1441251436827507 1.8226848584979563 -0.14389319457574767 0.516171862254034 -0.320716864629792 0.5194194413279479 0.517146650601408 0.5168498100068281 -0.476102882878794 1.8834415252984702 0.524959121795784 -0.45275050410454004 1.964665716026468 1.9642881484015877 -0.5638978056181221 0.472630119006924 1.461606818256556 -0.0589340215798704 -0.14661273478697723 -0.0626252035941798 1.455763293160926 -0.1441893672300659 1.451490255424176 -0.5694173468253478 1.9932506970727344 2.000009999999994 -0.0430427737126982 0.467229310132478 1.982236078476236 -0.32529438586249 -0.0459143172387904 -0.050994946174076794 -0.08136748857314345 -0.56320309058466 -0.1262790760868502 0.466541762967962 0.463071545141512 0.45743810216474823 0.45991152141593017 0.45967491181548 1.989659272981014 -0.146188176110687 -0.568252848508256 1.9901871004334017 1.7982960092940943 -0.1448852134919434 -0.0914951504978026 -0.66582427753329 -0.1463265716708894 0.4653497000298098 -0.004542326436234624 0.0574511724148238 0.0574511724148238 1.980871107649682 0.056420402692208 -0.571336877740404 -0.147134485940281 -0.1394100682551164 1.9702550725779 -0.054282630655983 -0.5591863271864259 -0.567061029813412 -0.144398788325435 1.992345655989906 0.46472337949046 -0.1261881121937812 -0.1437387572270666 -0.0805080639001646 0.464578407587312 -0.110258463726755 1.7705818650907041; 0.11125965616186761 -0.01749531547013788 0.1848549524007316 -0.003971530534057983 -0.033855195875345405 -0.03328844270121141 -0.0335996477453926 -0.029750627137549793 -0.13619192715455997 -0.1369702835663832 -0.1374761892769756 -0.18449100747797242 -0.03963057000477 -0.053456133722767404 -0.1370484606599576 0.0174027457435593 -0.13699928675832604 -0.1366999221582218 -0.027407894861026207 -0.136848839566666 -0.1370735535238956 -0.1369411084477162 -0.1362730582021808 -0.13638134207557778 -0.13325548871048803 -0.17988688282756937 -0.027610405108156797 -0.027676205726190405 -0.052054237705792004 0.008650847322835767 -0.027296291544512803 0.008707383106200096 -0.1764243743154172 -0.1329382021450108 -0.1827795485857558 -0.18370099350081742 -0.1829160409891948 -0.16508136569266096 -0.030213650650112205 -0.187660318092618 -0.13192916364187518 -0.0360366481938854 -0.035877549403499805 -0.15094865377733022 -0.18414170450666287 -0.028753982041302403 0.00289916536093034 0.007748594288816758 0.003665464406959899 -0.028668904441374002 0.00868051513501298 -0.028802115858902015 -0.15718641665310787 -0.042030615754254 -0.0430427737126982 2.0000099999999943 -0.1810531868317808 -0.0428662020344 0.1634165499074246 -0.018569353365799575 0.028701729808880597 -0.027006113854168984 -0.157442918410826 -0.031006423132103 -0.1848977578266076 -0.1832108071580156 -0.1862068470995389 -0.18401128593365837 -0.1833464791041334 -0.0430716794181754 0.00783328243987302 -0.1587984385661342 -0.04316613467618781 -0.0453624072493094 0.00803223393755314 0.028479130593112592 -0.1891767163612792 0.00771126699720056 -0.18271771020289418 -0.0033533513273606985 0.01672870928328858 0.01672870928328858 -0.043744520119171 0.0202558485246748 -0.1574062752954192 0.00816490399673332 -0.0305716923718298 -0.0439451574813502 -0.0261959331666698 -0.157539092178086 -0.1570331668898934 0.00843148259562622 -0.0432736431995274 -0.1824908063438662 -0.09656046081158177 0.0095838765743278 -0.0283023230007698 -0.1844808518980442 -0.0252831253583272 -0.0339506271250278; -0.3269069133171101 0.334128151093278 -0.632327051580724 0.10906509052134859 0.613770041032582 0.610793962568172 0.609607555859912 -0.24575248875851796 1.1847236034075557 1.188274273653616 1.189321330764572 1.72297740704557 0.21926455886049198 0.53094376971942 1.189574097243268 0.110474221910595 1.190695704691792 1.189803893926414 0.422458274914998 1.192591001304072 1.191324001017246 1.188872637106574 1.188970660627322 1.19391831261957 1.19119327478525 1.736179603811928 0.423137757342192 0.424838395664208 0.522121225375532 0.366906911044398 0.421485291944608 0.3673482614476 1.723319140386736 1.185982743671394 1.725129005415358 1.73800464385588 1.736166417737592 1.487754791418134 0.4394029415831482 1.70169285300895 1.159325004498398 0.460438385208196 0.459835055842888 1.472954990990228 1.979302039723672 0.618064268181864 0.0754916634924354 0.36916728828165 -0.1881159509143814 0.617490203593926 0.365279121460426 0.617382140756104 1.47388115231371 0.466222396851054 0.467229310132478 -0.1810531868317808 2.000009999999996 0.46190046871783 -0.726280999694254 -0.107713368959549 -0.104006445870302 -0.290015623564352 1.464226814875654 0.30834957480787 1.984765464237028 1.954943865251812 1.956616861207356 1.956468930565754 1.956750916102424 0.464241139183152 0.369468904919268 1.478392443006508 0.464516943727522 0.411895077149074 0.370465049275712 0.1903712671256862 1.0061484442026 0.369792107080332 1.970633238850794 -0.0474763332117504 0.1150997364345468 0.1150997364345468 0.468788250317376 0.284045459450312 1.471391689845576 0.369923916970132 0.344887912997646 0.468078539787218 0.1284460273766004 1.44773850035016 1.474831780735466 0.367462715184346 0.465263151362778 1.97022519300221 1.21198103561726 0.369203387220314 0.1121509134123546 1.96755144759429 -0.01948300787014646 0.416673366923584; -0.44902710498381 -0.1864007919160652 -0.292211537652602 0.433329196431518 -0.257266082718288 -0.255955170653256 -0.257117233167734 -0.058965954391791 -0.31464105774674794 -0.320041985278646 -0.31933614760408 0.5090987530703441 -0.08204922268112083 -0.19221240520951577 -0.31971917525662 0.034324237982611 -0.317310071383474 -0.321147278629838 1.822663873667364 -0.320267773925472 -0.319592637423464 -0.321292790895088 -0.319496875486816 -0.319373882001068 -0.321732222582522 0.5115406908008882 1.822742101704044 1.82339949650847 -0.194390630523743 -0.1422602832616534 1.811911835836732 -0.142028377001675 0.509236583937404 -0.324526047477568 0.514080609197706 0.511844468704008 0.51138576348656 -0.47953907047183 1.879721951117832 0.519233355191498 -0.453781163000646 1.954003142060334 1.9536029769257 -0.563556966389964 0.468431583691458 1.449497392879226 -0.0588366555765448 -0.1444373696603136 -0.0615017969488408 1.447955825245752 -0.141993958286785 1.4436667986939165 -0.56899706890911 1.981427469417186 1.982236078476236 -0.0428662020344 0.46190046871783 2.000010000000016 -0.328642301683618 -0.0465300958825696 -0.0515565783459118 -0.081862298771758 -0.562918588631008 -0.127569882853255 0.461115438559598 0.458211560986606 0.452032897695452 0.454283114624334 0.454175711729664 1.97784355154956 -0.1439722291161136 -0.567954332603402 1.978335490214946 1.800328731555644 -0.1426181629615656 -0.0918442725885598 -0.665369879960062 -0.1441526681455612 0.459838758096554 -0.0047080082869274 0.0560501038626296 0.0560501038626296 1.970312808657856 0.0587928074209008 -0.57106908003037 -0.1449616099860712 -0.1408040214641438 1.95970935083274 -0.0514651101881518 -0.559780805327092 -0.566780982798976 -0.1422130703514314 1.980627729703522 0.459237661948304 -0.1266118536958314 -0.1414302314418952 -0.079646796208893 0.459056180507968 -0.1092904525499058 1.768940187141716; 0.342901678427432 -0.10974745459061919 0.501913338610464 -0.0789947356735186 -0.206335522650284 -0.206379594285396 -0.204892521492684 0.179382101660847 -0.38400311625056993 -0.385386879944946 -0.3857188594095 -0.641508137826268 -0.081474639480053 -0.17140539673783 -0.383369790023848 0.001347164627409165 -0.380536070429888 -0.380069368056266 -0.303884318708952 -0.380895806092596 -0.38251902193617 -0.381610444219572 -0.3810567340858 -0.380168791607936 -0.380599150044554 -0.642331171578038 -0.30239710645584 -0.303507482288628 -0.1710418870529044 -0.1274429311382528 -0.301542378715826 -0.127436977298756 -0.636590569080592 -0.37990630090782 -0.640043564435122 -0.643295452090924 -0.643979481215574 -0.477093689285048 -0.30759328665796204 -0.631172538297252 -0.358032806973864 -0.325493471308552 -0.325518121538022 -0.46256559886912 -0.72565294232141 -0.340507166830422 -0.011348769627791942 -0.1295533564859706 0.0708259413092672 -0.340194446431234 -0.1306470903738604 -0.33894856809028007 -0.466415989606066 -0.324362206727396 -0.32529438586249 0.1634165499074246 -0.726280999694254 -0.328642301683618 2.000009999999996 -0.10426686187068619 0.050848610148868 0.1441979945763836 -0.46093344399412 -0.1034639792515684 -0.729776430579664 -0.7321879157438 -0.7333866564155559 -0.7265104654870921 -0.726314017350472 -0.32581424763940403 -0.1301907115584678 -0.464165493678804 -0.326132817010242 -0.295775780408822 -0.1297537148274148 -0.07121774222608461 -0.260847666122104 -0.1301867008058904 -0.726289456182526 -0.00030462024908436275 -0.00265825025389324 -0.00265825025389324 -0.32071795901068 0.056004008279127 -0.462931885337558 -0.1298409220214598 -0.10322384914782 -0.324355160686022 -0.0019529487045342984 -0.467296379543784 -0.46519740025474804 -0.131564378346175 -0.32370403228861994 -0.7250602616120061 -0.459787761138444 -0.1319857004388448 -0.0224639486111336 -0.7268323290532839 0.007260204377229861 -0.334416303902004; 0.029803994881861005 0.023670518203683596 0.0909616508575788 -0.01712455101441538 0.006452945567395018 -0.004478590773016482 -0.00451528112450476 0.00734618789318976 -0.07788211463879119 -0.08195557342279498 -0.0820284356616558 -0.0896639118592702 -5.749963179223455e-5 0.0235514767255218 -0.0817872517236306 0.01354864426646 -0.0816420801712328 -0.0815417985034014 -0.0442634218501708 -0.0819199193355676 -0.0780745904228366 -0.0818724629801674 -0.0775695051700192 -0.0869842404267454 -0.079456233568143 -0.08907259629445723 -0.0445762713602278 -0.0447513702621218 0.0246845067591134 -0.029112380405497 -0.0441559160789686 -0.0291110203423004 -0.0895213267934 -0.0822352560334222 -0.087956473682792 -0.0892643041225768 -0.0884090350932878 -0.062827518889648 -0.046991805304384814 -0.090360560690039 -0.0778270282189734 -0.0447806333450496 -0.0443640471182932 -0.0634180348801382 -0.1067834509277634 -0.056656971471084 -0.00956729141793946 -0.0295545959033158 0.0255361533402724 -0.0566702166150944 -0.0298041059427246 -0.056505652353625195 -0.0629683097860004 -0.0457807381376694 -0.0459143172387904 -0.018569353365799575 -0.107713368959549 -0.0465300958825696 -0.10426686187068619 2.000009999999996 -0.030592436238089403 0.24760011795746603 -0.0635720008935168 -0.013529412987743322 -0.10759144584817361 -0.1076853838050022 -0.1068869855731958 -0.1082998282801018 -0.10800476117377382 -0.04621801509450419 -0.0298816930176216 -0.06338288761691559 -0.0462928815484524 -0.0586840553020564 -0.0298577388412636 -0.0242549555863262 -0.007679507053245396 -0.0297514735024012 -0.1060290911895202 0.00261729921399766 0.013211443924682358 0.013211443924682358 -0.0467164130081056 0.026898830377655204 -0.063002457963004 -0.029620197315951 -0.0397209288552532 -0.0468138338878416 -0.010745687813702802 -0.0646094442694592 -0.0627659203551572 -0.0295157122579552 -0.04610033348590759 -0.10538430553128939 -0.0570548963629228 -0.0296111033789492 -0.01030194581694262 -0.1068898845071442 -0.00199825153374103 -0.04677833318723561; 0.036537282753455205 -0.0363175731738038 0.057014125749948796 -0.0320993208382518 -0.05448413735998281 -0.05458462195529 -0.0545909227593894 -0.0149839969297924 -0.04542717715423679 -0.0454948546327726 -0.045601105254602196 -0.0951108535323732 -0.01590595098552282 -0.04348787791849519 -0.045414096817148204 0.0110885233669381 -0.045253802783767 -0.04519185977997521 -0.042376857036071205 -0.0455156256149488 -0.045585045252288 -0.04542211817145699 -0.04511826927853601 -0.0453802909377058 -0.04673976313269459 -0.09463715310539841 -0.042634853881147794 -0.0427759890135258 -0.0432282703044846 -0.04143467512442639 -0.0423197057189402 -0.04143273939207081 -0.09512396725332478 -0.0456433167743684 -0.09385713849455178 -0.09478636183129421 -0.09407144368755921 -0.06974608646500022 -0.044470875297266 -0.08421149479723938 -0.0377630647496336 -0.050196195906291805 -0.04986790679178999 -0.06618732295942939 -0.11848106153820502 -0.06881876278457599 -0.007830092248325482 -0.041689851893588016 0.036932428646491794 -0.0688596601870478 -0.04204181192792398 -0.0687246450769114 -0.0657353201367724 -0.050890658608872597 -0.050994946174076794 0.028701729808880597 -0.104006445870302 -0.0515565783459118 0.050848610148868 -0.030592436238089403 2.0000099999999943 0.017296046661800103 -0.0597544972514534 -0.0220326310523626 -0.1090533917880706 -0.1042613424330868 -0.1118662709485366 -0.1047482607604728 -0.1044861900984044 -0.04934769896599659 -0.041971867078146596 -0.05328541068969699 -0.04940364803563639 -0.049921419498335604 -0.0420172908947504 -0.01985081580810416 -0.0333416993172124 -0.041831295324188805 -0.1162446240878444 -0.004045429767495919 0.01081255081975144 0.01081255081975144 -0.0516069860911466 -0.00495169596351754 -0.052988867335089 -0.0417823895545906 -0.019061384634937457 -0.0516799694138918 -0.0087945190731026 -0.0606706915394658 -0.05904187157949059 -0.041729173669620395 -0.0511136487973834 -0.1114962988827454 -0.0814986906685168 -0.041878447669008806 -0.00843135037495124 -0.1117369002539144 0.00151542714222769 -0.0426433389971806; 0.1515327797090082 -0.0255623133428966 0.252001515283496 -0.0380465583103734 -0.12470084005590881 -0.12101474931083 -0.121081992269888 -0.016230305903146044 -0.1527055084696822 -0.1527599079287876 -0.15301128547459342 -0.266311778919864 -0.0408447332422288 -0.1204585937055592 -0.1522150335850972 0.0374487824558946 -0.1521648736965052 -0.1495052327650618 -0.075125576245344 -0.15251329220104182 -0.14992266045180058 -0.14975839516349437 -0.15055898086445338 -0.15285059117316677 -0.1538217277536922 -0.257405979762156 -0.07450705091795062 -0.07872897936035342 -0.1150410306858748 -0.03075661022467805 -0.0736584578735408 -0.03075514662879138 -0.26479604432761594 -0.14865744437958917 -0.2623874936123799 -0.26418988694890194 -0.2631664560477482 -0.22319796269857398 -0.0761398176541596 -0.259731454972974 -0.15016143576968993 -0.08302138746245741 -0.0826610983794358 -0.21611500821737603 -0.28577088145578794 -0.08558009747944142 -0.031312032822376615 -0.0317513585721482 -0.004718622947192937 -0.08595221447407278 -0.030897115683359407 -0.0860306763946494 -0.216920540913318 -0.0811821571874368 -0.08136748857314345 -0.027006113854168984 -0.290015623564352 -0.081862298771758 0.1441979945763836 0.24760011795746603 0.017296046661800103 2.000009999999996 -0.219893748821274 -0.023839285697582605 -0.290659728166034 -0.2844453228545741 -0.287786146513332 -0.29171311582515 -0.28880637034689804 -0.077858125648557 -0.03139366212656939 -0.221716789184404 -0.07805393683930519 -0.09687996808529438 -0.031578836758831996 -0.014290578542032242 0.11117525319276979 -0.0314553172840778 -0.285953174532128 0.00033449910688898403 0.0365849518253196 0.0365849518253196 -0.08173647281697638 0.01229973965713662 -0.216873311158612 -0.0312643808849204 -0.0489063711176948 -0.0780424894224618 -0.0365666043281592 -0.22140958076900397 -0.217269708198564 -0.0305012865131518 -0.08214113900994338 -0.285942465636952 -0.16865745814026004 -0.029462374050117403 -0.0014571799085781826 -0.28584486413705396 -0.023810788094523 -0.089076624597975; -0.07089980365066421 0.0614915554767196 -0.435182941980378 -0.10092905385572519 0.19929529838261997 0.19985817699509117 0.1999485499729938 -0.1681775346798106 1.5114240455479118 1.516185038801612 1.51557712240322 1.639768932041908 0.28882403470292595 0.66035613151298 1.52074538655041 0.0761594985889974 1.518434200014072 1.518862994419782 -0.521742632626482 1.52088768980816 1.521846983171746 1.520904518854826 1.518076708245562 1.521008561168936 1.51841801756947 1.6466348726714495 -0.52194646344004 -0.522594891685904 0.680519905717298 -0.505550339089268 -0.519075079730708 -0.505431315873104 1.638659772708806 1.51638471361599 1.633475633901288 1.649211388012714 1.65577280892078 1.92014702853976 -0.5372821619038137 1.60866159019174 1.563790805349864 -0.558392770501556 -0.558052500342216 1.950727271075902 1.450470252523822 -0.788837065470154 0.1294563829248108 -0.506274263056132 -0.1325304703615948 -0.786101094840394 -0.507969814829798 -0.7870459844439504 1.973239943465366 -0.563873601160842 -0.56320309058466 -0.157442918410826 1.464226814875654 -0.562918588631008 -0.46093344399412 -0.0635720008935168 -0.0597544972514534 -0.219893748821274 2.000009999999996 0.41960581380164 1.4623583759164898 1.4352192877327938 1.440323635622494 1.439846898743124 1.442433657175704 -0.5643259343045461 -0.5063787871052359 1.961511802210964 -0.5638345643001621 -0.516390301415184 -0.50343907228696 -0.274780653331656 1.590854936792682 -0.507893484237404 1.46720351376564 -0.038004449355346005 0.061170231758203596 0.061170231758203596 -0.566755032493332 0.20655873785406803 1.972732985847546 -0.505674831261566 0.482257400402446 -0.5626206745868139 0.1741459616125022 1.944789705513934 1.97107740063401 -0.5052570630294881 -0.5642984419694699 1.470061151855948 0.942167981732462 -0.501983782641092 0.1688066768029012 1.468674672812962 0.03823928058258999 -0.50723519540999; 0.07665957102475 -0.1479991960307148 -0.262187731486076 -0.0377453841917496 -0.1874560585939726 -0.1864364959291206 -0.1859646257819202 -0.058161037963753 0.5082398199415519 0.506945715538876 0.50793919653215 0.353111508646924 0.02071017598497001 -0.1358372152694424 0.501296511002728 -0.0008258516885011145 0.505501206574426 0.508968440506602 -0.116829739000513 0.502107861964872 0.50357774811483 0.505803107361028 0.502589783914118 0.510593694007492 0.504254428831042 0.354833033282776 -0.1170069890985934 -0.1171670384931954 -0.134095639751663 -0.11311022216078681 -0.1163736814303172 -0.1129531887508784 0.345755489552976 0.504566471251076 0.347706040410824 0.352539261054428 0.354048991976746 0.412354558700968 -0.12072916279823659 0.322115228791586 0.49339485720462 -0.1250571269114794 -0.1246593399715664 0.41820005056616 0.30448720622723 -0.1761480805498964 -0.0195164260493617 -0.1148090057927122 -0.0359806445368112 -0.176054018184926 -0.113128858228699 -0.17585954541076604 0.4202502376325 -0.1263504029806642 -0.1262790760868502 -0.031006423132103 0.30834957480787 -0.127569882853255 -0.1034639792515684 -0.013529412987743322 -0.0220326310523626 -0.023839285697582605 0.41960581380164 2.000009999999996 0.3086905217768481 0.301977839221828 0.3063514203481659 0.305484977839568 0.3023801968269241 -0.12500726832370315 -0.11459185876004982 0.419246059166058 -0.1258495629557332 -0.1230106635556648 -0.1139142251695878 -0.0722240200868988 0.33943635741758593 -0.11475625311452602 0.312792054038974 -0.0091421515985455 -0.01699741931000158 -0.01699741931000158 -0.1255515140222918 0.020192831021579996 0.421562527888518 -0.1139983567908934 -0.11198772332773341 -0.12323621091136061 -0.054701188976105014 0.414580387530558 0.42224156989252193 -0.11320107702313578 -0.12646266519121158 0.30563471101771006 0.4609082604532859 -0.11155823346622 -0.027387989628974596 0.30911603732790005 -0.00675851028492072 -0.102138084920671; -0.32735170321486606 0.33365469869051 -0.63181208616348 0.11115327132720301 0.6149713035308161 0.611444677176006 0.610258016140776 -0.24259369326353597 1.184659055534008 1.18801680298954 1.188742998017144 1.722360274648468 0.21818932517903997 0.530922465531174 1.189525377831622 0.1105663240213984 1.189986677436286 1.189616321650208 0.422257186335326 1.192409863516032 1.19145827952372 1.18878018560793 1.188770451958248 1.193736393900168 1.190478779012006 1.7354443336818581 0.422937952562682 0.424641620351828 0.52157144561165 0.36483900404273795 0.420588635977902 0.365280896629966 1.722542866148962 1.185775368008716 1.72459823599414 1.737272871984338 1.734749818475938 1.48677149683327 0.43781009460971987 1.699621423618284 1.15891036931341 0.458964527957344 0.458357486967198 1.47081613410132 1.9758003919118 0.616720808986114 0.0814110665404096 0.367984844638028 -0.1799436814143892 0.61571701225045 0.36320266875666 0.616034714003902 1.472478749083816 0.465532871356394 0.466541762967962 -0.1848977578266076 1.984765464237028 0.461115438559598 -0.729776430579664 -0.10759144584817361 -0.1090533917880706 -0.290659728166034 1.4623583759164898 0.3086905217768481 2.0000099999999925 1.9599646461825118 1.9593877837662157 1.9627318499302042 1.9615688185706017 0.46543379028764004 0.369606697059672 1.477952106100046 0.465165029070672 0.413017197490244 0.369434340119876 0.1887915598736118 1.00770570530766 0.36861265792893 1.96806527613869 -0.0475469021209436 0.11522445689621161 0.11522445689621161 0.468111045411852 0.2843214817087081 1.470211982520462 0.36803781337066405 0.34468470886829794 0.46740194368809407 0.12876633687107777 1.4479606556014941 1.4724433206588379 0.36626796224021396 0.464577538468574 1.9673484014824 1.213521291613302 0.36688628009176194 0.11290906606492498 1.9650239370306797 -0.021133579603573798 0.41785231618795193; -0.317765535590652 0.330106419626658 -0.61746965585742 0.10952514383309561 0.609094282263652 0.606819283735428 0.605848775845572 -0.241117809520786 1.158391705186126 1.161779804624866 1.162299694535892 1.6892851833953342 0.21210592250427798 0.5217627708772501 1.163481396310716 0.105569939859791 1.164493062420836 1.164126069794912 0.419407231357818 1.167175813624938 1.165946720041458 1.16309846990721 1.162428611218676 1.167695744722794 1.16480991113884 1.7023205996806763 0.420125390796104 0.421862677037542 0.511831178794354 0.369030956230098 0.417631026532832 0.369571214606338 1.689652782675302 1.159893739121686 1.69142445892559 1.704597980303164 1.702481635422396 1.456454819263222 0.43389678755070776 1.668474935574968 1.133497872935574 0.455166098191172 0.454512100931156 1.442317736720956 1.946440235363056 0.616519281244046 0.077979143784848 0.370948276204128 -0.1860826400149448 0.615744875127594 0.367375972215488 0.615818367489948 1.443462665249488 0.462050069157134 0.463071545141512 -0.1832108071580156 1.954943865251812 0.458211560986606 -0.7321879157438 -0.1076853838050022 -0.1042613424330868 -0.2844453228545741 1.4352192877327938 0.301977839221828 1.9599646461825118 2.000009999999996 1.9744916261149863 1.9768590393580157 1.974551883883828 0.463437260566268 0.37223217222697 1.4495668569201117 0.4613119032166739 0.41048895358337595 0.37211642609867607 0.19538258876429845 0.9870475109613162 0.3713537348654259 1.9393332481149963 -0.047580144447963 0.110321940132108 0.110321940132108 0.46509415000076004 0.280083148893362 1.441310530035766 0.37049858626626997 0.33671271109215994 0.46336372458267605 0.121596881287375 1.423564764110762 1.442536148311888 0.36968924341316406 0.46116396621938605 1.9366365002186483 1.2032768751258518 0.3696174607459601 0.10691075847127418 1.9342952776682318 -0.019525557850490236 0.4171295463633901; -0.31703040909064406 0.325632838562562 -0.615445987215406 0.1092993681153854 0.607904148509878 0.605648834176398 0.603799108965848 -0.239732465134794 1.163973762120228 1.16735674388242 1.167855888428198 1.6913382188043702 0.211270018689816 0.5232195531246221 1.16901867096029 0.1046315453770322 1.1695122770330661 1.1691330767301982 0.41210746989039404 1.1726894468879439 1.1704393645074977 1.1681425513529942 1.168015160266446 1.173203388263856 1.1703953895688521 1.704876754103416 0.4128223217080921 0.4145508736361901 0.5144586049949139 0.36738013526759006 0.410315589876576 0.367916094362118 1.69235626764926 1.1655197251066762 1.69418680136781 1.7071334485218905 1.7051390987610817 1.4624785381544898 0.42744917362149204 1.6694458163259502 1.1377419519287797 0.44954536488736785 0.44889037870193 1.4470011276244523 1.948178354015212 0.6113973642012698 0.0836101542188232 0.37017056154204403 -0.1891684795375411 0.609922170450464 0.36584055750245603 0.610699817546024 1.4493783125344073 0.4566199981465801 0.45743810216474823 -0.1862068470995389 1.956616861207356 0.452032897695452 -0.7333866564155559 -0.1068869855731958 -0.1118662709485366 -0.287786146513332 1.440323635622494 0.3063514203481659 1.9593877837662157 1.9744916261149863 2.000009999999996 1.9716402418813102 1.968328244796606 0.457601461963348 0.3712821025939859 1.45481979868995 0.45548969114469606 0.405123857357202 0.371169643423322 0.19295292118787896 0.9903789405908939 0.37057295908707993 1.940970060056512 -0.04766111425550519 0.10936295747129778 0.10936295747129778 0.45923817182882404 0.27773191393445607 1.446582517490686 0.369722841037106 0.33839913727568804 0.457522806876398 0.12364924237559279 1.4279390027102918 1.448853573846348 0.36891939040865807 0.45555455455323995 1.9390802944964862 1.205360671910126 0.36868525000464997 0.10522141265135279 1.9359236756730323 -0.01929289546047871 0.4109878077298259; -0.31956892602277204 0.33147351052797 -0.61838377478192 0.1103218614002486 0.608219482697056 0.605978593756508 0.604989117817662 -0.23843922117023003 1.1634584977486158 1.1668050252904678 1.167265765474092 1.692641817741188 0.22131901773298998 0.523031714735358 1.16846858424325 0.1067859474479324 1.169505640769454 1.168614340634656 0.415100285550634 1.172184564462816 1.169902920922656 1.1675996561811257 1.167529772195726 1.1728704484633723 1.170145090308206 1.7061473539302199 0.4158046039592039 0.41754026706078606 0.5132272782658361 0.3661369037621319 0.4133196734701679 0.36668107660247806 1.693158917339756 1.1649892726323798 1.6955262778273377 1.7084830533004023 1.706222336640839 1.461697267105876 0.43028751555592204 1.670265761009844 1.1387945888406252 0.4520293294248039 0.4513842950622942 1.4466340614378537 1.947481906951508 0.612447826296498 0.0854361331610256 0.3686437186856101 -0.18851627845263086 0.6109699235460803 0.363687165794818 0.6117392730812439 1.4477492444226097 0.4588870179180883 0.45991152141593017 -0.18401128593365837 1.956468930565754 0.454283114624334 -0.7265104654870921 -0.1082998282801018 -0.1047482607604728 -0.29171311582515 1.439846898743124 0.305484977839568 1.9627318499302042 1.9768590393580157 1.9716402418813102 2.000009999999992 1.986164061790448 0.46010436532577403 0.37006626084164196 1.4542008287432717 0.45769900470001795 0.40669067524329605 0.370060206677758 0.19350872391068344 0.987305517297536 0.36928598174866806 1.9413902588519818 -0.0484508320189488 0.11149389501352185 0.11149389501352185 0.46182387093416 0.2798609747732399 1.4455530715948999 0.368648296362424 0.33645749224592203 0.46068565658070404 0.12326919269234901 1.4260360546064783 1.44765628138624 0.36679375432521005 0.45798822275450995 1.938109883892402 1.2036053197759362 0.36748749707590594 0.10674344624808162 1.9357390060369217 -0.0211405535944146 0.411328659071288; -0.32145008228598604 0.331787574947022 -0.621197916824386 0.10519132951356099 0.608159081063878 0.605916923242928 0.604933006163428 -0.236913522280266 1.164889028949652 1.167713212384028 1.168178774280006 1.693174079808942 0.21883126381369805 0.523951041913848 1.16937299691506 0.1005059136126884 1.169877718558668 1.169513793520374 0.413233545299446 1.172916296915826 1.171340647409506 1.168506166486302 1.168427422021728 1.173757315710634 1.17107744558779 1.7066480918843436 0.4139393208727 0.415673537271116 0.515506249259842 0.36368259063973396 0.4114453864443 0.364225148547536 1.693688310337582 1.165898095976324 1.696150193884608 1.708977692882884 1.707765128404932 1.463434180864766 0.4297583463320019 1.670691829467452 1.1380928560522 0.451796475592138 0.45115005559934 1.44791903235554 1.94746140897876 0.61060539333368 0.0827029026949694 0.366208261273804 -0.1878670757370144 0.609126466225196 0.361231620937806 0.6094629834850362 1.449992914076618 0.458652555660514 0.45967491181548 -0.1833464791041334 1.956750916102424 0.454175711729664 -0.726314017350472 -0.10800476117377382 -0.1044861900984044 -0.28880637034689804 1.442433657175704 0.3023801968269241 1.9615688185706017 1.974551883883828 1.968328244796606 1.986164061790448 2.000009999999996 0.459869674774952 0.36779152440759605 1.4556165262548562 0.4574662786821519 0.406166138200276 0.367640588184482 0.193781584661792 0.9880726856003139 0.36688350638423994 1.941913115762138 -0.049129230284506005 0.1049569328614826 0.1049569328614826 0.4614325600304379 0.27602638029562593 1.4476825645189981 0.366036486990394 0.337051123989394 0.46046195959362607 0.11955859300508763 1.4290365184148044 1.4504375212716538 0.3643439976210279 0.45775888910197404 1.939666535930796 1.2034027391109439 0.3650247211899381 0.10618117191806278 1.9373199952028302 -0.020497501217292803 0.40980053813913997; -0.4509584560837361 -0.1867223724559976 -0.28965008662715 0.43225358156813 -0.261671828140146 -0.260512880588436 -0.261673538392932 -0.05923235621834301 -0.3112125123443479 -0.316644792851718 -0.315837497157374 0.510914340548718 -0.07877409221870141 -0.19538824246382042 -0.316335934322078 0.035801414024599 -0.313924324736136 -0.317675863516948 1.826663538213326 -0.316684482402068 -0.316003335715004 -0.317695915287022 -0.317068567512426 -0.315978895332732 -0.319336158776898 0.5142242845841283 1.826741095715938 1.82739206267933 -0.1976530152865714 -0.145269420126618 1.814647305226104 -0.1450371782092214 0.512118571485372 -0.321055541819204 0.517361759957974 0.515256595061214 0.51496193015986 -0.476153285853602 1.8786563565694587 0.52236538349532 -0.451517696248196 1.960492679168622 1.960107751093464 -0.564072604822714 0.46939928745611 1.4530912346409197 -0.058986739752528 -0.1477536468711468 -0.061628466165902 1.45156621780604 -0.1453359330947246 1.447287334155972 -0.570567594867972 1.988902820731438 1.989659272981014 -0.0430716794181754 0.464241139183152 1.97784355154956 -0.32581424763940403 -0.04621801509450419 -0.04934769896599659 -0.077858125648557 -0.5643259343045461 -0.12500726832370315 0.46543379028764004 0.463437260566268 0.457601461963348 0.46010436532577403 0.459869674774952 2.000009999999996 -0.14746353433814882 -0.569433506567714 1.9878517927226957 1.7982655451903662 -0.14607433479649667 -0.09163364196143378 -0.6665474076491141 -0.1474282357899464 0.46278665543727593 -0.00454884915318252 0.0575451858336982 0.0575451858336982 1.9766697866903238 0.05643123118335099 -0.5713460848253301 -0.1482788015481198 -0.13946824916980516 1.9679280153492802 -0.05450894088375661 -0.5604554115813679 -0.567244433102126 -0.14573747875685242 1.988017687018912 0.4614956725927559 -0.12716384340211057 -0.1448468362372972 -0.08474752292650801 0.46146802276024596 -0.10930930452589241 1.7676585962848719; -0.0157266699876828 0.7473303994337461 -0.04705701220155581 -0.057010694791726 1.060161872580994 1.054033029975662 1.05332434443566 -0.0704185897003638 -0.407046431771424 -0.40389857597147 -0.405129364532288 -0.5817445776249981 -0.0719827941765472 -0.17624683572703181 -0.407710443749344 0.0291381593144674 -0.40944884942056 -0.404802483094786 -0.1325930057140114 -0.405221468337012 -0.40622083063817 -0.409515674076842 -0.405701131905402 -0.406339968419804 -0.40540988259845 -0.5821224865629061 -0.1311443727097308 -0.1270885696486308 -0.1755343411931768 1.9785336149836419 -0.1281559625395698 1.97862924207017 -0.582421103564162 -0.406717893639896 -0.577188283644524 -0.581495271000148 -0.584326056145046 -0.51079701379524 -0.13018365268746623 -0.580445701099608 -0.399634791929444 -0.1416762788858876 -0.143201247837454 -0.505986133223078 0.374533060342216 1.231332265087056 -0.0399830404587712 1.989330641678778 -0.0385870072406008 1.226834553884626 1.9780875981719 1.2317949971262654 -0.504386158668068 -0.1459497929414074 -0.146188176110687 0.00783328243987302 0.369468904919268 -0.1439722291161136 -0.1301907115584678 -0.0298816930176216 -0.041971867078146596 -0.03139366212656939 -0.5063787871052359 -0.11459185876004982 0.369606697059672 0.37223217222697 0.3712821025939859 0.37006626084164196 0.36779152440759605 -0.14746353433814882 2.000009999999996 -0.5064570087640001 -0.14601213527613138 -0.1348760635104484 1.9868290452321562 1.0775677616142143 -0.5722432312332537 1.9959292439945764 0.37117296936715394 -0.011480042563724022 0.03559198737704981 0.03559198737704981 -0.1464901367229826 0.06584631450471441 -0.510473215832702 1.990716304323878 -0.14572694360094843 -0.1465002785027834 -0.048199168708744605 -0.5034774503835859 -0.5060905868185019 1.9836511298951358 -0.14629317374745965 0.367053849258812 0.6335747096085838 1.979709274765164 -0.0379358955451882 0.364262450655634 0.00591041700117302 -0.11148791274569822; -0.06512987854116341 0.0635306419944254 -0.430093953714458 -0.09893638315118242 0.22575441492882797 0.225135597207618 0.224976744171208 -0.1702060031014684 1.507601816811292 1.512334779917522 1.51165926942248 1.6529406050395217 0.2868237532266061 0.694424569236658 1.515851990551322 0.0771775470821854 1.516221107372078 1.51500594185277 -0.52847641031646 1.517003236998288 1.516043195001128 1.516349262459552 1.514305897269 1.51716698120348 1.5142435386433 1.660348195053234 -0.528548271700834 -0.52919786142762 0.686339889706998 -0.505470617904108 -0.525688779837652 -0.505219168972386 1.65116121176557 1.510928227264624 1.648008205026794 1.663850047800842 1.669413641377986 1.937151769559144 -0.5439403245693339 1.62185107051218 1.558354916732516 -0.56155280505825 -0.561202126701636 1.966156971062204 1.463965683005764 -0.792605068179738 0.1271115753258574 -0.505199504271952 -0.1270184940291096 -0.789349095436784 -0.506821703309282 -0.7895859398681042 1.970762877956694 -0.56877590893936 -0.568252848508256 -0.1587984385661342 1.478392443006508 -0.567954332603402 -0.464165493678804 -0.06338288761691559 -0.05328541068969699 -0.221716789184404 1.961511802210964 0.419246059166058 1.477952106100046 1.4495668569201117 1.45481979868995 1.4542008287432717 1.4556165262548562 -0.569433506567714 -0.5064570087640001 2.000009999999994 -0.5714452105509719 -0.518533437904622 -0.5048241502321 -0.2772136673162702 1.6033475150104903 -0.5080571371783779 1.4623809280693083 -0.03790227197187601 0.062172457460085005 0.062172457460085005 -0.571864032287344 0.21074206524017994 1.9732248700827122 -0.506611183842518 0.48095016847599 -0.5683711280946723 0.17779798315145437 1.9400211825372602 1.9706247764907043 -0.506342859284426 -0.569046066230392 1.4643464474080723 0.9353438183491599 -0.503999119338702 0.17180191232684378 1.46409420733702 0.03395183526526759 -0.511908694468068; -0.44689068979544405 -0.1910011564899292 -0.292930422958788 0.43130920968429187 -0.262246401987818 -0.261087523136404 -0.262249082993938 -0.0593570452785034 -0.30948503721423803 -0.31491838867449 -0.31423925201511 0.5112199494371541 -0.0806957266418842 -0.19661745894492078 -0.31461603960149387 0.035874591204189205 -0.31235837236554603 -0.31630473800934206 1.8241594939328878 -0.315120497528018 -0.31443550772884793 -0.3163223087449801 -0.31549219711088394 -0.314412187115254 -0.31777571459971 0.5143769549682758 1.8242359872561598 1.8248873410297923 -0.1973169338769773 -0.14453334492972345 1.8134816804139218 -0.14430092643946005 0.5129930147152361 -0.31927770679931206 0.5168061109758879 0.514700047348126 0.514407112045998 -0.477134497138474 1.88081153898071 0.5225467647907439 -0.45101010556533405 1.960758805900814 1.9603756236858516 -0.5643623703391659 0.4699220051963237 1.4542216144754634 -0.05907382049916678 -0.14670213072475574 -0.061709744792359175 1.4526996096499518 -0.1442723908555253 1.448421194012604 -0.570044907298648 1.9894311988859 1.9901871004334017 -0.04316613467618781 0.464516943727522 1.978335490214946 -0.326132817010242 -0.0462928815484524 -0.04940364803563639 -0.07805393683930519 -0.5638345643001621 -0.1258495629557332 0.465165029070672 0.4613119032166739 0.45548969114469606 0.45769900470001795 0.4574662786821519 1.9878517927226957 -0.14601213527613138 -0.5714452105509719 2.0000099999999965 1.7955729244251262 -0.14466855437976195 -0.09216080532215601 -0.6677636581083198 -0.14637803476218317 0.4635848696733121 -0.004547653645494899 0.056185561996581805 0.056185561996581805 1.978327085260452 0.05660194781432941 -0.570863492623234 -0.14722300554412143 -0.13912355822157793 1.9681724561705918 -0.05446447809488019 -0.5588387973096601 -0.5667299763331459 -0.14448312228558657 1.9885417134049161 0.463048222150206 -0.12656746480529346 -0.14382113575382222 -0.07993939795789462 0.4631790410224519 -0.10983304241949599 1.7772994896382501; -0.37987573590117607 -0.19186548525174 -0.258944989331928 0.417532007726124 -0.253440501116064 -0.252296705898104 -0.25327132548501996 -0.0369152568542118 -0.278836456005206 -0.284294915537444 -0.28256278891634 0.4590990119172861 -0.0695685042233066 -0.193251593287642 -0.282878347265994 0.030387944741091615 -0.281043754772232 -0.28279600431095403 1.6610109754026459 -0.28278415832392 -0.28540581329496206 -0.28499167440393797 -0.282123596473932 -0.2831047393952 -0.28766227773932396 0.46068600277271987 1.6567584036878418 1.6575035113128063 -0.1957065966429201 -0.13560512886677187 1.65005720455791 -0.1353639639233486 0.46001192844016 -0.286735764094704 0.46399746762775 0.461303990859066 0.4616527587883162 -0.43718051656037793 1.70330042702005 0.46910641475092624 -0.39971319324095395 1.764366912723302 1.7638854639522257 -0.513301133997904 0.4167603186251718 1.3113617121382022 -0.05001562631607256 -0.13777781538177272 -0.05769722183911235 1.307738823900992 -0.13445802614067462 1.302527513177398 -0.517319400320458 1.7941340150184617 1.7982960092940943 -0.0453624072493094 0.411895077149074 1.800328731555644 -0.295775780408822 -0.0586840553020564 -0.049921419498335604 -0.09687996808529438 -0.516390301415184 -0.1230106635556648 0.413017197490244 0.41048895358337595 0.405123857357202 0.40669067524329605 0.406166138200276 1.7982655451903662 -0.1348760635104484 -0.518533437904622 1.7955729244251262 2.000009999999996 -0.13352443593337437 -0.0821453217897472 -0.6060425163702999 -0.135540605992438 0.410449211998402 -0.0007767021253449334 0.051374429857749995 0.051374429857749995 1.7835614523536918 0.053307315860718005 -0.5200815005170022 -0.13656015702411278 -0.11737101556731902 1.77751177517137 -0.0437227987931658 -0.5180893730704221 -0.5150749518487661 -0.134624118303168 1.7939242502772519 0.4102557225744521 -0.15700511683916318 -0.1366550028000658 -0.05468396718085958 0.41062728002870597 -0.05970823806396601 1.7177355497773505; -0.01496287037262112 0.74414277283952 -0.0449532907483468 -0.056959033626946 1.057162086099642 1.050910991530852 1.050206099721468 -0.06915958542380822 -0.40459540992754195 -0.403053113118334 -0.402666233684388 -0.5781230420396462 -0.067760771795004 -0.17570042814818837 -0.405261562627326 0.0291137051973266 -0.407001172719588 -0.40234256741699 -0.1311773767388252 -0.402003237670828 -0.403758299476094 -0.407065873198158 -0.403501272534264 -0.40388918204482 -0.402930886964736 -0.5785048416021479 -0.1297211995922838 -0.1256504048473944 -0.1749433160528126 1.973025798202698 -0.1265009965604764 1.973121946824992 -0.578779496978468 -0.404250327898502 -0.573764382024822 -0.578105468710722 -0.580842382862152 -0.508092091120306 -0.128566792234397 -0.577140743396884 -0.397338786904044 -0.1388365357752518 -0.1403648656622718 -0.503133342689172 0.375532216508984 1.228639657934744 -0.0399687923106368 1.9839788563008 -0.0385716882747636 1.224114581464174 1.972548116086196 1.2342131556328684 -0.502486910735916 -0.1446451841932408 -0.1448852134919434 0.00803223393755314 0.370465049275712 -0.1426181629615656 -0.1297537148274148 -0.0298577388412636 -0.0420172908947504 -0.031578836758831996 -0.50343907228696 -0.1139142251695878 0.369434340119876 0.37211642609867607 0.371169643423322 0.370060206677758 0.367640588184482 -0.14607433479649667 1.9868290452321562 -0.5048241502321 -0.14466855437976195 -0.13352443593337437 2.000009999999996 1.0738835100352901 -0.5699144978491258 1.9906165130681177 0.3718890300527418 -0.011522662431429662 0.035589461398773 0.035589461398773 -0.14514995411258078 0.06635562646842057 -0.5085991730907519 1.987114586580358 -0.1453781793797488 -0.14515929146638462 -0.04744777372197259 -0.5002031817081841 -0.5052597711049723 1.9782212384353897 -0.1449926348026172 0.36786076974689785 0.6325925800708841 1.97417935395075 -0.037783033042693204 0.36509622266057606 0.007369774181492303 -0.1085869650916284; 0.208808038224008 0.588163018837498 -0.1521225585574358 -0.0434739350317378 0.572445899672658 0.577528224667252 0.578377099347008 -0.011297129585522397 -0.227860331635208 -0.224537767080512 -0.225412135051478 -0.331051450864294 -0.03929595447689202 -0.1056542356365848 -0.226776681418948 0.030671270747427197 -0.228668498192638 -0.226147757521846 -0.0770377754916616 -0.224849250765408 -0.225206951055504 -0.22713824981876 -0.225339025794572 -0.224023449910326 -0.227130572121176 -0.330001841083352 -0.0740554817424872 -0.0741624955888604 -0.1033051935193258 1.079657215795062 -0.0730696924976616 1.079707129834616 -0.325685914792224 -0.22584201004215 -0.328143695565134 -0.329659793626984 -0.328272306316742 -0.281709866054206 -0.08131067527720541 -0.321671243424356 -0.215553279012756 -0.086455639709724 -0.0899096176985996 -0.277067484366398 0.1942904957781912 0.661419542673064 -0.009691826519402438 1.071758234962088 -0.0228825128840668 0.652608894557314 1.066316911210348 0.6552447583586544 -0.274079793930092 -0.0921159570195362 -0.0914951504978026 0.028479130593112592 0.1903712671256862 -0.0918442725885598 -0.07121774222608461 -0.0242549555863262 -0.01985081580810416 -0.014290578542032242 -0.274780653331656 -0.0722240200868988 0.1887915598736118 0.19538258876429845 0.19295292118787896 0.19350872391068344 0.193781584661792 -0.09163364196143378 1.0775677616142143 -0.2772136673162702 -0.09216080532215601 -0.0821453217897472 1.0738835100352901 2.000009999999996 -0.28231323105443606 1.0789552197984884 0.19330824447491643 -0.005730551103833738 0.0294795028028658 0.0294795028028658 -0.09322829790185222 0.04553840287769161 -0.276476963367612 1.0789371229285483 -0.065976452749006 -0.09361426464624621 -0.0333909785560204 -0.27674593838201794 -0.27789312571248614 1.0777539233919378 -0.092452074821497 0.19373339203374804 0.340162044078706 1.0825870066546859 -0.019553809583700805 0.19254071473442963 -0.0220570287166852 -0.06068972643896843; 0.1834869710471098 -0.0512014534263878 -0.06665321321953882 -0.1497492994290674 0.0775583736703264 0.076939184260247 0.0770853284712352 0.1890356650125568 1.20607223822249 1.209117031648736 1.208802795647704 1.227601172860718 0.23717273470148204 0.567626573101854 1.212424026793272 0.1024309067311692 1.212517529448816 1.212762888823296 -0.61621538274729 1.213019785035986 1.21395640457082 1.213804987096866 1.21110072216712 1.216641696573872 1.209672450203668 1.2363220656037417 -0.615510845008294 -0.617235457432154 0.56419731356307 -0.56930766898883 -0.61200558480494 -0.569024766452334 1.228337899566208 1.20882825373387 1.22438757678285 1.236360740543588 1.241595402587384 1.562879299177372 -0.6362699260028278 1.208280106449208 1.258221814392052 -0.659033984636308 -0.658386923931712 1.59549861253429 0.993601989463448 -0.910998730518758 0.0930046020107494 -0.571715484476788 -0.1421492669188834 -0.910060723066462 -0.569374585859762 -0.9093150685733418 1.600618592537498 -0.665582395230438 -0.66582427753329 -0.1891767163612792 1.0061484442026 -0.665369879960062 -0.260847666122104 -0.007679507053245396 -0.0333416993172124 0.11117525319276979 1.590854936792682 0.33943635741758593 1.00770570530766 0.9870475109613162 0.9903789405908939 0.987305517297536 0.9880726856003139 -0.6665474076491141 -0.5722432312332537 1.6033475150104903 -0.6677636581083198 -0.6060425163702999 -0.5699144978491258 -0.28231323105443606 2.000009999999996 -0.5734042532808521 0.993175127332996 -0.04282789978420419 0.08412995078732521 0.08412995078732521 -0.66879854387253 0.27213055811224796 1.603882972651712 -0.5714443810217498 0.39527789309559 -0.66311825885919 0.14755887774224763 1.579768650401006 1.6009721704930862 -0.5703855799307059 -0.665541795451042 0.9978496632892521 0.651111146794944 -0.56904430912726 0.14568835891482562 0.99701110032532 0.0398076452107832 -0.602023185789694; -0.015360113628743002 0.74873962886435 -0.046998362428002795 -0.057065609627642996 1.061873296771698 1.0557790504796922 1.055126344207052 -0.06913323041204539 -0.408223926389768 -0.405086595903802 -0.406314577890976 -0.583341607748366 -0.06835517861308579 -0.17701549145181045 -0.408887964795616 0.0291650002162096 -0.410625289659844 -0.405985279758228 -0.1324804489968868 -0.405646903193782 -0.407404928256878 -0.410661735268112 -0.406985240054844 -0.407517807603846 -0.406573570899004 -0.583716790359416 -0.1310356023905316 -0.1269879818818022 -0.1763067529421102 1.982509629394428 -0.1280515503859076 1.982604978584374 -0.584025589413218 -0.407903434845882 -0.578798137624734 -0.583089215919174 -0.58590947806264 -0.512415018818984 -0.13008891382852744 -0.58200717055356 -0.400661330268192 -0.1400381204375534 -0.1415579324001534 -0.507478958869626 0.374858442680326 1.233907932721372 -0.0408993789371912 1.99323424574528 -0.0386118218013416 1.229426191758474 1.982082728929832 1.234376293376876 -0.505997711615624 -0.1460888525883392 -0.1463265716708894 0.00771126699720056 0.369792107080332 -0.1441526681455612 -0.1301867008058904 -0.0297514735024012 -0.041831295324188805 -0.0314553172840778 -0.507893484237404 -0.11475625311452602 0.36861265792893 0.3713537348654259 0.37057295908707993 0.36928598174866806 0.36688350638423994 -0.1474282357899464 1.9959292439945764 -0.5080571371783779 -0.14637803476218317 -0.135540605992438 1.9906165130681177 1.0789552197984884 -0.5734042532808521 2.0000099999999956 0.3713729390489582 -0.011457760757674122 0.0355959166615992 0.0355959166615992 -0.14655470544794544 0.0664183706715196 -0.5119435995394439 1.9946363666366724 -0.14578579326116176 -0.14664156948823434 -0.048291464212741206 -0.5051121078673402 -0.507566938451382 1.987600442873208 -0.1464319322981996 0.367226004638758 0.6325935049769343 1.983667751364644 -0.0385414445104584 0.3643036574357739 0.005194419994507921 -0.1111285152005986; -0.33410730093819807 0.339352992650176 -0.643929820455668 0.11291260562091399 0.588232745081964 0.586646246420846 0.586148421973928 -0.24175813471905 1.191533183571474 1.1952919731184737 1.196612151219816 1.708173251836652 0.22384014996468804 0.49690234778935793 1.1973226323891821 0.108831442391778 1.197507550114146 1.197538486725512 0.42086122291093 1.199238521999076 1.1977997235410203 1.1949700830263918 1.1955069793458237 1.2005161348937199 1.1971595397684698 1.719822659964758 0.42156148965033 0.42327399364740004 0.5176426819311439 0.3674952123050581 0.41915232959441806 0.3679313386683699 1.7063009407948142 1.192693092098168 1.7108343055649398 1.721700497661878 1.7198024531426683 1.472500225443096 0.43600920397414206 1.6859561230452882 1.1661076743490515 0.458423117136658 0.45779555794751786 1.4585040660658457 1.9639896550717157 0.617682760073006 0.0807545762455372 0.37071345618442797 -0.19004330881679735 0.61666701135035 0.3658632181427679 0.617266822012632 1.47780312360785 0.46434085748982584 0.4653497000298098 -0.18271771020289418 1.970633238850794 0.459838758096554 -0.726289456182526 -0.1060290911895202 -0.1162446240878444 -0.285953174532128 1.46720351376564 0.312792054038974 1.96806527613869 1.9393332481149963 1.940970060056512 1.9413902588519818 1.941913115762138 0.46278665543727593 0.37117296936715394 1.4623809280693083 0.4635848696733121 0.410449211998402 0.3718890300527418 0.19330824447491643 0.993175127332996 0.3713729390489582 2.0000099999999925 -0.04756364356451141 0.12101145857756498 0.12101145857756498 0.46898974514573 0.2803093361786858 1.474933775229876 0.3705661067967899 0.3478655560643879 0.46830390481623796 0.12905702511721234 1.4524480102287158 1.4783039278871837 0.37018598508492007 0.4634226718474859 1.9755105144357292 1.216040346689314 0.37094000314606584 0.10813770910052518 1.97135886939797 -0.016613765454730842 0.41291821280182606; 0.0035374763978456 -0.01046257864775174 0.0290801054461942 1.5178830414797062e-18 -0.01701923375174028 -0.01710575016007932 -0.01709623848189614 -0.005004012410249099 -0.0331469271132328 -0.0331642110245982 -0.0330925658669494 -0.0431530143336506 -0.00377962508749056 -0.01188416033627098 -0.03295668150294901 0.19251420962638097 -0.0316108067608586 -0.0315812982612354 -0.004821957055213119 -0.03156033134375719 -0.032025145144044004 -0.031640469759388196 -0.03170185400875979 -0.0315705576354392 -0.03202863845434759 -0.04116529997388921 -0.004823488614131424 -0.004821787503484239 -0.012010587454743876 -0.011540622306626598 -0.004846130057935867 -0.01154008315539794 -0.04139171534780062 -0.03175146782509179 -0.040972335338218195 -0.04115073861804 -0.015511191675209531 -0.016197815979716226 -0.004713020753265618 -0.040095513688351986 -0.013620619761941611 -0.004586025450803341 -0.004590309497503562 -0.038049309449281206 -0.04811659585771721 -0.011383676551019174 -0.0034554234327494 -0.01251345038986564 7.48099499014998e-18 -0.011398918046007837 -0.011567502129040397 -0.01139874973642072 -0.03782572778978761 -0.004544031519223697 -0.004542326436234624 -0.0033533513273606985 -0.0474763332117504 -0.0047080082869274 -0.00030462024908436275 0.00261729921399766 -0.004045429767495919 0.00033449910688898403 -0.038004449355346005 -0.0091421515985455 -0.0475469021209436 -0.047580144447963 -0.04766111425550519 -0.0484508320189488 -0.049129230284506005 -0.00454884915318252 -0.011480042563724022 -0.03790227197187601 -0.004547653645494899 -0.0007767021253449334 -0.011522662431429662 -0.005730551103833738 -0.04282789978420419 -0.011457760757674122 -0.04756364356451141 2.0000099999999956 0.20032333394670998 0.20032333394670998 -0.004527290827876581 -0.09063493929697661 -0.037782101138417186 -0.011496123929142697 -0.012133499329214802 -0.004525006336265559 -0.00579429799621522 -0.037827866694828814 -0.03776549029405579 -0.010470265047675183 -0.0045309728721297 -0.046905756683545595 -0.038930799026916596 -0.010512689717493163 -0.00809475973228204 -0.046969286572685384 -0.005642143669026776 -0.014075253242538256; 0.07826907631321121 0.0635570708347174 0.0952076939060244 0.025931021438649594 0.058744213320829 0.0584883027230254 0.0587032419682022 -0.024948456575417 0.026550674812263204 0.027095401570100403 0.0276386470539806 0.0808151829946312 0.029744010482333 0.040500472622407 0.027354045436972 1.8001405844262839 0.0475164587115662 0.0473460807734466 0.0524486145835512 0.0474679861674532 0.0379174506919254 0.0414766446627796 0.0408563222636136 0.041191973802334 0.0428737023341064 0.09452460040886801 0.0526107113405802 0.0526488433731698 0.039442312776734 0.035197220265997 0.0524758276563558 0.0351528406935502 0.094383315134568 0.040984278939812 0.0934474020163306 0.0944209869443368 0.0938618057464836 0.0645166714097012 0.05378675041989701 0.0894161952136468 0.0348352034860234 0.0573188060312406 0.0572276761004468 0.0611655475965878 0.1139384700508848 0.0695465072506064 0.01622391688766406 0.035621664793258 0.0156162516918874 0.0695468746366416 0.0352720557242618 0.06964614488527518 0.0610766462582742 0.0574794642936464 0.0574511724148238 0.01672870928328858 0.1150997364345468 0.0560501038626296 -0.00265825025389324 0.013211443924682358 0.01081255081975144 0.0365849518253196 0.061170231758203596 -0.01699741931000158 0.11522445689621161 0.110321940132108 0.10936295747129778 0.11149389501352185 0.1049569328614826 0.0575451858336982 0.03559198737704981 0.062172457460085005 0.056185561996581805 0.051374429857749995 0.035589461398773 0.0294795028028658 0.08412995078732521 0.0355959166615992 0.12101145857756498 0.20032333394670998 2.0000100000000556 2.0000000000000595 0.057885991971402 0.8659677707769978 0.061307279635754414 0.035401187970648396 0.011959307631615622 0.058032556702601816 -0.007741230265143741 0.06057424398394181 0.06105223557937199 0.0352775792533054 0.05756159433186861 0.11344019233828921 0.06294255897656241 0.03522842844878918 -0.027519188898976605 0.1142510155901164 0.018734389084503686 0.05245165751045441; 0.07826907631321121 0.0635570708347174 0.0952076939060244 0.025931021438649594 0.058744213320829 0.0584883027230254 0.0587032419682022 -0.024948456575417 0.026550674812263204 0.027095401570100403 0.0276386470539806 0.0808151829946312 0.029744010482333 0.040500472622407 0.027354045436972 1.8001405844262839 0.0475164587115662 0.0473460807734466 0.0524486145835512 0.0474679861674532 0.0379174506919254 0.0414766446627796 0.0408563222636136 0.041191973802334 0.0428737023341064 0.09452460040886801 0.0526107113405802 0.0526488433731698 0.039442312776734 0.035197220265997 0.0524758276563558 0.0351528406935502 0.094383315134568 0.040984278939812 0.0934474020163306 0.0944209869443368 0.0938618057464836 0.0645166714097012 0.05378675041989701 0.0894161952136468 0.0348352034860234 0.0573188060312406 0.0572276761004468 0.0611655475965878 0.1139384700508848 0.0695465072506064 0.01622391688766406 0.035621664793258 0.0156162516918874 0.0695468746366416 0.0352720557242618 0.06964614488527518 0.0610766462582742 0.0574794642936464 0.0574511724148238 0.01672870928328858 0.1150997364345468 0.0560501038626296 -0.00265825025389324 0.013211443924682358 0.01081255081975144 0.0365849518253196 0.061170231758203596 -0.01699741931000158 0.11522445689621161 0.110321940132108 0.10936295747129778 0.11149389501352185 0.1049569328614826 0.0575451858336982 0.03559198737704981 0.062172457460085005 0.056185561996581805 0.051374429857749995 0.035589461398773 0.0294795028028658 0.08412995078732521 0.0355959166615992 0.12101145857756498 0.20032333394670998 2.0000000000000595 2.0000100000000556 0.057885991971402 0.8659677707769979 0.061307279635754414 0.035401187970648396 0.01195930763161562 0.05803255670260182 -0.007741230265143741 0.06057424398394181 0.06105223557937199 0.0352775792533054 0.05756159433186861 0.11344019233828921 0.06294255897656241 0.03522842844878918 -0.0275191888989766 0.1142510155901164 0.018734389084503682 0.05245165751045441; -0.45105753160426 -0.1788160018148424 -0.293924306218962 0.42885224998999993 -0.2592598180874 -0.258071335626892 -0.259240136532618 -0.0585576610402596 -0.312234658379028 -0.31767032625126596 -0.316928890882602 0.5074776922479922 -0.08111849301786819 -0.19271911708375483 -0.317373569554154 0.028120812073082 -0.314965951148054 -0.318690602329012 1.817503012588226 -0.317711676398122 -0.317232116046418 -0.318911141593202 -0.318058478848986 -0.3169901140363 -0.313578902032208 0.5175304511597619 1.817572933790634 1.818218224980204 -0.1946021168526248 -0.1444086796847304 1.806889208460476 -0.1441753061658884 0.514514184449854 -0.316580414937066 0.520912304581714 0.517854592699664 0.5070140742115 -0.479952949820008 1.8722793290472795 0.526916340947486 -0.455636247044058 1.951602291345702 1.9512271463898 -0.567458918109478 0.47421368394739 1.4645069034861 -0.0595576915748754 -0.1469156132150064 -0.063205989231674 1.463018481036022 -0.1444741016797524 1.458754347709716 -0.572948294765548 1.980118128618888 1.980871107649682 -0.043744520119171 0.468788250317376 1.970312808657856 -0.32071795901068 -0.0467164130081056 -0.0516069860911466 -0.08173647281697638 -0.566755032493332 -0.1255515140222918 0.468111045411852 0.46509415000076004 0.45923817182882404 0.46182387093416 0.4614325600304379 1.9766697866903238 -0.1464901367229826 -0.571864032287344 1.978327085260452 1.7835614523536918 -0.14514995411258078 -0.09322829790185222 -0.66879854387253 -0.14655470544794544 0.46898974514573 -0.004527290827876581 0.057885991971402 0.057885991971402 2.000009999999996 0.05755191713434759 -0.5748640790586563 -0.1474222701224136 -0.1426625394869474 1.9831919026389426 -0.055119044488297984 -0.5618291916437722 -0.5705884917813139 -0.14468752712681274 1.979222381857852 0.46695619926574 -0.12737590975623542 -0.1440230934161526 -0.0862364691716162 0.46713980263624605 -0.11111449404016502 1.7587684265399364; 0.1593946897204906 0.0874953001275356 0.1992696521103332 0.026519938879124398 0.1238895134355204 0.1233496370640692 0.1238030123523062 0.0252890019780794 0.12799820384342558 0.1291847214148318 0.1267779227264464 0.217891164823698 0.0380296429012654 0.09545846290740198 0.1428465741712368 0.8301106761471821 0.1475959039846078 0.1471879024061932 0.0627945417147464 0.1496863614010826 0.1459619234785302 0.1491364717080702 0.1479476495720966 0.1483998008688426 0.150413862473184 0.238799646651136 0.0625262881850598 0.063218615791918 0.0933544036048056 0.064970561178168 0.0621205546081258 0.0654063593425504 0.241468433957488 0.1483142167018904 0.239528616389224 0.238779401272568 0.232678712167196 0.208869949733124 0.05447367917606802 0.241612061461784 0.1492568493109948 0.0555181913739004 0.0552648961626604 0.20892703134707796 0.281718427972998 0.0900471415586588 0.0337930546601588 0.0664478160298096 0.0329389155557384 0.0891077134496244 0.0656370489222158 0.08931698626144431 0.208361387254362 0.0564557686644794 0.056420402692208 0.0202558485246748 0.284045459450312 0.0587928074209008 0.056004008279127 0.026898830377655204 -0.00495169596351754 0.01229973965713662 0.20655873785406803 0.020192831021579996 0.2843214817087081 0.280083148893362 0.27773191393445607 0.2798609747732399 0.27602638029562593 0.05643123118335099 0.06584631450471441 0.21074206524017994 0.05660194781432941 0.053307315860718005 0.06635562646842057 0.04553840287769161 0.27213055811224796 0.0664183706715196 0.2803093361786858 -0.09063493929697661 0.8659677707769978 0.8659677707769979 0.05755191713434759 2.0000099999999863 0.208667223529078 0.0659634861709064 0.05413067176261159 0.05330730675692161 0.01666950863423511 0.20417077806973996 0.20809194017507593 0.06568672933489661 0.056815179698219996 0.282211935065656 0.1549835337670454 0.06554771097327419 -0.021733235137448795 0.28120060353404414 0.01348469144920222 0.05533630174548979; -0.0646608612800804 0.0665320380814448 -0.436986939978444 -0.09734176080594861 0.207550851054416 0.20819088721302803 0.20850871861881 -0.1682870475438822 1.51590296816022 1.5200967318501841 1.5203459496447522 1.647785171008176 0.291536959020652 0.674259135632548 1.522504680650362 0.0762389907652934 1.522180193282318 1.5220478377386981 -0.530371597544732 1.5244492927341038 1.5231077195424203 1.52343483519006 1.5220722485937557 1.5265096217251017 1.5204494106271038 1.655605420815302 -0.53043663807865 -0.531079899512276 0.6945648621841339 -0.5085206968737599 -0.52762632454297 -0.5082720980496938 1.64429175227479 1.5176295351636682 1.6401387711737798 1.656051638397606 1.6615575411537915 1.9312528993922518 -0.5464569138583881 1.6159237267910276 1.5665291570897426 -0.564703669986622 -0.5643663946512201 1.962580589846696 1.4574202287488018 -0.79651731381761 0.13147159939939743 -0.5092449226078761 -0.13257876340553565 -0.7945277292122038 -0.5098381290834039 -0.79423771722076 1.984862121399278 -0.5720078897167516 -0.571336877740404 -0.1574062752954192 1.471391689845576 -0.57106908003037 -0.462931885337558 -0.063002457963004 -0.052988867335089 -0.216873311158612 1.972732985847546 0.421562527888518 1.470211982520462 1.441310530035766 1.446582517490686 1.4455530715948999 1.4476825645189981 -0.5713460848253301 -0.510473215832702 1.9732248700827122 -0.570863492623234 -0.5200815005170022 -0.5085991730907519 -0.276476963367612 1.603882972651712 -0.5119435995394439 1.474933775229876 -0.037782101138417186 0.061307279635754414 0.061307279635754414 -0.5748640790586563 0.208667223529078 2.0000099999999943 -0.51177694392821 0.4812150102986698 -0.5696740710070118 0.17835737439933524 1.9554608398768438 1.987394271585284 -0.5093323081437261 -0.5722501352000301 1.477902295523328 0.94394114855667 -0.5070653950567322 0.17504801083748345 1.477191076880286 0.0378248106015082 -0.513695433407204; -0.0135256684462829 0.751201427081112 -0.048178177905395206 -0.0565935214556978 1.061308064455734 1.05515828468904 1.054453544964748 -0.07008924414831721 -0.406755861875524 -0.404453651935646 -0.4048158324524 -0.5824661127662 -0.06782337755986262 -0.17716023501258177 -0.406280949123858 0.0289286227162354 -0.409149812718966 -0.404500219117898 -0.133481204533166 -0.405017402344258 -0.406677912202316 -0.409187289857266 -0.40551087874996 -0.4060440680579 -0.40392107455832 -0.582854348707188 -0.1320279773332634 -0.1279651593457644 -0.176482181418827 1.9818137826147537 -0.1290450495288518 1.981908700764864 -0.583165597608524 -0.406426070722958 -0.57794499157626 -0.582228123238914 -0.585201492223706 -0.511233913024244 -0.13100333580770565 -0.58139931117074 -0.398187421574784 -0.1411308565255088 -0.1426594155667312 -0.505252382846262 0.374101428720424 1.229687998494256 -0.0406214108600942 1.990329290286602 -0.0383417348204846 1.22517769820595 1.98136818872701 1.2301470623803241 -0.504580724645288 -0.1468959182714628 -0.147134485940281 0.00816490399673332 0.369923916970132 -0.1449616099860712 -0.1298409220214598 -0.029620197315951 -0.0417823895545906 -0.0312643808849204 -0.505674831261566 -0.1139983567908934 0.36803781337066405 0.37049858626626997 0.369722841037106 0.368648296362424 0.366036486990394 -0.1482788015481198 1.990716304323878 -0.506611183842518 -0.14722300554412143 -0.13656015702411278 1.987114586580358 1.0789371229285483 -0.5714443810217498 1.9946363666366724 0.3705661067967899 -0.011496123929142697 0.035401187970648396 0.035401187970648396 -0.1474222701224136 0.0659634861709064 -0.51177694392821 2.0000099999999965 -0.14518924706460168 -0.1474288554053056 -0.047938238929256596 -0.5038536033031402 -0.5064599343092822 1.9846251833716912 -0.14723290823592386 0.3664491145040738 0.6304287672892959 1.9852844701802965 -0.038091759236421406 0.36353107165178394 0.005701998814512999 -0.11013760768576403; -0.1096437696703938 -0.0271573350932232 -0.320595116464426 -0.0693476318335304 -0.23771974550859004 -0.238629060598566 -0.234900407280784 -0.0427362268329288 0.595496064083608 0.59344143449037 0.595473952687648 0.397461191499696 -0.0856499496161806 -0.1791046082143352 0.597530702524484 0.01573554075655426 0.594988521758998 0.593508205496476 -0.138558767337928 0.598508610110518 0.593652017583916 0.594792252606254 0.601719747377386 0.597522369921072 0.594414560866334 0.40202826721938395 -0.138904813837836 -0.1394162419279198 -0.178807315553719 -0.1489096957946818 -0.1384901104839494 -0.1490036259124158 0.400848905604026 0.591424523971966 0.398889417735992 0.401103031275986 0.402792840554106 0.47240330222454 -0.1376322756838568 0.411000601192354 0.611222336856568 -0.1396274703584742 -0.1409434349592922 0.473675044573168 0.335942266865226 -0.208243155665892 -0.04504309891681241 -0.1412453601732004 -0.01793464844525252 -0.206050367088222 -0.1443177380430706 -0.20647482579374807 0.47621729336608 -0.1394911029681674 -0.1394100682551164 -0.0305716923718298 0.344887912997646 -0.1408040214641438 -0.10322384914782 -0.0397209288552532 -0.019061384634937457 -0.0489063711176948 0.482257400402446 -0.11198772332773341 0.34468470886829794 0.33671271109215994 0.33839913727568804 0.33645749224592203 0.337051123989394 -0.13946824916980516 -0.14572694360094843 0.48095016847599 -0.13912355822157793 -0.11737101556731902 -0.1453781793797488 -0.065976452749006 0.39527789309559 -0.14578579326116176 0.3478655560643879 -0.012133499329214802 0.011959307631615622 0.01195930763161562 -0.1426625394869474 0.05413067176261159 0.4812150102986698 -0.14518924706460168 2.000009999999996 -0.14376087615409336 0.70311574652198 0.47565196680340993 0.477083333649574 -0.14405353695428408 -0.14010901360099137 0.3432280688096361 0.06720458328144165 -0.14345462341194723 -0.009814977841462028 0.3430858904035761 -0.0019938498974191535 -0.1222788230044812; -0.448171405818764 -0.1817812968986856 -0.294568656708296 0.416585268652806 -0.261972567381606 -0.260791757226894 -0.261961677728648 -0.060379462873831204 -0.303326692980216 -0.308762122926318 -0.30813203653595 0.5082072535826863 -0.08145159118893142 -0.19595209721529183 -0.309697017371476 0.0282805088594576 -0.306066677253456 -0.309795894635648 1.804944302035034 -0.308825655924054 -0.308128232062922 -0.309803460269006 -0.309159578859924 -0.308134317818992 -0.304604850760754 0.5175595701057039 1.805051114256342 1.805699456735256 -0.1978498848917368 -0.1444125593557128 1.794279981111848 -0.1441787310739906 0.514830706283714 -0.30743359345952 0.521258751171062 0.518021607627294 0.50703651030705 -0.474062854487296 1.861390357513022 0.526814731817954 -0.44818499769445 1.940917631452368 1.940535540702896 -0.562392990733096 0.473435550076498 1.456175451805384 -0.0597436291849122 -0.1469262538623256 -0.0633819959841098 1.454678849715994 -0.144478031988149 1.4504156450621113 -0.568696225680212 1.969498774158258 1.9702550725779 -0.0439451574813502 0.468078539787218 1.95970935083274 -0.324355160686022 -0.0468138338878416 -0.0516799694138918 -0.0780424894224618 -0.5626206745868139 -0.12323621091136061 0.46740194368809407 0.46336372458267605 0.457522806876398 0.46068565658070404 0.46046195959362607 1.9679280153492802 -0.1465002785027834 -0.5683711280946723 1.9681724561705918 1.77751177517137 -0.14515929146638462 -0.09361426464624621 -0.66311825885919 -0.14664156948823434 0.46830390481623796 -0.004525006336265559 0.058032556702601816 0.05803255670260182 1.9831919026389426 0.05330730675692161 -0.5696740710070118 -0.1474288554053056 -0.14376087615409336 2.000009999999996 -0.054469990264619204 -0.557890040328294 -0.5653997588657441 -0.144693128059793 1.9686278333563436 0.46567638998116206 -0.12279907237407267 -0.14431662472769993 -0.08061331542931677 0.4654922966303441 -0.10922553917342878 1.7537712221794375; 0.1383746585817472 -0.0608751360044052 -0.0868277235701932 -0.0281720937682352 -0.061255874080214 -0.0606160924503232 -0.0570105583201274 -0.00466386764147392 0.1972451235934018 0.19962408505252 0.1982277924745446 0.1422527976517972 -0.0352259731454138 -0.036919083809066 0.197843409189512 -0.005679960334690259 0.198539383507327 0.1985221399112734 -0.0501138688057602 0.198830791909921 0.1994287447064488 0.1996428445638304 0.201153979939914 0.1995112613280192 0.1972714830026484 0.14569312527046463 -0.050363016860799 -0.0504361600313696 -0.038724431133398 -0.0475769581935204 -0.0499742521820926 -0.0474947931737718 0.1475130747499844 0.201306382652476 0.1492454953938666 0.1457541290042128 0.1464464615406284 0.1698206791029694 -0.04707258093433821 0.153351319328634 0.207687847368804 -0.0537062426980142 -0.0535244061009766 0.1774744568135972 0.1279839272234792 -0.0758730178398276 -0.025692310350150196 -0.048311367695861 -0.003809285337894961 -0.075825869016405 -0.0476763154041928 -0.075985590407111 0.1759977906947854 -0.0543000384597742 -0.054282630655983 -0.0261959331666698 0.1284460273766004 -0.0514651101881518 -0.0019529487045342984 -0.010745687813702802 -0.0087945190731026 -0.0365666043281592 0.1741459616125022 -0.054701188976105014 0.12876633687107777 0.121596881287375 0.12364924237559279 0.12326919269234901 0.11955859300508763 -0.05450894088375661 -0.048199168708744605 0.17779798315145437 -0.05446447809488019 -0.0437227987931658 -0.04744777372197259 -0.0333909785560204 0.14755887774224763 -0.048291464212741206 0.12905702511721234 -0.00579429799621522 -0.007741230265143741 -0.007741230265143741 -0.055119044488297984 0.01666950863423511 0.17835737439933524 -0.047938238929256596 0.70311574652198 -0.054469990264619204 2.0000099999999903 0.16761852545231218 0.17252629839472963 -0.047712322950803215 -0.0545405443711074 0.12522128378222866 -0.005434695494365142 -0.0469630007684684 -0.034718157960390995 0.12676329390740004 0.009180930635316102 -0.05199795438023119; -0.06658276095457862 0.073088602990442 -0.42892315280782595 -0.1002801230294158 0.20613085090873398 0.20694732976684999 0.207294509810786 -0.1654087788765672 1.4903536087235763 1.493377250587152 1.49447918555575 1.624219460499614 0.291444951480594 0.667568255092486 1.496830949509252 0.07563355806104957 1.4958574815117218 1.495617098387878 -0.5203559855888481 1.4974206320368662 1.497030068428902 1.496210254943366 1.4949569995062657 1.4984574976727918 1.4943526078968659 1.62892617964103 -0.5206947098521442 -0.5213465091032539 0.6877940393658138 -0.5013602473603598 -0.5176439569039979 -0.5012378393163119 1.6197482793403717 1.4927426712990124 1.615322459264464 1.6310419964984202 1.6366452362333597 1.901019558024684 -0.5350657230374819 1.589866329569358 1.536999428021134 -0.552532987024134 -0.5521904119178421 1.9320390224676183 1.4354287350216817 -0.7812653160959236 0.12407082874085362 -0.5022223501970681 -0.1361562927387514 -0.7794333860806602 -0.5039106661678622 -0.7791258211082801 1.9522191209230262 -0.5598588082726419 -0.5591863271864259 -0.157539092178086 1.44773850035016 -0.559780805327092 -0.467296379543784 -0.0646094442694592 -0.0606706915394658 -0.22140958076900397 1.944789705513934 0.414580387530558 1.4479606556014941 1.423564764110762 1.4279390027102918 1.4260360546064783 1.4290365184148044 -0.5604554115813679 -0.5034774503835859 1.9400211825372602 -0.5588387973096601 -0.5180893730704221 -0.5002031817081841 -0.27674593838201794 1.579768650401006 -0.5051121078673402 1.4524480102287158 -0.037827866694828814 0.06057424398394181 0.06057424398394181 -0.5618291916437722 0.20417077806973996 1.9554608398768438 -0.5038536033031402 0.47565196680340993 -0.557890040328294 0.16761852545231218 2.000009999999992 1.95856967555526 -0.5033885687866239 -0.560131229223156 1.45778715613996 0.9396136330123099 -0.500200801596274 0.16245135195773958 1.4586753189154027 0.04396609018024761 -0.5134293894922981; -0.06919369778353861 0.0719655345204688 -0.44029000748224 -0.0967403331521768 0.209940926902878 0.210583221327394 0.210904027398708 -0.16780742046147978 1.5150653157754357 1.520218345170022 1.520773375632126 1.6500240973017757 0.2920279268210881 0.673678160222254 1.522925701018232 0.0759681917408366 1.522599185898856 1.522463534674872 -0.526973184767034 1.523796131659676 1.522453795756958 1.523443210640456 1.521383348957504 1.525822967641014 1.520356930156844 1.655817918192532 -0.527177746917576 -0.527820762681474 0.693945064216124 -0.504127567243308 -0.524354828087266 -0.503880048864598 1.64546998197322 1.518014750423182 1.64185402625609 1.657272030686004 1.662762587685308 1.929695286473376 -0.541374252222438 1.61604802500608 1.565710198839516 -0.562293025219372 -0.561956506092408 1.961965166819164 1.461131253057908 -0.790175138009394 0.1280756852537348 -0.504865163614788 -0.1356814404169924 -0.788180090424784 -0.506540425161886 -0.7891252989373283 1.982689847487534 -0.567730113827008 -0.567061029813412 -0.1570331668898934 1.474831780735466 -0.566780982798976 -0.46519740025474804 -0.0627659203551572 -0.05904187157949059 -0.217269708198564 1.97107740063401 0.42224156989252193 1.4724433206588379 1.442536148311888 1.448853573846348 1.44765628138624 1.4504375212716538 -0.567244433102126 -0.5060905868185019 1.9706247764907043 -0.5667299763331459 -0.5150749518487661 -0.5052597711049723 -0.27789312571248614 1.6009721704930862 -0.507566938451382 1.4783039278871837 -0.03776549029405579 0.06105223557937199 0.06105223557937199 -0.5705884917813139 0.20809194017507593 1.987394271585284 -0.5064599343092822 0.477083333649574 -0.5653997588657441 0.17252629839472963 1.95856967555526 2.000009999999992 -0.5060452861221464 -0.5679793306939499 1.4841448401750954 0.951173548699032 -0.50266519996914 0.17538937488799003 1.4827621056814762 0.04114781204973618 -0.5091921161292103; -0.016173461376342905 0.746523496229862 -0.0487941473169642 -0.056307809007697995 1.058999386783456 1.052813386456216 1.052111173415842 -0.0698825467395196 -0.405682180727742 -0.402504533943598 -0.403724554028012 -0.579511171480682 -0.067501007662499 -0.17651723542974357 -0.40632460748536 0.0287854893129932 -0.408233530110672 -0.403416974860448 -0.1306644107051204 -0.403933619176 -0.404831725590828 -0.408109419341696 -0.404433223660288 -0.404967358842252 -0.405073029838674 -0.5799059437711079 -0.1292053477717174 -0.1251352776958136 -0.1740761010625834 1.97001859053617 -0.1262075445983674 1.970113762381014 -0.580319472601804 -0.405509491535552 -0.57610709093152 -0.580393125980884 -0.58325207657055 -0.510776472584546 -0.12821251591342558 -0.578643422241362 -0.39958433463678 -0.1380796575793176 -0.1396107389305284 -0.505892846131596 0.372585652454192 1.226995828549094 -0.040452252112978 1.980994390318452 -0.0381686522919848 1.22246919285668 1.971865630083698 1.227449288479364 -0.502124763132144 -0.1441585516357384 -0.144398788325435 0.00843148259562622 0.367462715184346 -0.1422130703514314 -0.131564378346175 -0.0295157122579552 -0.041729173669620395 -0.0305012865131518 -0.5052570630294881 -0.11320107702313578 0.36626796224021396 0.36968924341316406 0.36891939040865807 0.36679375432521005 0.3643439976210279 -0.14573747875685242 1.9836511298951358 -0.506342859284426 -0.14448312228558657 -0.134624118303168 1.9782212384353897 1.0777539233919378 -0.5703855799307059 1.987600442873208 0.37018598508492007 -0.010470265047675183 0.0352775792533054 0.0352775792533054 -0.14468752712681274 0.06568672933489661 -0.5093323081437261 1.9846251833716912 -0.14405353695428408 -0.144693128059793 -0.047712322950803215 -0.5033885687866239 -0.5060452861221464 2.000009999999994 -0.1445010872438284 0.37177747155897195 0.639079283342328 1.9851079363474309 -0.03781383770335758 0.368870240492716 0.00554323037015574 -0.11064327764583382; -0.4466430160012221 -0.1866631885113028 -0.291214930177112 0.425183371144186 -0.260352236871198 -0.259184057465806 -0.260345815139686 -0.0594918259287324 -0.31161763852396196 -0.316965567438228 -0.316291566971188 0.5118355702906461 -0.08332556726977061 -0.19399255889168376 -0.31666111630402 0.03595056528119139 -0.314136303651148 -0.317989266234812 1.826094449725946 -0.317004735986794 -0.316328302056372 -0.318011916673898 -0.317138913671374 -0.316293904023194 -0.31983511613833 0.515145282077026 1.826171090762884 1.826816112144736 -0.1959877063985404 -0.144550801094908 1.815544253098556 -0.144280983103633 0.512941506847184 -0.321546282118654 0.51776448507678 0.515467774968694 0.515181942721112 -0.477163667491836 1.8823029707258079 0.52239856843839 -0.452966177489756 1.963127186092696 1.962756639363024 -0.564679578909644 0.470677916604308 1.455534446929108 -0.0591149578707866 -0.1467174756876938 -0.062784619483811 1.454408060346454 -0.1442909857206804 1.4501436401587986 -0.57033200724324 1.991595970719202 1.992345655989906 -0.0432736431995274 0.465263151362778 1.980627729703522 -0.32370403228861994 -0.04610033348590759 -0.0511136487973834 -0.08214113900994338 -0.5642984419694699 -0.12646266519121158 0.464577538468574 0.46116396621938605 0.45555455455323995 0.45798822275450995 0.45775888910197404 1.988017687018912 -0.14629317374745965 -0.569046066230392 1.9885417134049161 1.7939242502772519 -0.1449926348026172 -0.092452074821497 -0.665541795451042 -0.1464319322981996 0.4634226718474859 -0.0045309728721297 0.05756159433186861 0.05756159433186861 1.979222381857852 0.056815179698219996 -0.5722501352000301 -0.14723290823592386 -0.14010901360099137 1.9686278333563436 -0.0545405443711074 -0.560131229223156 -0.5679793306939499 -0.1445010872438284 2.0000099999999916 0.464771800148942 -0.12596947204162795 -0.14416616102676882 -0.0807593230464286 0.4646150675960102 -0.10961020992913802 1.7772353001152097; -0.32939823714894406 0.336561530697284 -0.640334558063286 0.1097243996781948 0.5864139687516101 0.584862181320432 0.584003973852642 -0.238732454231068 1.191516932349192 1.195457942677812 1.196228074013144 1.70902571950142 0.22004882313751997 0.499856709287212 1.197072099595568 0.1087511170941968 1.197105495661432 1.197267581844066 0.419846304382694 1.19885632503825 1.198551529930858 1.196189837664424 1.194639802106298 1.200560938863816 1.196672849493984 1.721050537390214 0.420393462801888 0.422099811054802 0.52081693202413 0.363382382427476 0.418691867677488 0.363818171378112 1.708750525344098 1.192895280404066 1.709378778992802 1.72215462845273 1.720250938061888 1.473776349662092 0.4359110833503081 1.68522771091316 1.165872822430296 0.457838193215034 0.45721605225773 1.459877989656672 1.962483073772432 0.61437784803479 0.0764340117563976 0.366596892583384 -0.1952519862920242 0.613364989755986 0.361748397937188 0.6136934994254681 1.480361698562358 0.464987123015272 0.46472337949046 -0.1824908063438662 1.97022519300221 0.459237661948304 -0.7250602616120061 -0.10538430553128939 -0.1114962988827454 -0.285942465636952 1.470061151855948 0.30563471101771006 1.9673484014824 1.9366365002186483 1.9390802944964862 1.938109883892402 1.939666535930796 0.4614956725927559 0.367053849258812 1.4643464474080723 0.463048222150206 0.4102557225744521 0.36786076974689785 0.19373339203374804 0.9978496632892521 0.367226004638758 1.9755105144357292 -0.046905756683545595 0.11344019233828921 0.11344019233828921 0.46695619926574 0.282211935065656 1.477902295523328 0.3664491145040738 0.3432280688096361 0.46567638998116206 0.12522128378222866 1.45778715613996 1.4841448401750954 0.37177747155897195 0.464771800148942 2.0000099999999916 1.2259159429238384 0.372543893580276 0.11743655517196262 1.9813939665438505 -0.019294763063559577 0.41753752108747205; -0.064360895530088 0.49020926995112 -0.30164682005889 -0.032331302492839 0.805633630741486 0.7986172294709502 0.797665402786024 -0.1488291023275346 0.5862623300867079 0.590609754593264 0.589978010355654 0.8615903627838781 0.08001625332639281 0.5513900546005861 0.58768702403468 0.06105827930297221 0.59108965781002 0.591661107795694 -0.12528899290495998 0.592464664345306 0.5905319730322601 0.589637869490564 0.590542085013952 0.592480908386968 0.5876530619075481 0.8696727551285481 -0.1245824632667152 -0.12202617876396697 0.5654107407499022 0.628580229437 -0.12498564903832744 0.628947299664364 0.861490218473646 0.588441190153452 0.864002364573614 0.8714018851289118 0.8695194185873423 0.923724923291038 -0.12335573877387418 0.8477095628469401 0.583367245484936 -0.12697336702468004 -0.12722590476450318 0.9348872925605141 1.2095438373855119 0.3257377333616759 0.010573541986831952 0.6334174157904919 -0.12668860620181466 0.3291226440275499 0.6277523190558977 0.33129736260302994 0.9463754184952281 -0.12800271025893145 -0.1261881121937812 -0.09656046081158177 1.21198103561726 -0.1266118536958314 -0.459787761138444 -0.0570548963629228 -0.0814986906685168 -0.16865745814026004 0.942167981732462 0.4609082604532859 1.213521291613302 1.2032768751258518 1.205360671910126 1.2036053197759362 1.2034027391109439 -0.12716384340211057 0.6335747096085838 0.9353438183491599 -0.12656746480529346 -0.15700511683916318 0.6325925800708841 0.340162044078706 0.651111146794944 0.6325935049769343 1.216040346689314 -0.038930799026916596 0.06294255897656241 0.06294255897656241 -0.12737590975623542 0.1549835337670454 0.94394114855667 0.6304287672892959 0.06720458328144165 -0.12279907237407267 -0.005434695494365142 0.9396136330123099 0.951173548699032 0.639079283342328 -0.12596947204162795 1.2259159429238384 2.000009999999996 0.6370300075247477 0.16903281289328997 1.2213788709360742 -0.011956509451143692 -0.13350367596198837; -0.01685424922853526 0.75033326657725 -0.04815402792640701 -0.0561392302130172 1.05555549172241 1.04929149028255 1.04865492730522 -0.069824084170376 -0.404900117588566 -0.40256596658829 -0.402951924961782 -0.579011428933174 -0.06683349510931941 -0.17768494245422434 -0.404405298504174 0.028702293245968003 -0.407290002699554 -0.402651951164144 -0.1299684563332714 -0.403138474658852 -0.40419786268809 -0.40752145881418 -0.403642856708664 -0.404212673602654 -0.402048544152328 -0.5792690857423259 -0.1285358906586236 -0.1244488823307284 -0.1751002819603332 1.9729184380644837 -0.125525435869751 1.973013579150226 -0.579823608089042 -0.404718134531666 -0.574484934562096 -0.578789742909284 -0.581670560668656 -0.509575724337924 -0.12752920420474423 -0.57822721601025 -0.396457415919902 -0.1373925245580642 -0.1389308586504762 -0.503590170612106 0.373353679836216 1.22748292610101 -0.0403679424129154 1.979309002290024 -0.0372159875066744 1.220253893530348 1.970083069629284 1.225204810244262 -0.50077239804683 -0.143496976821784 -0.1437387572270666 0.0095838765743278 0.369203387220314 -0.1414302314418952 -0.1319857004388448 -0.0296111033789492 -0.041878447669008806 -0.029462374050117403 -0.501983782641092 -0.11155823346622 0.36688628009176194 0.3696174607459601 0.36868525000464997 0.36748749707590594 0.3650247211899381 -0.1448468362372972 1.979709274765164 -0.503999119338702 -0.14382113575382222 -0.1366550028000658 1.97417935395075 1.0825870066546859 -0.56904430912726 1.983667751364644 0.37094000314606584 -0.010512689717493163 0.03522842844878918 0.03522842844878918 -0.1440230934161526 0.06554771097327419 -0.5070653950567322 1.9852844701802965 -0.14345462341194723 -0.14431662472769993 -0.0469630007684684 -0.500200801596274 -0.50266519996914 1.9851079363474309 -0.14416616102676882 0.372543893580276 0.6370300075247477 2.000009999999997 -0.037586621690695786 0.3696657407054241 0.00013938635583955142 -0.10990453064144884; -0.124262727410146 -0.01683495726510172 -0.0811733386327222 -7.300065243528303e-5 -0.0632336227436086 -0.0660514412792494 -0.0663286960281528 -0.0267016352728282 0.18687715764013618 0.1883113005860716 0.1937048435259516 0.1274217421708584 -0.02109003423363479 -0.0499656224258618 0.1973190383158844 -0.024457371719871602 0.200996009235316 0.201045650669792 -0.0751937727995896 0.200609561665802 0.1976762712582686 0.1979031294666386 0.1961361770596734 0.20116230639445 0.1953054924036906 0.12826554330856602 -0.0754840685624906 -0.075561285276644 -0.046163876588734 -0.03761053861906081 -0.0706695717782838 -0.0375168769008068 0.1249698360870716 0.1962712797725604 0.1196319173534062 0.1259524632740562 0.1267644181653288 0.166391400641491 -0.07320525734866601 0.1157286760764876 0.20433253458195 -0.0801029934765542 -0.0799111333536114 0.169083285829049 0.110824089857002 -0.089817224979964 -0.0272905273291158 -0.0383328417736334 -0.026945210974867004 -0.089667982261575 -0.037684968233321 -0.08988564841627822 0.170366333463153 -0.080534871902349 -0.0805080639001646 -0.0283023230007698 0.1121509134123546 -0.079646796208893 -0.0224639486111336 -0.01030194581694262 -0.00843135037495124 -0.0014571799085781826 0.1688066768029012 -0.027387989628974596 0.11290906606492498 0.10691075847127418 0.10522141265135279 0.10674344624808162 0.10618117191806278 -0.08474752292650801 -0.0379358955451882 0.17180191232684378 -0.07993939795789462 -0.05468396718085958 -0.037783033042693204 -0.019553809583700805 0.14568835891482562 -0.0385414445104584 0.10813770910052518 -0.00809475973228204 -0.027519188898976605 -0.0275191888989766 -0.0862364691716162 -0.021733235137448795 0.17504801083748345 -0.038091759236421406 -0.009814977841462028 -0.08061331542931677 -0.034718157960390995 0.16245135195773958 0.17538937488799003 -0.03781383770335758 -0.0807593230464286 0.11743655517196262 0.16903281289328997 -0.037586621690695786 2.000009999999996 0.1142561461253948 -0.029017703147261993 -0.05156591780956058; -0.33204660783667006 0.337383941757628 -0.642895907398978 0.1103117180723438 0.584991201147052 0.582908079287208 0.582397629844414 -0.23413078891196198 1.1894014066549057 1.193318904772864 1.193510774492518 1.70651883667335 0.217062976756358 0.49890449858229 1.194303012527314 0.1095895288551842 1.194471220820864 1.19451209867197 0.419059597051312 1.197309241548844 1.197459291529198 1.194083498493086 1.193646425602126 1.198611619176738 1.196496800148334 1.7190013831551283 0.41948608983116 0.421192198565256 0.52061857438521 0.360629134239416 0.417793300000092 0.36080289008648 1.70783169666272 1.190873723885458 1.707375039792772 1.720098718842452 1.718653910761466 1.472070204286012 0.43661625453379993 1.68371525766402 1.16413924733856 0.457834810145048 0.457218816687704 1.457584183139066 1.958095387881362 0.612776557381158 0.0785681280045098 0.363670967756974 -0.1917152658696336 0.611337213911556 0.358821052893732 0.6116609609607443 1.4785572564331 0.463692123584816 0.464578407587312 -0.1844808518980442 1.96755144759429 0.459056180507968 -0.7268323290532839 -0.1068898845071442 -0.1117369002539144 -0.28584486413705396 1.468674672812962 0.30911603732790005 1.9650239370306797 1.9342952776682318 1.9359236756730323 1.9357390060369217 1.9373199952028302 0.46146802276024596 0.364262450655634 1.46409420733702 0.4631790410224519 0.41062728002870597 0.36509622266057606 0.19254071473442963 0.99701110032532 0.3643036574357739 1.97135886939797 -0.046969286572685384 0.1142510155901164 0.1142510155901164 0.46713980263624605 0.28120060353404414 1.477191076880286 0.36353107165178394 0.3430858904035761 0.4654922966303441 0.12676329390740004 1.4586753189154027 1.4827621056814762 0.368870240492716 0.4646150675960102 1.9813939665438505 1.2213788709360742 0.3696657407054241 0.1142561461253948 2.000009999999992 -0.018437596167690477 0.41776039807762805; 0.0032244107056488202 0.01741327091353462 0.0010025752387468458 -0.0374117153258862 0.0168377906904345 0.01613802078865866 0.01590218371426646 0.0269672535774878 0.01721751328352808 0.0197610053166223 0.0216194375699942 -0.021290877527545014 0.002005820224475162 0.01868054853043164 0.01618769324443572 0.020925224259080805 0.0228598306337352 0.0205651561207762 -0.0980700949526656 0.01641282179148042 0.01584974872641562 0.01612449823957136 0.0219971398117882 0.01786229921844434 0.0210325982875448 -0.02096237047510879 -0.0993557980271658 -0.0993407599502302 0.0206475256537512 0.00629744602598834 -0.0987979347563288 0.00634994975325094 -0.0228189464283204 0.01766044221268382 -0.0178569913185381 -0.02045235248145 -0.01933387233622986 0.0341089006047072 -0.10214050339066662 -0.01882764934318234 0.0297037472687006 -0.1090313066246154 -0.1089089478694214 0.0390698458601614 -0.0218770717678356 -0.08392503073335 -0.00433393971496532 0.00527106915321058 -0.019787918762358442 -0.0834122628372142 0.00604250836605052 -0.08322860707699539 0.041324323803675 -0.110210500924414 -0.110258463726755 -0.0252831253583272 -0.01948300787014646 -0.1092904525499058 0.007260204377229861 -0.00199825153374103 0.00151542714222769 -0.023810788094523 0.03823928058258999 -0.00675851028492072 -0.021133579603573798 -0.019525557850490236 -0.01929289546047871 -0.0211405535944146 -0.020497501217292803 -0.10930930452589241 0.00591041700117302 0.03395183526526759 -0.10983304241949599 -0.05970823806396601 0.007369774181492303 -0.0220570287166852 0.0398076452107832 0.005194419994507921 -0.016613765454730842 -0.005642143669026776 0.018734389084503686 0.018734389084503682 -0.11111449404016502 0.01348469144920222 0.0378248106015082 0.005701998814512999 -0.0019938498974191535 -0.10922553917342878 0.009180930635316102 0.04396609018024761 0.04114781204973618 0.00554323037015574 -0.10961020992913802 -0.019294763063559577 -0.011956509451143692 0.00013938635583955142 -0.029017703147261993 -0.018437596167690477 2.000009999999996 -0.15392678152635203; -0.38722448208980204 -0.1660792662931726 -0.259461694289858 0.389404461035444 -0.219905256884248 -0.219496730966528 -0.220125444507318 -0.05577946427732379 -0.28217547053008596 -0.285656203970554 -0.28536830894634 0.45098803299310797 -0.0708779019326876 -0.17573070320781242 -0.284894538351428 0.03346623902027839 -0.284429639590232 -0.286771066839516 1.6270004419867403 -0.285332057727932 -0.28714198230456994 -0.287099986157324 -0.283913886066368 -0.285509089009056 -0.28765099904197816 0.4537807845609761 1.6305097589460942 1.6281160475285805 -0.17817015041735054 -0.10866643198655045 1.610724179632448 -0.10848662984212973 0.4511873871118 -0.289132046526306 0.45471402500238 0.451885603855138 0.45244602959192426 -0.43269842589839 1.679394551511678 0.4604185258028704 -0.405885625714192 1.7393725734387624 1.7384613293263858 -0.507235088715774 0.42271772898568 1.3046243528607375 -0.055265050974238 -0.11127797448819722 -0.05825823063450618 1.3040828623775158 -0.10981174390878928 1.29947572686534 -0.5127152362350541 1.7699269715024162 1.7705818650907041 -0.0339506271250278 0.416673366923584 1.768940187141716 -0.334416303902004 -0.04677833318723561 -0.0426433389971806 -0.089076624597975 -0.50723519540999 -0.102138084920671 0.41785231618795193 0.4171295463633901 0.4109878077298259 0.411328659071288 0.40980053813913997 1.7676585962848719 -0.11148791274569822 -0.511908694468068 1.7772994896382501 1.7177355497773505 -0.1085869650916284 -0.06068972643896843 -0.602023185789694 -0.1111285152005986 0.41291821280182606 -0.014075253242538256 0.05245165751045441 0.05245165751045441 1.7587684265399364 0.05533630174548979 -0.513695433407204 -0.11013760768576403 -0.1222788230044812 1.7537712221794375 -0.05199795438023119 -0.5134293894922981 -0.5091921161292103 -0.11064327764583382 1.7772353001152097 0.41753752108747205 -0.13350367596198837 -0.10990453064144884 -0.05156591780956058 0.41776039807762805 -0.15392678152635203 2.0000099999999956]
L = cholesky(Symmetric(A))

x = zeros(100)
x[10] = sqrt(abs(δ))

Lcopy = deepcopy(L)
xcopy = deepcopy(x)

@time lowrankupdate!(L, x)
@time lowrankupdate_turbo!(Lcopy, xcopy);

  0.000012 seconds
  0.000011 seconds


In [147]:
all(L.factors .≈ Lcopy.factors)

true

In [148]:
all(x .≈ xcopy)

true